In [ ]:
!pip install gradio
!pip install anthropic
!pip install langgraph langchain langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.3/286.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import pandas as pd
import tempfile
import glob
import os
from anthropic import Anthropic
from openai import OpenAI
import re
import traceback
import io
from contextlib import redirect_stdout
import numpy as np
import logging
import json
from typing import List, Optional, Dict, Any, Union
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import StateGraph, END

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('rent_roll_analyzer')



In [ ]:
from typing import TypedDict, List, Optional, Union, Dict, Any

# Define the state as a TypedDict
class AgentState(TypedDict, total=False):
    messages: List[Dict[str, str]]
    df: Optional[pd.DataFrame]
    issues: List[str]
    execution_plan: Optional[str]
    needs_clarification: bool
    clarification_question: Optional[str]
    generate_code: bool
    code_execution_results: Optional[str]
    final_response: Optional[str]
    anthropic_client: Optional[Any]  # For Claude API
    openai_client: Optional[Any]     # For OpenAI API


In [ ]:
def read_rent_roll_simple(file_path):
    """
    Improved function to read rent roll Excel files that handles special formatting
    commonly found in commercial real estate rent roll sheets.
    """
    # Read the raw Excel file with no header
    df = pd.read_excel(file_path, header=None)

    # Find the row containing the column headers
    header_row = None
    for i, row in df.iterrows():
        if row.iloc[0] == "Current":
            header_row = i + 1  # Headers are in the row after "Current"
            break

    if header_row is None:
        logger.warning("Could not find header row with 'Current' marker. Falling back to standard loading.")
        return pd.read_excel(file_path)

    # Get the headers
    headers = []
    for val in df.iloc[header_row]:
        if pd.isna(val):
            headers.append("NaN")  # Use "NaN" for empty header cells
        else:
            headers.append(str(val))

    # Create a new dataframe starting after the header row
    data_rows = df.iloc[(header_row+1):].values

    # Create a new dataframe with the extracted headers
    result_df = pd.DataFrame(data_rows, columns=headers)

    logger.info(f"Successfully loaded rent roll with {len(result_df)} rows using specialized loader")
    return result_df

In [ ]:
def analyze_rent_roll_gpt(file_path, api_key):
    """
    Analyzes a CRE rent roll Excel file by sending the data rows to GPT-4.
    """
    # Load the rent roll
    try:
        df = read_rent_roll_simple(file_path)
        logger.info("File loaded successfully for GPT analysis.")
    except Exception as e:
        logger.error(f"Error loading file: {e}")
        return []

    # Initialize OpenAI client
    client = OpenAI(api_key=api_key)

    # Convert the DataFrame to CSV string format
    csv_data = df.to_csv(index=False)
    logger.info(f"Converted DataFrame to CSV with {len(df)} rows and {len(df.columns)} columns")

    # Enhance the system prompt to focus on general rent roll issues
    system_prompt = """
    You are a Commercial Real Estate rent roll expert specializing in identifying data quality, formatting, and consistency issues.

    When analyzing any CRE rent roll, rigorously check for these common categories of issues:

    1. DUPLICATE OR REDUNDANT ENTRIES: Look for any repeated charges, fees, or line items
    2. INCONSISTENT TERMINOLOGY: Identify any unclear, non-standard, or ambiguous descriptions
    3. DATE ANOMALIES: Flag any suspicious or illogical date patterns across move-in, lease start/end
    4. RENT DISCREPANCIES: Identify deviations between market rent values and actual charged amounts
    5. CALCULATION INCONSISTENCIES: Check if component charges properly sum to totals
    6. EXCEL ARTIFACTS: Identify any visible formulas, function calls, or spreadsheet mechanics
    7. FORMATTING IRREGULARITIES: Notice inconsistent data entry patterns or splitting of information
    8. BALANCE ANOMALIES: Identify unusual balances, especially negative values
    9. OCCUPANCY MISMATCHES: Look for occupied units with zero rent or vacant units with charges
    10. UNIT IDENTIFICATION PATTERNS: Check for inconsistencies in unit numbering or identification

    Be extremely thorough and specific in your analysis. Report ALL issues you find, regardless of how minor they may seem.
    DO NOT return "No issues detected" unless you've comprehensively analyzed the data for each category above.
    """

    # Use a simplified prompt focused on analyzing the raw CSV data
    prompt = (
        f"Please analyze this Commercial Real Estate rent roll data in CSV format and identify ALL potential issues "
        f"that could affect data quality, accuracy, or decision-making.\n\n{csv_data}\n\n"

        f"Based on your expertise in CRE rent rolls, provide a numbered list of ALL issues you can identify, including but not limited to:\n\n"

        f"- Any duplicate or redundant charges\n"
        f"- Unclear, non-standard, or inconsistent descriptions\n"
        f"- Suspicious or illogical date patterns\n"
        f"- Inconsistencies between market rent and actual rent values\n"
        f"- Calculation errors where components don't match totals\n"
        f"- Spreadsheet artifacts like visible formulas\n"
        f"- Inconsistent data entry patterns\n"
        f"- Unusual balance values\n"
        f"- Occupancy status mismatches\n"
        f"- Inconsistent unit numbering or identification\n\n"

        f"IMPORTANT: For each issue found, please reference the specific unit(s) affected and explain why it's problematic. "
        f"Be comprehensive - rent roll accuracy is critical for CRE investment and property management decisions."
    )

    try:
        logger.info("Sending request to GPT-4 for analysis...")
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000,
            temperature=0.3
        )
        response_text = response.choices[0].message.content
        logger.info("Received response from GPT-4.")

        # Simple parsing of the response - split by numbered items
        lines = response_text.split('\n')
        issues = []
        current_issue = ""

        for line in lines:
            # If it's a new numbered item
            if line.strip() and line[0].isdigit() and '. ' in line[:5]:
                # If we were building a previous issue, add it
                if current_issue:
                    issues.append(current_issue.strip())
                current_issue = line.strip()
            elif line.strip() and current_issue:
                # Continue building the current issue
                current_issue += " " + line.strip()

        # Add the last issue if there is one
        if current_issue:
            issues.append(current_issue.strip())

        if not issues:
            issues.append("No issues detected by GPT-4.")

        logger.info(f"Identified {len(issues)} issues in the rent roll")
        return issues

    except Exception as e:
        logger.error(f"Error calling GPT-4 for analysis: {e}")
        logger.error(traceback.format_exc())
        return ["Failed to analyze rent roll due to API error."]

In [ ]:
def determine_action(state):
    """Decide whether to answer directly, ask for clarification, or generate code."""

    messages = state["messages"]
    user_message = messages[-1]["content"] if messages[-1]["role"] == "user" else ""
    df = state["df"]

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    # Get column information for context
    if df is not None:
        try:
            # Safer way to get column data types
            column_info = []
            for col in df.columns:
                try:
                    dtype_str = str(df[col].dtype)  # Convert dtype to string directly
                    column_info.append(f"- {col}: {dtype_str}")
                except:
                    column_info.append(f"- {col}: unknown type")
            column_info_str = "\n".join(column_info)
            df_preview = df.head(3).to_string()
        except Exception as e:
            logger.error(f"Error getting column info: {e}")
            column_info_str = "Error retrieving column information"
            df_preview = "Error retrieving data preview"
    else:
        column_info_str = "No dataframe loaded"
        df_preview = "No data available"

    # Use GPT-4 to analyze the query and determine the best action
    prompt = f"""
    User query: {user_message}

    Dataframe information:
    - Rows: {len(df) if df is not None else 'No data loaded'}
    - Columns: {column_info_str}

    Data preview:
    {df_preview}

    Analyze the user query and determine the most appropriate action:
    1. If the query is ambiguous or lacks specificity, choose "ask_clarification"
    2. If the query can be answered with a simple explanation without analysis, choose "text_response"
    3. If the query requires data analysis, calculations, or visualizations, choose "generate_code"

    Respond with a JSON object containing:
    {{"action": "ask_clarification" | "text_response" | "generate_code", "reason": "brief explanation"}}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a decision-making agent for a rent roll analysis system. Output ONLY a JSON object with the determined action and reason."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.2
        )

        response_text = response.choices[0].message.content

        # Extract JSON from the response
        json_match = re.search(r'{.*}', response_text, re.DOTALL)
        if json_match:
            action_data = json.loads(json_match.group(0))
            action = action_data.get("action", "text_response")
        else:
            # Default to text response if parsing fails
            action = "text_response"

        logger.info(f"Determined action using GPT-4: {action}")

        # Create a new state dict with updated values
        new_state = dict(state)  # Create a copy
        new_state["needs_clarification"] = action == "ask_clarification"
        new_state["generate_code"] = action == "generate_code"

        return new_state
    except Exception as e:
        logger.error(f"Error in determine_action: {e}")
        # Default to text response on error
        new_state = dict(state)
        new_state["needs_clarification"] = False
        new_state["generate_code"] = False
        return new_state

In [ ]:

def ask_clarification(state: AgentState) -> Dict:
    """Generate a clarification question for the user using GPT-4."""

    messages = state["messages"]
    user_message = messages[-1]["content"] if messages[-1]["role"] == "user" else ""

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": """You are a commercial real estate rent roll analyst.
                Generate a clear, specific clarification question to better understand
                what the user is asking about their rent roll data."""},
                {"role": "user", "content": f"My question is: {user_message}"}
            ],
            max_tokens=300,
            temperature=0.3
        )

        clarification_question = response.choices[0].message.content

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["clarification_question"] = clarification_question
        new_state["final_response"] = clarification_question

        # Add the clarification question to the messages
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": clarification_question})
        new_state["messages"] = new_messages

        logger.info(f"Generated clarification question using GPT-4: {clarification_question[:50]}...")
        return new_state
    except Exception as e:
        logger.error(f"Error in ask_clarification: {e}")
        # Fallback to a generic clarification question
        generic_question = "Could you please clarify what specific aspect of the rent roll you'd like me to analyze?"

        new_state = dict(state)
        new_state["clarification_question"] = generic_question
        new_state["final_response"] = generic_question

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": generic_question})
        new_state["messages"] = new_messages

        return new_state

In [ ]:
def generate_text_response(state):
    """Generate a simple text response to the user query using GPT-4."""

    messages = state["messages"]
    df = state["df"]
    issues = state["issues"]

    # Create OpenAI client for this function call
    client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

    # Prepare context for GPT-4
    issues_text = "\n".join([f"- {issue}" for issue in issues])

    # Get column and data preview for context
    if df is not None:
        column_info = ", ".join(df.columns)
        data_stats = []
        for col in df.columns[:10]:  # Limit to first 10 columns to avoid token limits
            try:
                if pd.api.types.is_numeric_dtype(df[col]):
                    stat = f"- {col}: min={df[col].min()}, max={df[col].max()}, mean={df[col].mean():.2f}, null={df[col].isna().sum()}"
                else:
                    unique_vals = df[col].nunique()
                    stat = f"- {col}: unique values={unique_vals}, null={df[col].isna().sum()}"
                data_stats.append(stat)
            except:
                data_stats.append(f"- {col}: [error calculating stats]")
        data_stats_str = "\n".join(data_stats)
        df_preview = df.head(3).to_string()
    else:
        column_info = "No columns available"
        data_stats_str = "No data statistics available"
        df_preview = "No data preview available"

    system_prompt = f"""You are a commercial real estate rent roll analyst.
    The rent roll data has {len(df) if df is not None else 0} rows and
    {len(df.columns) if df is not None else 0} columns.

    Column information: {column_info}

    Data statistics:
    {data_stats_str}

    Data preview:
    {df_preview}

    Identified issues:
    {issues_text}

    Provide a concise, informative answer to the user's question.
    Focus on being helpful and direct, with only 1-2 paragraphs.
    Do not include code or detailed analysis unless absolutely necessary.
    """

    # Extract system message and filter other messages
    filtered_messages = []
    for msg in messages:
        if msg["role"] != "system":
            filtered_messages.append(msg)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                *filtered_messages
            ],
            max_tokens=1000,
            temperature=0.3
        )

        text_response = response.choices[0].message.content

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["final_response"] = text_response

        # Add the response to the messages
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": text_response})
        new_state["messages"] = new_messages

        logger.info(f"Generated text response using GPT-4: {text_response[:50]}...")
        return new_state
    except Exception as e:
        logger.error(f"Error in generate_text_response: {e}")
        # Fallback to a generic response
        fallback_response = "I'm sorry, I'm having trouble analyzing your rent roll data right now. Could you try rephrasing your question?"

        new_state = dict(state)
        new_state["final_response"] = fallback_response

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": fallback_response})
        new_state["messages"] = new_messages

        return new_state

In [ ]:
def trim_dataframe_output(output_text, max_rows=20, max_chars=None):
    """
    Extremely simplified function that just returns the first 20 lines of output.

    Args:
        output_text: The text output
        max_rows: Maximum number of rows to keep (default: 20)
        max_chars: Not used, kept for compatibility

    Returns:
        Trimmed text showing only top rows
    """
    lines = output_text.split('\n')

    if len(lines) <= max_rows:
        return output_text

    trimmed_lines = lines[:max_rows]
    trimmed_lines.append(f"... [output truncated, showing first {max_rows} lines only] ...")

    return '\n'.join(trimmed_lines)

In [ ]:

from datetime import datetime
def save_dataframe_version(df, operation_description=""):
    """Save the current state of the dataframe as both CSV and Excel files.

    Args:
        df: The dataframe to save
        operation_description: A string describing what operation was performed

    Returns:
        version_name: The name of the version that was saved
    """
    import os
    from datetime import datetime

    # Create versions directory if it doesn't exist
    versions_dir = "rent_roll_versions"
    os.makedirs(versions_dir, exist_ok=True)

    # Generate version name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    version_name = f"v_{timestamp}"

    # Create filenames for both CSV and Excel
    csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
    excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

    # Save as CSV
    df.to_csv(csv_filename, index=False)

    # Save as Excel
    df.to_excel(excel_filename, index=False, engine='openpyxl')

    # Add version metadata to the registry
    if 'app_state' in globals():
        version_info = {
            'name': version_name,
            'description': operation_description,
            'timestamp': timestamp,
            'filename': csv_filename,  # Keep CSV as primary for backward compatibility
            'excel_filename': excel_filename,  # Add Excel filename
            'is_original': len(app_state["df_versions"]) == 0  # First one is original
        }
        app_state["df_versions"].append(version_info)

    print(f"✓ Saved dataframe version {version_name}: {operation_description}")
    print(f"  - CSV: {csv_filename}")
    print(f"  - Excel: {excel_filename}")

    # Return the version name for reference
    return version_name

def get_versions_info_for_prompt():
    """Generate version information for the Claude prompt."""
    if not app_state["df_versions"]:
        return "No versions available yet."

    # Find the original version
    original = next((v for v in app_state["df_versions"] if v.get('is_original')), app_state["df_versions"][0])

    # Get the latest version
    latest = app_state["df_versions"][-1]

    # Format all versions
    all_versions = []
    for i, version in enumerate(app_state["df_versions"]):
        status = []
        if version == original:
            status.append("ORIGINAL")
        if version == latest:
            status.append("LATEST")

        status_str = f" ({', '.join(status)})" if status else ""
        all_versions.append(f"{i+1}. {version['name']}{status_str}: {version['description']}")

    versions_text = "\n".join(all_versions)

    return f"""
DATAFRAME VERSION HISTORY:
{versions_text}

Original version: {original['name']}
Latest version: {latest['name']}
Total versions: {len(app_state["df_versions"])}
"""


In [ ]:
def generate_code_and_execute(state: AgentState) -> Dict:
    """
    Generate and execute code using a two-step AI approach:
    1. Use GPT-4 to create an optimal prompt for Claude
    2. Have Claude generate the code based on this optimized prompt
    3. Execute the code and handle errors with up to 3 retries
    """
    messages = state["messages"]
    df = state["df"]

    # Get OpenAI client from state or create new one
    openai_client = state.get("openai_client") or OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
    # Get Anthropic client from state or create new one
    anthropic_client = state.get("anthropic_client") or Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

    # Get column information for context
    column_info = ", ".join(df.columns) if df is not None else "No columns available"

    # Create SAMPLE dataframe content for GPT-4.1 (first 50 rows instead of full dataset)
    if df is not None:
        # Get first 50 rows for GPT-4.1 context
        sample_df = df.head(50)

        # Convert sample dataframe to string representation
        df_sample_content = sample_df.to_string(index=False)

        # Also get CSV format for better structure
        df_csv_content = sample_df.to_csv(index=False)

        # Prepare comprehensive data summary with sample data
        df_summary = f"""
SAMPLE DATAFRAME CONTENT (First 50 rows out of {len(df)} total rows):
{df_sample_content}

CSV FORMAT (First 50 rows):
{df_csv_content}

FULL DATAFRAME STATISTICS:
- Shape: {df.shape}
- Columns: {list(df.columns)}
- Data types: {dict(df.dtypes)}
- Memory usage: {df.memory_usage(deep=True).sum()} bytes
- Null values per column: {dict(df.isnull().sum())}

NOTE: This is a sample of the first 50 rows. The complete dataframe has {len(df)} rows.
"""
    else:
        df_summary = "No data available"

    # Create versions directory if it doesn't exist
    versions_dir = "rent_roll_versions"
    os.makedirs(versions_dir, exist_ok=True)

    # Print initial state for debugging
    print(f"\n==== STARTING CODE GENERATION ====")
    print(f"User query: {messages[-1]['content'] if messages[-1]['role'] == 'user' else 'No user query found'}")
    print(f"Dataframe has {len(df) if df is not None else 0} rows and {len(df.columns) if df is not None else 0} columns")
    print(f"Sending FIRST 50 ROWS to GPT-4.1 (sample instead of full dataset)")

    try:
        # First, use GPT-4 to create the optimal prompt for Claude
        print("\n==== STEP 1: GENERATING PROMPT WITH GPT-4 (WITH SAMPLE OF 50 ROWS) ====")
        versions_info = get_versions_info_for_prompt()

        # System prompt for GPT-4 to create a Claude prompt
        gpt_system_prompt = f"""You are an expert at creating prompts for Claude AI to generate code.
        Your task is to analyze the user query history and convert it into an optimal prompt for Claude to generate Python code that analyzes a rent roll dataframe.

        CRITICAL INFORMATION: The dataframe is ALREADY LOADED and available as 'df'.
        It contains REAL DATA with {len(df)} rows and {len(df.columns)} columns.

        HERE IS A SAMPLE OF THE DATAFRAME CONTENT (FIRST 50 ROWS OUT OF {len(df)} TOTAL ROWS):
        {df_summary}

        IMPORTANT: This is only a sample of the first 50 rows to give you context about the data structure and content.
        The actual dataframe that Claude will work with contains ALL {len(df)} rows.

        # IMPORTANT: DATAFRAME VERSION MANAGEMENT
        {versions_info}

        # IMPORTANT VERSION IDENTIFICATION:
        - Versions are stored in chronological order by timestamp
        - The original version is always the first one saved (earliest timestamp)
        - The latest version is always the most recent one saved (latest timestamp)
        - When a user says "original dataframe," load the version with the earliest timestamp
        - When a user says "latest version," use the current df (which is already the latest)
        - When a user specifies a version by name (e.g., "v_20250518_112345"), load that exact version

        ALL versions are saved as CSV files in the "rent_roll_versions" directory.
        For example, to load a specific version:

        ```python
        # To load a specific version (e.g., the original version)
        import pandas as pd
        import os

        # Example: Load the original version
        original_version_name = "{{app_state["df_versions"][0]['name'] if app_state["df_versions"] else "v_example"}}"
        original_file_path = os.path.join("rent_roll_versions", f"rent_roll_{{original_version_name}}.csv")
        original_df = pd.read_csv(original_file_path)

        print(f"Loaded original version: {{original_version_name}}")
        print(f"Shape: {{original_df.shape}}")

        # You can either work with this as a separate dataframe, or replace the current df:
        # df = original_df  # This would replace the current df with the original
        ```

        If you make any changes to the dataframe, ALWAYS save a new version using save_dataframe_version().

        Some important guidelines to include in your prompt to Claude:
        1. The variable 'df' is ALREADY DEFINED and CONTAINS ALL {len(df)} ROWS OF DATA. Claude must not say "I need to see the data first"
        2. Claude should explain its approach step by step before showing code
        3. Code must be wrapped in ```python and ``` blocks
        4. Code MUST display ALL rows in the output when showing tables (no limiting rows)
        5. Claude should not attempt to clean data unless specifically requested
        6. Code should include proper error handling
        7. IMPORTANT: After performing any analysis or showing results, Claude should ALWAYS call the save_dataframe_version() function to maintain version history, even if no changes were made to the dataframe.
        8. CRITICAL: Claude should NOT use try-except blocks in its code. Any errors should be allowed to propagate naturally. This ensures that our retry system can properly handle errors.

        Your output will be directly sent to Claude, so format it as a complete system prompt.
        Include any table formatting functions that might be useful.

        Make sure to include these helper functions in your prompt:

        ```python
        # For tabular display with proper formatting (PREFERRED METHOD):
        def print_formatted_table(df, title=None): #Print a dataframe with proper formatting without modifying data
            if title:
                print(f"\\n{{title}}")
                print("=" * 80)

            # Create a display copy (doesn't change original df)
            display_df = df.copy()

            # Set pandas display options for better readability
            # Show ALL rows - no limits
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            pd.set_option('display.colheader_justify', 'left')
            pd.set_option('display.precision', 2)

            # Display the dataframe - ALL rows will be shown
            print(display_df)

            # Reset display options to default
            pd.reset_option('display.max_rows')
            pd.reset_option('display.max_columns')
            pd.reset_option('display.width')
            pd.reset_option('display.colheader_justify')
            pd.reset_option('display.precision')
        ```

        ```python
        # For bordered table display with precise control:
        def print_bordered_table(df, title=None): #Print a dataframe with borders for better readability - SHOWS ALL ROWS
            if title:
                print(f"\\n{{title}}")
                print("=" * 80)

            if len(df) == 0:
                print("No data available")
                return

            # Create a display copy (doesn't change original data)
            display_df = df.copy()

            # Calculate column widths for display purposes only
            col_widths = {{}}
            for col in display_df.columns:
                # Convert values to string only for width calculation
                col_values = display_df[col].astype(str)
                max_data_width = col_values.str.len().max()
                col_widths[col] = max(len(str(col)), max_data_width) + 2  # +2 for padding

            # Create header row
            header = "| " + " | ".join(str(col).ljust(col_widths[col]) for col in display_df.columns) + " |"
            separator = "+" + "+".join("-" * (col_widths[col] + 2) for col in display_df.columns) + "+"

            # Print header
            print(separator)
            print(header)
            print(separator)

            # Print ALL rows - NO LIMIT
            for i in range(len(display_df)):
                row = display_df.iloc[i]
                row_str = "| " + " | ".join(str(val).ljust(col_widths[col]) for col, val in row.items()) + " |"
                print(row_str)

            print(separator)
            print(f"Total rows: {{len(display_df)}}")
        ```

        ```python
        # Function to save dataframe versions
        def save_dataframe_version(df, operation_description=""):
            \"\"\"Save the current state of the dataframe as both CSV and Excel files.

            This function should be called whenever you make changes to the dataframe,
            or after generating analysis results, to maintain version history.

            Args:
                df: The dataframe to save
                operation_description: A string describing what operation was performed

            Returns:
                version_name: The name of the version that was saved
            \"\"\"
            import os
            from datetime import datetime

            # Create versions directory if it doesn't exist
            versions_dir = "rent_roll_versions"
            os.makedirs(versions_dir, exist_ok=True)

            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{{timestamp}}"

            # Create filenames for both CSV and Excel
            csv_filename = os.path.join(versions_dir, f"rent_roll_{{version_name}}.csv")
            excel_filename = os.path.join(versions_dir, f"rent_roll_{{version_name}}.xlsx")

            # Save as CSV
            df.to_csv(csv_filename, index=False)

            # Save as Excel
            df.to_excel(excel_filename, index=False, engine='openpyxl')

            print(f"✓ Saved dataframe version {{version_name}}: {{operation_description}}")
            print(f"  - CSV: {{csv_filename}}")
            print(f"  - Excel: {{excel_filename}}")

            # Return the version name for reference
            return version_name
        ```
        """

        # Filter out system messages and DON'T trim dataframe outputs in the conversation history
        filtered_messages = []
        for msg in messages:
            if msg["role"] != "system":
                # Don't trim here anymore
                filtered_messages.append({"role": msg["role"], "content": msg["content"]})

        # Convert the messages to the format expected by OpenAI
        gpt_messages = [{"role": "system", "content": gpt_system_prompt}]
        for msg in filtered_messages:
            gpt_messages.append(msg)

        # Add a final message explaining the task clearly
        gpt_messages.append({
            "role": "user",
            "content": f"Based on this conversation history and the SAMPLE dataframe content (first 50 rows) provided above, create the optimal Claude prompt to generate Python code for rent roll analysis. The prompt should emphasize that the dataframe already exists and is loaded as 'df' with ALL {len(df)} rows, that ALL rows should be displayed when requested, and that versions should be saved with save_dataframe_version() function. You have access to a representative sample of the data structure."
        })

        # Get the optimized prompt from GPT-4
        gpt_response = openai_client.chat.completions.create(
            model="gpt-4.1",
            messages=gpt_messages,
            max_tokens=4000,  # Increased token limit to handle larger dataframes
            temperature=0.3
        )

        claude_system_prompt = gpt_response.choices[0].message.content

        # Print the generated prompt for debugging
        print("\n==== GPT-4 GENERATED PROMPT FOR CLAUDE ====")
        print(claude_system_prompt[:500] + "..." if len(claude_system_prompt) > 500 else claude_system_prompt)
        print("==== END OF PROMPT (TRUNCATED) ====\n")

        logger.info("Generated optimized prompt for Claude using GPT-4 with sample dataframe (50 rows)")

        # Now use the GPT-4 generated prompt to ask Claude for code
        print("\n==== STEP 2: SENDING TO CLAUDE FOR CODE GENERATION ====")
        logger.info("Sending optimized prompt to Claude for code generation")

        # Prepare messages for Claude with the sample dataframe content
        claude_messages = filtered_messages.copy()

        # Add the sample dataframe content to help Claude understand the data exists
        sample_data_message = {
            "role": "user",
            "content": f"Here is a SAMPLE of the dataframe that's already loaded as 'df' (showing first 50 rows out of {len(df)} total rows):\n\n{df_summary}\n\nPlease process my request using this FULL dataset of {len(df)} rows and remember to save versions with save_dataframe_version()."
        }
        claude_messages.append(sample_data_message)

        # Try to get code from Claude
        claude_response = anthropic_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            system=claude_system_prompt,
            messages=claude_messages,
            max_tokens=4000,  # Increased to handle more complex responses
            temperature=0.3
        )

        # Extract the response text from Claude
        response_text = claude_response.content[0].text

        # Print Claude's response for debugging
        print("\n==== CLAUDE'S RESPONSE ====")
        print(response_text[:500] + "..." if len(response_text) > 500 else response_text)
        print("==== END OF CLAUDE RESPONSE (TRUNCATED) ====\n")

        # Extract code blocks
        code_blocks = re.findall(r'```python\s*(.*?)\s*```', response_text, re.DOTALL)

        # Print extracted code blocks for debugging
        print(f"\n==== EXTRACTED {len(code_blocks)} CODE BLOCKS ====")
        for i, block in enumerate(code_blocks):
            print(f"\n-- Code Block {i+1} --")
            print(block[:200] + "..." if len(block) > 200 else block)

        # If no code blocks are found, add emergency code
        if len(code_blocks) == 0:
            emergency_code = """
            # Emergency code to display the dataframe
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)

            print("\\n=== RENT ROLL DATA ===\\n")
            print(f"Displaying all {len(df)} rows and {len(df.columns)} columns\\n")

            # Print the entire dataframe
            print(df)

            # Save a version of the dataframe
            from datetime import datetime
            import os

            # Create versions directory if it doesn't exist
            versions_dir = "rent_roll_versions"
            os.makedirs(versions_dir, exist_ok=True)

            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{timestamp}"

            # Create filename
            filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")

            # Save dataframe
            df.to_csv(filename, index=False)

            print(f"✓ Saved dataframe version {version_name}: Emergency display of data")
            """
            code_blocks.append(emergency_code)
            print("\n-- Added Emergency Code Block --")
            print("Emergency code added since Claude didn't generate code")

        # Rest of the function remains the same (helper functions, execution loop, etc.)
        # Define helper functions
        def print_formatted_table(df, title=None):
            if title:
                print(f"\n{title}")
                print("=" * 80)

            # Create a display copy (doesn't change original df)
            display_df = df.copy()

            # Set pandas display options for better readability
            # Show ALL rows - no limits
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            pd.set_option('display.colheader_justify', 'left')
            pd.set_option('display.precision', 2)

            # Display the dataframe - ALL rows will be shown
            print(display_df)

            # Reset display options to default
            pd.reset_option('display.max_rows')
            pd.reset_option('display.max_columns')
            pd.reset_option('display.width')
            pd.reset_option('display.colheader_justify')
            pd.reset_option('display.precision')

        def print_bordered_table(df, title=None):
            if title:
                print(f"\n{title}")
                print("=" * 80)

            if len(df) == 0:
                print("No data available")
                return

            # Create a display copy (doesn't change original data)
            display_df = df.copy()

            # Calculate column widths for display purposes only
            col_widths = {}
            for col in display_df.columns:
                # Convert values to string only for width calculation
                col_values = display_df[col].astype(str)
                max_data_width = col_values.str.len().max()
                col_widths[col] = max(len(str(col)), max_data_width) + 2  # +2 for padding

            # Create header row
            header = "| " + " | ".join(str(col).ljust(col_widths[col]) for col in display_df.columns) + " |"
            separator = "+" + "+".join("-" * (col_widths[col] + 2) for col in display_df.columns) + "+"

            # Print header
            print(separator)
            print(header)
            print(separator)

            # Print ALL rows - NO LIMIT
            for i in range(len(display_df)):
                row = display_df.iloc[i]
                row_str = "| " + " | ".join(str(val).ljust(col_widths[col]) for col, val in row.items()) + " |"
                print(row_str)

            print(separator)
            print(f"Total rows: {len(display_df)}")

        # Add to globals_dict before executing code
        globals_dict = {
            "df": df,
            "pd": pd,
            "np": np,
            "os": os,                   # Add os for folder creation
            "datetime": datetime,       # Add datetime for timestamp
            "versions_dir": versions_dir,  # Pass the versions directory
            "print_formatted_table": print_formatted_table,  # Add the helper function
            "print_bordered_table": print_bordered_table,    # Add the helper function
            "save_dataframe_version": save_dataframe_version  # Make sure this is defined too
        }

        execution_results = ""
        all_executed_successfully = False
        max_retries = 5  # Maximum number of retries
        retry_count = 0  # Initialize retry counter
        failed_code = ""  # Store the failed code for context
        error_msg = ""    # Store the error message

        print("\n==== STEP 3: EXECUTING CODE WITH RETRIES ====")

        # Main retry loop (rest of the execution code remains the same)
        while not all_executed_successfully and retry_count <= max_retries:
            # If this is a retry attempt (not the first try)
            if retry_count > 0:
                print(f"\n==== RETRY ATTEMPT {retry_count}/{max_retries} ====")

                # Create a retry message with more details each time
                retry_message = {
                    "role": "user",
                    "content": f"""The code you provided failed with this error: {error_msg}

                    Here is the code that failed:
                    ```python
                    {failed_code}
                    ```

                    This is retry attempt {retry_count} of {max_retries}.

                    {"After multiple attempts, please try a completely different approach." if retry_count >= 2 else "Please fix this specific error."}
                    IMPORTANT: DO NOT use try-except blocks in your code. Allow any errors to propagate naturally so our system can detect them.
                    Please fix this code to handle the specific error while maintaining the requirement to show ALL rows in the output and saving a version with save_dataframe_version().
                    Return the corrected code wrapped in ```python and ``` blocks."""
                }

                # Add this feedback to the messages
                fix_messages = claude_messages.copy()
                fix_messages.append({"role": "assistant", "content": response_text})
                fix_messages.append(retry_message)

                # Get Claude's fixed code
                retry_response = anthropic_client.messages.create(
                    model="claude-3-7-sonnet-20250219",
                    system=claude_system_prompt,
                    messages=fix_messages,
                    max_tokens=3500,
                    temperature=0.3
                )

                retry_text = retry_response.content[0].text
                print(f"\n==== CLAUDE'S FIX SUGGESTION (ATTEMPT {retry_count}) ====")
                print(retry_text[:500] + "..." if len(retry_text) > 500 else retry_text)

                # Extract the fixed code blocks
                fixed_code_blocks = re.findall(r'```python\s*(.*?)\s*```', retry_text, re.DOTALL)

                if fixed_code_blocks:
                    # Use the first fixed code block
                    code_to_execute = fixed_code_blocks[0]

                    # Update response text to include the fix explanation
                    fix_explanation = f"\n\n**🔧 Code Fix (Attempt {retry_count}):**\n"
                    fix_explanation += f"The code encountered an error. Here's the fix for retry attempt {retry_count}:\n"
                    fix_explanation += "\n```python\n" + code_to_execute + "\n```\n"

                    if retry_count == 1:
                        # First retry - add to original response
                        response_text = response_text + fix_explanation
                    else:
                        # Subsequent retries - replace previous fix explanation
                        prev_fix_marker = f"**🔧 Code Fix (Attempt {retry_count-1}):**"
                        if prev_fix_marker in response_text:
                            # Replace previous fix with new one
                            response_text = response_text.replace(
                                prev_fix_marker,
                                f"**🔧 Code Fix (Attempt {retry_count}):**"
                            )
                        else:
                            # Just append this fix
                            response_text = response_text + fix_explanation
                else:
                    # If no code blocks found in retry, try emergency code
                    code_to_execute = f"""
                    # Emergency code for retry {retry_count}
                    print(f"\\n=== EMERGENCY DISPLAY (RETRY {retry_count}) ===\\n")
                    print(f"DataFrame shape: {{df.shape}}")
                    print("\\nColumn names:")
                    for col in df.columns:
                        print(f"- {{col}}")

                    print("\\nFirst 10 rows:")
                    print(df.head(10))

                    save_dataframe_version(df, f"Emergency display after retry {retry_count}")
                    """
                    print(f"No code blocks found in retry. Using emergency code.")
            else:
                # Initial execution (not a retry)
                # Run the original code block
                if code_blocks:
                    code_to_execute = code_blocks[0]  # Use the first code block
                else:
                    # This should not happen due to the earlier check, but just in case
                    code_to_execute = """
                    print("No code blocks found. Displaying basic dataframe info.")
                    print(f"DataFrame shape: {df.shape}")
                    print(df.head())
                    save_dataframe_version(df, "Automatic save after initial execution")
                    """

            # Execute the current code
            print(f"\n{'Executing' if retry_count == 0 else 'Retrying'} code...")
            output_buffer = io.StringIO()
            try:
                # Store the code in case it fails
                failed_code = code_to_execute

                with redirect_stdout(output_buffer):
                    exec(code_to_execute, globals_dict)

                execution_output = output_buffer.getvalue()
                print(f"Execution {'successful' if retry_count == 0 else 'fixed on retry ' + str(retry_count)}! Output length: {len(execution_output)} characters")
                print(execution_output[:200] + "..." if len(execution_output) > 200 else execution_output)

                # ONLY trim the execution output for storing, not the entire response
                trimmed_output = trim_dataframe_output(execution_output, max_rows=20)

                # Format the results message based on retry count
                if retry_count == 0:
                    results_msg = "**✅ Code Execution Results:**"
                else:
                    results_msg = f"**✅ Code Execution Results (After Fix Attempt {retry_count}):**"

                execution_results = f"\n\n{results_msg}\n```\n{trimmed_output}\n```\n"

                # Check if a version was saved
                if "✓ Saved dataframe version" not in execution_output:
                    # Auto-save a version
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    version_name = f"v_{timestamp}"
                    csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
                    excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

                    # Save both CSV and Excel
                    df.to_csv(csv_filename, index=False)
                    df.to_excel(excel_filename, index=False, engine='openpyxl')

                    save_message = f"✓ Saved dataframe version {version_name}: Automatic save after {'execution' if retry_count == 0 else 'retry ' + str(retry_count)}"
                    print(save_message)
                    print(f"  - CSV: {csv_filename}")
                    print(f"  - Excel: {excel_filename}")
                    execution_results += f"\n{save_message}\n"

                # Mark as successful and break the retry loop
                all_executed_successfully = True
                logger.info(f"Successfully executed code {'' if retry_count == 0 else 'on retry ' + str(retry_count)}")
                break

            except Exception as e:
                # Execution failed
                error_msg = f"Error: {str(e)}"
                print(f"Execution failed with error: {error_msg}")

                # Log the error
                if retry_count == 0:
                    execution_results = f"\n\n**❌ Code Execution Failed:**\n```\n{error_msg}\n```\n"
                else:
                    execution_results = f"\n\n**❌ Code Execution Failed (Retry {retry_count}):**\n```\n{error_msg}\n```\n"

                logger.error(f"Code execution failed on {'initial attempt' if retry_count == 0 else 'retry ' + str(retry_count)}: {e}")
                logger.error(traceback.format_exc())

                # Increment retry counter
                retry_count += 1

                # If we've hit max retries and still failed, try emergency display as last resort
                if retry_count > max_retries:
                    print("\n==== MAX RETRIES REACHED, TRYING EMERGENCY DISPLAY ====")

                    # Create emergency display code
                    emergency_code = """
                    try:
                        print("\\n=== EMERGENCY FALLBACK DISPLAY ===\\n")
                        print(f"DataFrame shape: {df.shape}")
                        print("\\nColumn names:")
                        for col in df.columns:
                            print(f"- {col}")

                        print("\\nFirst 10 rows:")
                        print(df.head(10))

                        # Try to show some basic stats about numeric columns
                        try:
                            numeric_cols = df.select_dtypes(include=['number']).columns
                            if len(numeric_cols) > 0:
                                print("\\nBasic statistics for numeric columns:")
                                print(df[numeric_cols].describe())
                        except Exception as stats_err:
                            print(f"Could not generate statistics: {stats_err}")

                        # Save version - both CSV and Excel
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        version_name = f"v_{timestamp}_emergency"
                        csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
                        excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

                        df.to_csv(csv_filename, index=False)
                        df.to_excel(excel_filename, index=False, engine='openpyxl')

                        print(f"✓ Saved dataframe version {version_name}: Emergency display after all retries failed")
                        print(f"  - CSV: {csv_filename}")
                        print(f"  - Excel: {excel_filename}")
                    except Exception as e_inner:
                        print(f"Even emergency display failed: {e_inner}")
                    """

                    output_buffer = io.StringIO()
                    try:
                        with redirect_stdout(output_buffer):
                            exec(emergency_code, globals_dict)

                        emergency_output = output_buffer.getvalue()
                        # Only trim the emergency output, not the whole response
                        execution_results += f"\n\n**⚠️ Emergency Data Display (After {max_retries} Failed Retries):**\n```\n{trim_dataframe_output(emergency_output, max_rows=20)}\n```\n"
                    except Exception as e_final:
                        print(f"Emergency fallback also failed: {e_final}")
                        execution_results += f"\n\n**❌ All Recovery Attempts Failed**\n"

        # Add a note about the hybrid approach and retry attempts
        if retry_count > 0 and all_executed_successfully:
            hybrid_note = f"\n\n**📝 Note:** This analysis was performed using a hybrid approach with GPT-4 and Claude. The code was successfully fixed after {retry_count} retry attempts. GPT-4 received the complete dataframe for optimal context."
        elif retry_count > max_retries:
            hybrid_note = f"\n\n**📝 Note:** This analysis was attempted using a hybrid approach with GPT-4 and Claude, but all {max_retries} retry attempts failed. Some basic information was displayed as a fallback."
        else:
            hybrid_note = "\n\n**📝 Note:** This analysis was performed using a hybrid approach: GPT-4 optimized the prompt with full dataframe context, and Claude generated and executed the code for detailed rent roll analysis."

        # Combine the response and execution results
        full_response = response_text + execution_results + hybrid_note

        print("\n==== FINAL RESPONSE GENERATED ====")
        print(f"Original response length: {len(full_response)} characters")
        print(f"Retry attempts: {retry_count}")
        print(f"Execution successful: {all_executed_successfully}")

        # Create a new state dict with updated values
        new_state = dict(state)
        new_state["code_execution_results"] = execution_results
        new_state["final_response"] = full_response  # Don't trim the full response

        # Add the response to the messages - don't trim it here either
        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": full_response})
        new_state["messages"] = new_messages

        logger.info("Code generation and execution complete using hybrid GPT-4/Claude approach with full dataframe")
        print("\n==== CODE GENERATION COMPLETE ====")

        return new_state

    except Exception as e:
        logger.error(f"Error in hybrid code generation: {e}")
        logger.error(traceback.format_exc())
        print(f"\n==== ERROR IN CODE GENERATION ====\n{e}\n{traceback.format_exc()}")

        # Try to save a version even on error
        try:
            # Generate version name with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            version_name = f"v_{timestamp}_system_error"

            # Create filenames
            csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")
            excel_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.xlsx")

            # Save both formats
            df.to_csv(csv_filename, index=False)
            df.to_excel(excel_filename, index=False, engine='openpyxl')

            save_message = f"✓ Saved dataframe version {version_name}: System error - {str(e)[:100]}"
            print(save_message)
            print(f"  - CSV: {csv_filename}")
            print(f"  - Excel: {excel_filename}")
        except Exception as save_error:
            print(f"Failed to save error version: {save_error}")

        # Fallback to a generic response
        fallback_response = f"""
        I'm sorry, I encountered an issue while generating and executing code for your request.

        **Technical Details:** {str(e)}

        Could you try asking your question in a different way? For complex analyses, it sometimes helps to break down your request into smaller, more specific questions.
        """

        new_state = dict(state)
        new_state["final_response"] = fallback_response

        new_messages = state["messages"].copy()
        new_messages.append({"role": "assistant", "content": fallback_response})
        new_state["messages"] = new_messages

        return new_state

In [ ]:
# Build the LangGraph workflow
def create_agentic_rent_roll_analyzer():
    """Create and return the agentic rent roll analyzer workflow."""

    # Create the graph
    workflow = StateGraph(AgentState)

    # Add nodes to the graph
    workflow.add_node("determine_action", determine_action)
    workflow.add_node("ask_clarification", ask_clarification)
    workflow.add_node("generate_text_response", generate_text_response)
    workflow.add_node("generate_code_and_execute", generate_code_and_execute)

    # Set the entry point
    workflow.set_entry_point("determine_action")

    # Define conditional edges based on dictionary state values
    workflow.add_conditional_edges(
        "determine_action",
        lambda state: "ask_clarification" if state.get("needs_clarification") else
                      "generate_code_and_execute" if state.get("generate_code") else
                      "generate_text_response"
    )

    # Add edges to END
    workflow.add_edge("ask_clarification", END)
    workflow.add_edge("generate_text_response", END)
    workflow.add_edge("generate_code_and_execute", END)

    # Compile the graph
    agentic_analyzer = workflow.compile()

    return agentic_analyzer


In [ ]:
def upload_rent_roll(file, anthropic_api_key, openai_api_key, auto_analyze):
    """Process the uploaded rent roll file and initialize the chat."""
    global app_state

    logger.info("Starting rent roll upload and processing")

    # Use the default API keys if none are provided
    anthropic_key = anthropic_api_key if anthropic_api_key else DEFAULT_ANTHROPIC_API_KEY
    openai_key = openai_api_key if openai_api_key else DEFAULT_OPENAI_API_KEY
    logger.info("API keys configured")

    # Validate inputs
    if not file:
        logger.warning("No file uploaded")
        return "Please upload a rent roll Excel file.", None, gr.update(visible=False)

    try:
        # Save the uploaded file to a temporary location
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.xlsx')
        temp_file.close()
        file_path = temp_file.name
        logger.info(f"Created temporary file: {file_path}")

        # Copy the uploaded file to our temporary location
        with open(file.name, 'rb') as src_file, open(file_path, 'wb') as dst_file:
            dst_file.write(src_file.read())
        logger.info("File copied to temporary location")

        # Use our improved rent roll loader
        try:
            logger.info("Loading rent roll with specialized loader...")
            rent_roll_df = read_rent_roll_simple(file_path)
        except Exception as e:
            logger.warning(f"Error with specialized loader: {e}. Falling back to standard loading.")
            # Fallback to basic loading
            rent_roll_df = pd.read_excel(file_path)
            logger.info("Fallback: Loaded rent roll with default pandas settings")

        logger.info(f"Loaded rent roll data: {len(rent_roll_df)} rows, {len(rent_roll_df.columns)} columns")

        # Auto-analyze with GPT if selected
        if auto_analyze:
            logger.info("Auto-analyze option selected. Calling GPT for analysis...")
            issues_list = analyze_rent_roll_gpt(file_path, openai_key)  # Use OpenAI key for this
            logger.info(f"GPT analysis complete. Found {len(issues_list)} issues.")
        else:
            # Create empty issues list if not auto-analyzing
            issues_list = []
            logger.info("No auto-analysis performed.")

        # Initialize the global app state with version tracking
        app_state = {
            "df": rent_roll_df,
            "issues": issues_list,
            "anthropic_client": Anthropic(api_key=anthropic_key),
            "openai_client": OpenAI(api_key=openai_key),
            "system_message": "",  # Will be populated below
            "df_versions": []  # Initialize empty version registry
        }

        # Save the initial version
        initial_version = save_dataframe_version(rent_roll_df, "Initial upload - original dataset")
        logger.info(f"Created initial dataframe version: {initial_version}")

        # Create system message with data understanding
        column_info = []
        for col in rent_roll_df.columns:
            try:
                dtype_str = str(rent_roll_df[col].dtype)
                column_info.append(f"- {col}: {dtype_str}")
            except Exception as e:
                column_info.append(f"- {col}: [Error determining type: {str(e)}]")
        column_info_str = "\n".join(column_info)
        # Calculate basic stats about the data
        data_stats = []
        for col in rent_roll_df.columns:
            try:
                if pd.api.types.is_numeric_dtype(rent_roll_df[col]):
                    stat = f"- {col}: min={rent_roll_df[col].min()}, max={rent_roll_df[col].max()}, mean={rent_roll_df[col].mean():.2f}, null={rent_roll_df[col].isna().sum()}"
                else:
                    unique_vals = rent_roll_df[col].nunique()
                    stat = f"- {col}: unique values={unique_vals}, null={rent_roll_df[col].isna().sum()}"
                data_stats.append(stat)
            except:
                data_stats.append(f"- {col}: [error calculating stats]")
        data_stats_str = "\n".join(data_stats)

        # Format issues for display
        issues_text = "\n".join([f"- {issue}" for issue in issues_list])

        system_message = f"""
        You are a Commercial Real Estate rent roll assistant that has analyzed a rent roll and found the following issues:

        {issues_text}

        The rent roll data has {len(rent_roll_df)} rows and {len(rent_roll_df.columns)} columns.

        Column information:
        {column_info_str}

        Data statistics:
        {data_stats_str}

        When helping the user, follow these critical guidelines:
        1. DO NOT generate placeholder code with fake column names. Work ONLY with the actual columns from the dataframe.
        2. NEVER assume column names that don't exist in the actual data.
        3. Always start by examining the first few rows to understand the meaning of each column.
        4. If you can't identify which columns contain certain information, clearly state this limitation.
        5. DO NOT proceed with analysis using made-up column names that don't exist in the data.

        The entire dataframe is available as 'df' in the execution environment.

        Important instructions for code and calculations:
        1. ALWAYS share your chain of thought reasoning in your responses. For each analysis:
          - Begin with "**Thinking through this step by step:**" in bold
          - Clearly explain your understanding of the request
          - Describe your approach to solving the problem
          - Outline the data exploration steps you'll take
          - Explain why you're choosing specific columns and methods
          - Discuss any challenges you anticipate with the data structure
          This chain of thought should be visible to the user in your chat responses.
        """

        # Save the system message to the app state
        app_state["system_message"] = system_message

        # Clean up the temporary file
        os.unlink(file_path)
        logger.info("Temporary file removed")

        # Generate a preview of the data and issues
        preview_html = f"""
        <h3>Rent Roll Preview</h3>
        <p>Successfully loaded rent roll with {len(rent_roll_df)} rows and {len(rent_roll_df.columns)} columns.</p>
        {rent_roll_df.head(5).fillna('').to_html(index=False)}

        <h3>Identified Issues</h3>
        <ol>
        """

        # Format each issue for the HTML preview
        for issue in issues_list:
            # If issue starts with a number (like "1. Issue"), strip the number
            if issue and issue[0].isdigit() and ". " in issue[:5]:
                issue = issue[issue.find(". ")+2:]
            preview_html += f"<li>{issue}</li>"

        preview_html += """
        </ol>
        <p>You can now start asking questions in the chat below!</p>
        <p><strong>Note:</strong> This application uses GPT-4 for decision making and text responses,
        and Claude AI specifically for code generation and execution.</p>
        """
        version_choices = get_version_choices()
        # Make the chat interface visible
        logger.info("Setup complete. Ready for chat interaction.")
        return (
            "Rent roll loaded successfully! You can now start chatting.",
            preview_html,
            gr.update(visible=True),  # chatbot visibility
            gr.update(choices=version_choices, value=version_choices[-1] if version_choices else None)  # version dropdown
        )


    # Also update the error return:
    except Exception as e:
        logger.error(f"Error during rent roll processing: {e}")
        logger.error(traceback.format_exc())
        if 'file_path' in locals() and os.path.exists(file_path):
            os.unlink(file_path)
            logger.info("Cleaned up temporary file after error")
        return f"Error: {str(e)}", None, gr.update(visible=False), gr.update(choices=[], value=None)

In [ ]:
def load_latest_version_for_editing():
    """Load the most recent version of the dataframe for editing"""
    global app_state

    if app_state is None or app_state["df"] is None:
        return None, "No data loaded. Please upload a rent roll first."

    try:
        # Use the current dataframe (which is the latest)
        df = app_state["df"].copy()
        df = df.fillna('')
        # Get version info
        if app_state["df_versions"]:
            latest_version = app_state["df_versions"][-1]
            version_info = f"Loaded version: {latest_version['name']} - {latest_version['description']}"
        else:
            version_info = "Loaded current data (no versions saved yet)"

        logger.info(f"Loaded dataframe for editing: {df.shape}")
        return df, version_info
    except Exception as e:
        logger.error(f"Error loading data for editing: {e}")
        return None, f"Error loading data: {str(e)}"

def save_edited_dataframe(edited_df, description):
    """Save the edited dataframe as a new version"""
    global app_state

    if edited_df is None or edited_df.empty:
        return "No data to save", gr.update()

    try:
        # Convert the edited dataframe to proper pandas DataFrame if needed
        if not isinstance(edited_df, pd.DataFrame):
            edited_df = pd.DataFrame(edited_df)

        # Generate a meaningful description
        if not description:
            description = "Manual edits via data editor"

        # Save as new version
        version_name = save_dataframe_version(edited_df, description)

        # Update the app state with the edited dataframe
        app_state["df"] = edited_df

        # Log the changes
        logger.info(f"Saved edited dataframe as version {version_name}")

        # Return success message and update the view
        return f"✅ Successfully saved as version {version_name}", gr.update(value=edited_df)

    except Exception as e:
        logger.error(f"Error saving edited dataframe: {e}")
        return f"❌ Error saving: {str(e)}", gr.update()

def load_specific_version(version_name):
    """Load a specific version for editing"""
    global app_state

    if not version_name:
        return None, "Please select a version to load"

    try:
        # Find the version file
        versions_dir = "rent_roll_versions"
        csv_filename = os.path.join(versions_dir, f"rent_roll_{version_name}.csv")

        if os.path.exists(csv_filename):
            df = pd.read_csv(csv_filename)
            df = df.fillna('')
            logger.info(f"Loaded version {version_name} for editing")
            return df, f"Loaded version: {version_name}"
        else:
            return None, f"Version file not found: {version_name}"

    except Exception as e:
        logger.error(f"Error loading version {version_name}: {e}")
        return None, f"Error loading version: {str(e)}"

def get_version_choices():
    """Get list of available versions for dropdown"""
    global app_state

    if app_state and "df_versions" in app_state and app_state["df_versions"]:
        choices = []
        for i, version in enumerate(app_state["df_versions"]):
            status = ""
            if i == 0:
                status = " (ORIGINAL)"
            elif i == len(app_state["df_versions"]) - 1:
                status = " (LATEST)"

            choices.append(f"{version['name']}{status}")
        return choices
    return []

def refresh_version_dropdown():
    """Refresh the version dropdown choices"""
    choices = get_version_choices()
    if choices:
        return gr.update(choices=choices, value=choices[-1])  # Default to latest
    return gr.update(choices=[], value=None)

In [ ]:
class SessionRecorder:
    def __init__(self):
        self.sessions_dir = "copiloting_sessions"
        os.makedirs(self.sessions_dir, exist_ok=True)
        self.current_session_file = None
        self.current_session_data = {}

    def start_session_recording(self, rent_roll_filename):
        """Start recording the entire copiloting session"""
        session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        self.current_session_file = os.path.join(self.sessions_dir, f"{session_id}.txt")

        # Initialize session data
        self.current_session_data = {
            "session_id": session_id,
            "start_time": datetime.now().isoformat(),
            "rent_roll_file": rent_roll_filename,
            "conversation_history": [],
            "code_executions": [],
            "dataframe_versions": [],
            "issues_found": [],
            "user_goals": []
        }

        # Write session header to text file
        with open(self.current_session_file, 'w', encoding='utf-8') as f:
            f.write(f"=== RENT ROLL COPILOTING SESSION ===\n")
            f.write(f"Session ID: {session_id}\n")
            f.write(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Rent Roll File: {rent_roll_filename}\n")
            f.write(f"=" * 50 + "\n\n")

        print(f"📝 Started session recording: {session_id}")
        return session_id

    def record_conversation_turn(self, user_message, ai_response, action_type, code_executed=None, version_saved=None):
        """Record each conversation turn in real-time"""
        if not self.current_session_file:
            return

        timestamp = datetime.now().strftime('%H:%M:%S')
        turn_data = {
            "timestamp": timestamp,
            "user_message": user_message,
            "ai_response": ai_response,
            "action_type": action_type,
            "code_executed": code_executed,
            "version_saved": version_saved
        }

        # Add to session data
        self.current_session_data["conversation_history"].append(turn_data)

        # Append to text file immediately
        with open(self.current_session_file, 'a', encoding='utf-8') as f:
            f.write(f"[{timestamp}] USER: {user_message}\n")
            f.write(f"Action Type: {action_type}\n")

            if code_executed:
                f.write(f"CODE EXECUTED:\n```python\n{code_executed}\n```\n")

            f.write(f"AI RESPONSE: {ai_response}\n")

            if version_saved:
                f.write(f"VERSION SAVED: {version_saved}\n")

            f.write("-" * 80 + "\n\n")

        # Track code executions separately
        if code_executed:
            self.current_session_data["code_executions"].append({
                "timestamp": timestamp,
                "code": code_executed,
                "purpose": user_message,
                "result": ai_response[:200] + "..." if len(ai_response) > 200 else ai_response
            })

    def record_dataframe_version(self, version_name, description, shape, columns):
        """Record dataframe version changes"""
        version_info = {
            "timestamp": datetime.now().strftime('%H:%M:%S'),
            "version_name": version_name,
            "description": description,
            "shape": shape,
            "columns": columns
        }

        self.current_session_data["dataframe_versions"].append(version_info)

        # Append to text file
        if self.current_session_file:
            with open(self.current_session_file, 'a', encoding='utf-8') as f:
                f.write(f"VERSION SAVED: {version_name}\n")
                f.write(f"Description: {description}\n")
                f.write(f"Shape: {shape}\n")
                f.write(f"Columns: {columns}\n")
                f.write("-" * 40 + "\n\n")

    def record_issue_found(self, issue_description, severity="medium"):
        """Record issues found during analysis"""
        issue_info = {
            "timestamp": datetime.now().strftime('%H:%M:%S'),
            "description": issue_description,
            "severity": severity
        }

        self.current_session_data["issues_found"].append(issue_info)

        if self.current_session_file:
            with open(self.current_session_file, 'a', encoding='utf-8') as f:
                f.write(f"ISSUE FOUND [{severity.upper()}]: {issue_description}\n")
                f.write("-" * 40 + "\n\n")

    def finalize_session(self):
        """End session recording and return session data"""
        if not self.current_session_file:
            return None

        end_time = datetime.now()
        duration = end_time - datetime.fromisoformat(self.current_session_data["start_time"])

        # Write session summary
        with open(self.current_session_file, 'a', encoding='utf-8') as f:
            f.write("\n" + "=" * 50 + "\n")
            f.write("SESSION SUMMARY\n")
            f.write("=" * 50 + "\n")
            f.write(f"Session Duration: {duration.total_seconds()/60:.1f} minutes\n")
            f.write(f"Total Conversations: {len(self.current_session_data['conversation_history'])}\n")
            f.write(f"Code Executions: {len(self.current_session_data['code_executions'])}\n")
            f.write(f"Versions Created: {len(self.current_session_data['dataframe_versions'])}\n")
            f.write(f"Issues Found: {len(self.current_session_data['issues_found'])}\n")
            f.write(f"Ended: {end_time.strftime('%Y-%m-%d %H:%M:%S')}\n")

        # Update session data
        self.current_session_data["end_time"] = end_time.isoformat()
        self.current_session_data["duration_minutes"] = duration.total_seconds() / 60

        session_data = self.current_session_data.copy()

        # Reset for next session
        self.current_session_file = None
        self.current_session_data = {}

        print(f"✅ Session recording finalized: {session_data['session_id']}")
        return session_data

# Global session recorder
session_recorder = SessionRecorder()

In [ ]:
class EnhancedTemplateManager:
    def __init__(self):
        self.templates_dir = "rent_roll_templates"
        os.makedirs(self.templates_dir, exist_ok=True)
        self.current_session = None

    def create_template_from_session(self, session_data, starting_df, final_df, template_name):
        """Generate comprehensive template from complete session data using GPT-4"""

        print("🤖 Analyzing session with GPT-4.1 to generate instructions...")

        # Prepare comprehensive session context for GPT-4
        session_context = self._prepare_session_context(session_data)

        # Use GPT-4.1 to analyze and generate instructions
        client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

        analysis_prompt = f"""
        You are an expert at analyzing data analysis workflows and creating reusable instruction templates.

        I will provide you with a complete copiloting session where a user worked on a rent roll analysis.
        Your task is to:
        1. Analyze the entire workflow
        2. Identify the key transformation patterns
        3. Create step-by-step instructions that can be applied to similar rent roll files
        4. Generate reusable code templates with placeholders
        5. Document the business logic and decision points

        SESSION DATA:
        {session_context}

        STARTING DATAFRAME INFO:
        - Shape: {starting_df.shape}
        - Columns: {list(starting_df.columns)}
        - Sample data: {starting_df.head(2).to_string()}

        FINAL DATAFRAME INFO:
        - Shape: {final_df.shape}
        - Columns: {list(final_df.columns)}
        - Sample data: {final_df.head(2).to_string()}

        Please generate a comprehensive analysis in the following JSON format:
        {{
            "workflow_summary": "Brief description of what was accomplished",
            "key_transformations": [
                {{
                    "step_name": "Clean Tenant Names",
                    "description": "Standardize tenant name formatting",
                    "business_rule": "All tenant names should be Title Case with no extra whitespace",
                    "code_template": "df['{{column_name}}'] = df['{{original_column}}'].str.strip().str.title()",
                    "parameters": ["column_name", "original_column"],
                    "conditions": "Apply when tenant names have inconsistent formatting"
                }}
            ],
            "data_quality_improvements": [
                "List of data quality issues that were resolved"
            ],
            "reusable_patterns": [
                "Pattern 1: Column standardization",
                "Pattern 2: Missing value handling"
            ],
            "business_insights": [
                "Key insights discovered during analysis"
            ],
            "prerequisites": [
                "What conditions must be met for this template to work"
            ],
            "instructions_for_reuse": [
                "Step 1: Upload new rent roll file",
                "Step 2: Map columns (if different names)",
                "Step 3: Apply transformations in order"
            ]
        }}
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4.1",  # Using latest GPT-4
                messages=[
                    {"role": "system", "content": "You are an expert data analyst who creates reusable workflow templates from analysis sessions. Provide detailed, actionable instructions."},
                    {"role": "user", "content": analysis_prompt}
                ],
                max_tokens=4000,
                temperature=0.3
            )

            # Extract the analysis
            gpt_analysis = response.choices[0].message.content

            # Try to extract JSON from the response
            json_match = re.search(r'{.*}', gpt_analysis, re.DOTALL)
            if json_match:
                try:
                    workflow_analysis = json.loads(json_match.group(0))
                except:
                    # Fallback if JSON parsing fails
                    workflow_analysis = {"analysis": gpt_analysis}
            else:
                workflow_analysis = {"analysis": gpt_analysis}

        except Exception as e:
            print(f"❌ Error with GPT-4 analysis: {e}")
            workflow_analysis = {"error": str(e), "fallback_analysis": "Manual analysis required"}

        # Create template with all data
        template_id = f"template_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        # Save dataframes as text files
        starting_df_file = f"{template_id}_starting_df.txt"
        final_df_file = f"{template_id}_final_df.txt"
        session_file = f"{template_id}_session.txt"

        starting_df_path = os.path.join(self.templates_dir, starting_df_file)
        final_df_path = os.path.join(self.templates_dir, final_df_file)
        session_path = os.path.join(self.templates_dir, session_file)

        # Save dataframes
        starting_df.to_csv(starting_df_path, index=False)
        final_df.to_csv(final_df_path, index=False)

        # Save raw session data
        with open(session_path, 'w', encoding='utf-8') as f:
            f.write(session_context)

        # Create comprehensive template
        template_data = {
            "template_id": template_id,
            "template_name": template_name,
            "created_date": datetime.now().isoformat(),
            "source_session_id": session_data.get("session_id", "unknown"),
            "source_file_name": session_data.get("rent_roll_file", "unknown"),

            "files": {
                "starting_dataframe": starting_df_file,
                "final_dataframe": final_df_file,
                "raw_session": session_file
            },

            "session_summary": {
                "duration_minutes": session_data.get("duration_minutes", 0),
                "total_conversations": len(session_data.get("conversation_history", [])),
                "code_executions": len(session_data.get("code_executions", [])),
                "versions_created": len(session_data.get("dataframe_versions", [])),
                "issues_found": len(session_data.get("issues_found", []))
            },

            "gpt4_analysis": workflow_analysis,

            "raw_workflow_steps": session_data.get("conversation_history", []),
            "code_executions": session_data.get("code_executions", []),
            "dataframe_changes": session_data.get("dataframe_versions", []),
            "issues_identified": session_data.get("issues_found", [])
        }

        # Save template metadata
        template_json_path = os.path.join(self.templates_dir, f"{template_id}.json")
        with open(template_json_path, 'w') as f:
            json.dump(template_data, f, indent=2, default=str)

        print(f"✅ Comprehensive template created: {template_id}")
        print(f"📁 Starting DF: {starting_df_path}")
        print(f"📁 Final DF: {final_df_path}")
        print(f"📁 Session Data: {session_path}")
        print(f"📋 Template: {template_json_path}")

        return template_data

    def _prepare_session_context(self, session_data):
        """Prepare session data for GPT-4 analysis"""
        context = f"""
COPILOTING SESSION ANALYSIS
============================

Session ID: {session_data.get('session_id', 'N/A')}
Duration: {session_data.get('duration_minutes', 0):.1f} minutes
Rent Roll File: {session_data.get('rent_roll_file', 'N/A')}

CONVERSATION HISTORY:
"""

        for i, conv in enumerate(session_data.get('conversation_history', []), 1):
            context += f"""
--- Conversation {i} [{conv.get('timestamp', 'N/A')}] ---
USER QUERY: {conv.get('user_message', 'N/A')}
ACTION TYPE: {conv.get('action_type', 'N/A')}
"""
            if conv.get('code_executed'):
                context += f"CODE EXECUTED:\n{conv['code_executed']}\n"

            context += f"AI RESPONSE: {conv.get('ai_response', 'N/A')[:300]}...\n"

            if conv.get('version_saved'):
                context += f"VERSION SAVED: {conv['version_saved']}\n"

            context += "\n"

        context += "\nCODE EXECUTIONS SUMMARY:\n"
        for code_exec in session_data.get('code_executions', []):
            context += f"- [{code_exec.get('timestamp')}] {code_exec.get('purpose', 'N/A')}\n"
            context += f"  Code: {code_exec.get('code', 'N/A')[:100]}...\n"

        context += "\nISSUES IDENTIFIED:\n"
        for issue in session_data.get('issues_found', []):
            context += f"- [{issue.get('timestamp')}] {issue.get('description', 'N/A')}\n"

        context += "\nDATAFRAME VERSIONS:\n"
        for version in session_data.get('dataframe_versions', []):
            context += f"- {version.get('version_name', 'N/A')}: {version.get('description', 'N/A')}\n"

        return context

    def load_template_dataframes(self, template_id):
        """Load both starting and final dataframes from a template"""
        try:
            # Load template metadata
            template_json_path = os.path.join(self.templates_dir, f"{template_id}.json")
            with open(template_json_path, 'r') as f:
                template_data = json.load(f)

            # Load starting dataframe
            starting_df_path = os.path.join(self.templates_dir, template_data["files"]["starting_dataframe"])
            starting_df = pd.read_csv(starting_df_path)

            # Load final dataframe
            final_df_path = os.path.join(self.templates_dir, template_data["files"]["final_dataframe"])
            final_df = pd.read_csv(final_df_path)

            return template_data, starting_df, final_df

        except Exception as e:
            print(f"❌ Error loading template: {e}")
            return None, None, None

    def list_templates(self):
        """List all available templates"""
        try:
            json_files = [f for f in os.listdir(self.templates_dir) if f.endswith('.json')]
            templates = []

            for json_file in json_files:
                template_path = os.path.join(self.templates_dir, json_file)
                with open(template_path, 'r') as f:
                    template_data = json.load(f)

                templates.append({
                    "template_id": template_data["template_id"],
                    "template_name": template_data["template_name"],
                    "created_date": template_data["created_date"],
                    "source_file": template_data["source_file_name"],
                    "steps_count": len(template_data.get("raw_workflow_steps", [])),
                    "gpt4_analysis_available": "gpt4_analysis" in template_data
                })

            return sorted(templates, key=lambda x: x["created_date"], reverse=True)

        except Exception as e:
            print(f"❌ Error listing templates: {e}")
            return []

    def get_template_summary(self, template_id):
        """Get a human-readable summary of a template"""
        try:
            template_json_path = os.path.join(self.templates_dir, f"{template_id}.json")
            with open(template_json_path, 'r') as f:
                template_data = json.load(f)

            summary = f"""
📋 Template: {template_data.get('template_name', 'Unknown')}
🆔 ID: {template_data.get('template_id', 'Unknown')}
📅 Created: {template_data.get('created_date', 'Unknown')}
📁 Source File: {template_data.get('source_file_name', 'Unknown')}

📊 Session Summary:
• Duration: {template_data.get('session_summary', {}).get('duration_minutes', 0):.1f} minutes
• Conversations: {template_data.get('session_summary', {}).get('total_conversations', 0)}
• Code Executions: {template_data.get('session_summary', {}).get('code_executions', 0)}
• Versions Created: {template_data.get('session_summary', {}).get('versions_created', 0)}
• Issues Found: {template_data.get('session_summary', {}).get('issues_found', 0)}

🤖 GPT-4 Analysis: {'✅ Available' if 'gpt4_analysis' in template_data else '❌ Not Available'}
"""

            # Add GPT-4 analysis summary if available
            if 'gpt4_analysis' in template_data and isinstance(template_data['gpt4_analysis'], dict):
                gpt_analysis = template_data['gpt4_analysis']

                if 'workflow_summary' in gpt_analysis:
                    summary += f"\n🔍 Workflow Summary:\n{gpt_analysis['workflow_summary']}\n"

                if 'key_transformations' in gpt_analysis:
                    summary += f"\n🔧 Key Transformations ({len(gpt_analysis['key_transformations'])}):\n"
                    for i, transform in enumerate(gpt_analysis['key_transformations'][:3], 1):  # Show first 3
                        summary += f"{i}. {transform.get('step_name', 'Unknown')}: {transform.get('description', 'No description')}\n"
                    if len(gpt_analysis['key_transformations']) > 3:
                        summary += f"... and {len(gpt_analysis['key_transformations']) - 3} more\n"

                if 'prerequisites' in gpt_analysis:
                    summary += f"\n📋 Prerequisites:\n"
                    for prereq in gpt_analysis['prerequisites'][:3]:  # Show first 3
                        summary += f"• {prereq}\n"

            return summary

        except Exception as e:
            return f"❌ Error getting template summary: {str(e)}"

    def delete_template(self, template_id):
        """Delete a template and all its associated files"""
        try:
            template_json_path = os.path.join(self.templates_dir, f"{template_id}.json")

            if not os.path.exists(template_json_path):
                return f"❌ Template {template_id} not found"

            # Load template to get file list
            with open(template_json_path, 'r') as f:
                template_data = json.load(f)

            files_to_delete = []
            files_to_delete.append(template_json_path)  # The main template file

            # Add dataframe and session files
            if 'files' in template_data:
                for file_key, filename in template_data['files'].items():
                    file_path = os.path.join(self.templates_dir, filename)
                    if os.path.exists(file_path):
                        files_to_delete.append(file_path)

            # Delete all files
            deleted_count = 0
            for file_path in files_to_delete:
                try:
                    os.remove(file_path)
                    deleted_count += 1
                except Exception as e:
                    print(f"Warning: Could not delete {file_path}: {e}")

            return f"✅ Template {template_id} deleted successfully. Removed {deleted_count} files."

        except Exception as e:
            return f"❌ Error deleting template: {str(e)}"

# Global enhanced template manager
enhanced_template_manager = EnhancedTemplateManager()

In [ ]:
def create_template_from_current_session():
    """Create template from current copiloting session"""
    global app_state, session_recorder, enhanced_template_manager

    if not session_recorder.current_session_file:
        return "❌ No active session to create template from"

    if app_state is None or app_state["df"] is None:
        return "❌ No dataframe loaded"

    try:
        # Get starting dataframe (first version)
        if app_state.get("df_versions") and len(app_state["df_versions"]) > 0:
            first_version = app_state["df_versions"][0]
            starting_df_path = first_version.get("filename") or first_version.get("csv_filename")
            starting_df = pd.read_csv(starting_df_path)
        else:
            starting_df = app_state["df"]  # Fallback if no versions

        # Current dataframe is the final version
        final_df = app_state["df"]

        # Finalize current session
        session_data = session_recorder.finalize_session()

        if session_data is None:
            return "❌ Error finalizing session"

        # Generate template name suggestion
        template_name = f"Rent Roll Process {datetime.now().strftime('%Y-%m-%d %H:%M')}"

        # Create comprehensive template
        template_data = enhanced_template_manager.create_template_from_session(
            session_data=session_data,
            starting_df=starting_df,
            final_df=final_df,
            template_name=template_name
        )

        return f"✅ Template created successfully!\nTemplate ID: {template_data['template_id']}\nSteps captured: {len(session_data.get('conversation_history', []))}"

    except Exception as e:
        return f"❌ Error creating template: {str(e)}"

In [ ]:
# Global state for the application (Not part of graph state)
app_state = {
    "df": None,
    "anthropic_client": None,
    "openai_client": None,  # Added for GPT-4
    "issues": [],
    "system_message": ""
}
# Enhanced Chat Function with Complete Session Recording and Template Generation

def chat(message, history):
    """
    Enhanced chat function with comprehensive session recording and template generation.
    Records every interaction, code execution, and dataframe change for template creation.
    """
    global app_state, session_recorder, enhanced_template_manager

    logger.info(f"Received chat message: {message[:50]}...")

    # Check if system is ready
    if app_state is None or app_state["df"] is None:
        logger.warning("Chat attempted before setup is complete")
        return history + [(message, "Please upload a rent roll file and set up your API keys first.")]

    # Start session recording if not already started
    if not session_recorder.current_session_file:
        rent_roll_filename = getattr(app_state, 'original_filename', 'uploaded_rent_roll.xlsx')
        session_id = session_recorder.start_session_recording(rent_roll_filename)
        logger.info(f"Started new session recording: {session_id}")

        # Record initial dataframe state
        if app_state.get("df_versions") and len(app_state["df_versions"]) > 0:
            first_version = app_state["df_versions"][0]
            session_recorder.record_dataframe_version(
                version_name=first_version["name"],
                description=first_version["description"],
                shape=list(app_state["df"].shape),
                columns=list(app_state["df"].columns)
            )

    # Get previous messages from history
    prev_messages = []
    if history:
        for user_msg, assistant_msg in history:
            prev_messages.append({"role": "user", "content": user_msg})
            prev_messages.append({"role": "assistant", "content": assistant_msg})

    # Create message list without system message
    all_messages = []
    all_messages.extend(prev_messages)

    # Add the current user message
    all_messages.append({"role": "user", "content": message})

    # Create a state dictionary for the graph
    state = {
        "messages": all_messages,
        "system_message": app_state["system_message"],
        "df": app_state["df"],
        "issues": app_state["issues"],
        "needs_clarification": False,
        "generate_code": False,
        "execution_plan": None,
        "clarification_question": None,
        "code_execution_results": None,
        "final_response": None,
        "anthropic_client": app_state["anthropic_client"],
        "openai_client": app_state["openai_client"]
    }

    try:
        # Create the workflow if not already created
        if not hasattr(chat, "workflow"):
            chat.workflow = create_agentic_rent_roll_analyzer()
            logger.info("Created agentic workflow")

        # Run the workflow with the current state
        logger.info("Running agentic workflow")
        result = chat.workflow.invoke(state)

        # Get the final response from the result state
        final_response = result.get("final_response", "I'm sorry, I couldn't process your request.")
        logger.info(f"Received final response from workflow: {final_response[:50]}...")

        # === ENHANCED SESSION RECORDING ===

        # 1. Determine action type based on response content and workflow state
        action_type = "analysis"  # default

        if result.get("needs_clarification"):
            action_type = "clarification"
        elif result.get("generate_code"):
            action_type = "data_processing"
        elif "error" in final_response.lower() or "sorry" in final_response.lower():
            action_type = "error_handling"
        elif "```python" in final_response:
            action_type = "code_execution"
        elif any(keyword in message.lower() for keyword in ["clean", "fix", "correct", "standardize"]):
            action_type = "data_cleaning"
        elif any(keyword in message.lower() for keyword in ["calculate", "compute", "sum", "average"]):
            action_type = "calculation"
        elif any(keyword in message.lower() for keyword in ["find", "show", "display", "list"]):
            action_type = "data_exploration"
        elif any(keyword in message.lower() for keyword in ["chart", "graph", "plot", "visualize"]):
            action_type = "visualization"

        # 2. Extract executed code from response
        code_executed = None
        code_blocks = re.findall(r'```python\s*(.*?)\s*```', final_response, re.DOTALL)
        if code_blocks:
            # Combine all code blocks if multiple
            code_executed = "\n\n# --- Next Code Block ---\n\n".join(code_blocks)

        # 3. Check if a new dataframe version was saved
        version_saved = None
        if "✓ Saved dataframe version" in final_response:
            version_match = re.search(r'version (v_\w+)', final_response)
            if version_match:
                version_saved = version_match.group(1)
                logger.info(f"Detected new version saved: {version_saved}")

        # 4. Detect if issues were found or resolved
        if any(keyword in final_response.lower() for keyword in ["issue", "problem", "error", "missing", "duplicate"]):
            issue_description = message + " - " + final_response[:100] + "..."
            severity = "high" if any(word in final_response.lower() for word in ["critical", "error", "failed"]) else "medium"
            session_recorder.record_issue_found(issue_description, severity)

        # 5. Extract any business insights or patterns
        insights = []
        if "found" in final_response.lower() and any(word in final_response.lower() for word in ["units", "rent", "tenant"]):
            insights.append(f"Business insight from query: {message}")

        # 6. Record the complete conversation turn with enhanced metadata
        session_recorder.record_conversation_turn(
            user_message=message,
            ai_response=final_response,
            action_type=action_type,
            code_executed=code_executed,
            version_saved=version_saved
        )

        # 7. Record dataframe version details if saved
        if version_saved:
            # Find the latest version info
            latest_version = None
            if app_state.get("df_versions"):
                for version in app_state["df_versions"]:
                    if version["name"] == version_saved:
                        latest_version = version
                        break

            if latest_version:
                session_recorder.record_dataframe_version(
                    version_name=version_saved,
                    description=latest_version.get("description", "Auto-saved during copiloting"),
                    shape=list(app_state["df"].shape),
                    columns=list(app_state["df"].columns)
                )
            else:
                # Fallback if version not found in registry
                session_recorder.record_dataframe_version(
                    version_name=version_saved,
                    description="Auto-saved during copiloting session",
                    shape=list(app_state["df"].shape),
                    columns=list(app_state["df"].columns)
                )

        # 8. Track user goals and patterns
        user_goals = []
        if any(keyword in message.lower() for keyword in ["clean", "standardize", "fix"]):
            user_goals.append("Data cleaning and standardization")
        if any(keyword in message.lower() for keyword in ["analyze", "find", "calculate"]):
            user_goals.append("Data analysis and insights")
        if any(keyword in message.lower() for keyword in ["chart", "graph", "visualize"]):
            user_goals.append("Data visualization")

        if user_goals:
            session_recorder.current_session_data.setdefault("user_goals", []).extend(user_goals)

        # 9. Log session statistics
        if session_recorder.current_session_data:
            total_turns = len(session_recorder.current_session_data.get("conversation_history", []))
            total_code = len(session_recorder.current_session_data.get("code_executions", []))
            logger.info(f"Session stats - Turns: {total_turns}, Code executions: {total_code}")

        # Use the correct format for Gradio chatbot
        history_list = list(history) if history else []
        history_list.append((message, final_response))

        logger.info("Chat response processing complete with session recording")
        return history_list

    except Exception as e:
        logger.error(f"Error processing chat: {e}")
        logger.error(traceback.format_exc())

        # Record the error in session
        error_message = f"Error getting response: {str(e)}"

        if session_recorder.current_session_file:
            session_recorder.record_conversation_turn(
                user_message=message,
                ai_response=error_message,
                action_type="system_error",
                code_executed=None,
                version_saved=None
            )

            # Record as a system issue
            session_recorder.record_issue_found(
                f"System error during processing: {str(e)}",
                severity="high"
            )

        # Handle errors properly in the chat history format
        history_list = list(history) if history else []
        history_list.append((message, error_message))
        return history_list


def create_template_from_current_session(template_name_input=""):
    """
    Create a comprehensive template from the current copiloting session.
    This includes GPT-4.1 analysis of the entire workflow.
    """
    global app_state, session_recorder, enhanced_template_manager

    if not session_recorder.current_session_file:
        return "❌ No active copiloting session found. Please start chatting with the system first."

    if app_state is None or app_state["df"] is None:
        return "❌ No dataframe loaded. Cannot create template."

    try:
        logger.info("Starting template creation from current session...")

        # 1. Get starting dataframe (first version saved)
        starting_df = None
        if app_state.get("df_versions") and len(app_state["df_versions"]) > 0:
            # Load the original/first version
            first_version = app_state["df_versions"][0]
            starting_df_path = first_version.get("filename") or first_version.get("csv_filename")
            if starting_df_path and os.path.exists(starting_df_path):
                starting_df = pd.read_csv(starting_df_path)
                logger.info(f"Loaded starting dataframe from: {starting_df_path}")
            else:
                # Try to construct the path
                versions_dir = "rent_roll_versions"
                csv_filename = os.path.join(versions_dir, f"rent_roll_{first_version['name']}.csv")
                if os.path.exists(csv_filename):
                    starting_df = pd.read_csv(csv_filename)
                    logger.info(f"Loaded starting dataframe from: {csv_filename}")

        # Fallback: use current dataframe if no versions found
        if starting_df is None:
            starting_df = app_state["df"].copy()
            logger.warning("Using current dataframe as starting point (no version history found)")

        # 2. Current dataframe is the final version
        final_df = app_state["df"].copy()

        # 3. Finalize current session to get complete session data
        logger.info("Finalizing current session...")
        session_data = session_recorder.finalize_session()

        if session_data is None:
            return "❌ Error finalizing session data."

        # 4. Generate template name if not provided
        if not template_name_input.strip():
            rent_roll_file = session_data.get('rent_roll_file', 'Unknown')
            timestamp = datetime.now().strftime('%Y-%m-%d')
            template_name = f"Rent Roll Process - {rent_roll_file} - {timestamp}"
        else:
            template_name = template_name_input.strip()

        # 5. Create comprehensive template using GPT-4.1 analysis
        logger.info("Creating template with GPT-4.1 analysis...")
        template_data = enhanced_template_manager.create_template_from_session(
            session_data=session_data,
            starting_df=starting_df,
            final_df=final_df,
            template_name=template_name
        )

        # 6. Prepare success message with details
        session_stats = session_data.get('session_summary', {})
        success_message = f"""✅ Template Created Successfully!

📋 Template Details:
• Template ID: {template_data['template_id']}
• Template Name: {template_name}
• Source File: {session_data.get('rent_roll_file', 'Unknown')}

📊 Session Summary:
• Duration: {session_stats.get('duration_minutes', 0):.1f} minutes
• Conversations: {session_stats.get('total_conversations', 0)}
• Code Executions: {session_stats.get('code_executions', 0)}
• Versions Created: {session_stats.get('versions_created', 0)}
• Issues Found: {session_stats.get('issues_found', 0)}

📁 Files Created:
• Starting Dataframe: {template_data['files']['starting_dataframe']}
• Final Dataframe: {template_data['files']['final_dataframe']}
• Session Recording: {template_data['files']['raw_session']}
• Template Metadata: {template_data['template_id']}.json

🤖 GPT-4.1 Analysis: {'✅ Completed' if 'gpt4_analysis' in template_data else '❌ Failed'}

This template can now be applied to similar rent roll files using the Template Manager."""

        logger.info(f"Template creation completed: {template_data['template_id']}")
        return success_message

    except Exception as e:
        error_msg = f"❌ Error creating template: {str(e)}"
        logger.error(f"Template creation failed: {e}")
        logger.error(traceback.format_exc())
        return error_msg


def end_current_session():
    """
    Manually end the current copiloting session without creating a template.
    Useful for starting fresh or when session gets too long.
    """
    global session_recorder

    if not session_recorder.current_session_file:
        return "ℹ️ No active session to end."

    try:
        session_data = session_recorder.finalize_session()

        if session_data:
            session_stats = {
                'duration': session_data.get('duration_minutes', 0),
                'conversations': len(session_data.get('conversation_history', [])),
                'code_executions': len(session_data.get('code_executions', [])),
                'versions': len(session_data.get('dataframe_versions', []))
            }

            return f"""✅ Session Ended Successfully

            📊 Final Session Statistics:
            • Session ID: {session_data.get('session_id', 'Unknown')}
            • Duration: {session_stats['duration']:.1f} minutes
            • Total Conversations: {session_stats['conversations']}
            • Code Executions: {session_stats['code_executions']}
            • Dataframe Versions: {session_stats['versions']}

            💾 Session data saved to: {session_data.get('session_id', 'unknown')}.txt

            You can now start a new session or create a template from this completed session."""
        else:
            return "⚠️ Session ended but no data was saved."

    except Exception as e:
        return f"❌ Error ending session: {str(e)}"


# Additional helper function to get session status
def get_current_session_status():
    """Get the current session recording status and statistics."""
    global session_recorder

    if not session_recorder.current_session_file:
        return "📴 No active session recording"

    try:
        if session_recorder.current_session_data:
            data = session_recorder.current_session_data
            start_time = datetime.fromisoformat(data.get('start_time', datetime.now().isoformat()))
            duration = (datetime.now() - start_time).total_seconds() / 60

            status = f"""📹 Session Recording Active

            📊 Current Statistics:
            • Session ID: {data.get('session_id', 'Unknown')}
            • Duration: {duration:.1f} minutes
            • Conversations: {len(data.get('conversation_history', []))}
            • Code Executions: {len(data.get('code_executions', []))}
            • Versions Created: {len(data.get('dataframe_versions', []))}
            • Issues Found: {len(data.get('issues_found', []))}

            📁 Recording File: {session_recorder.current_session_file}

            All interactions are being automatically recorded for template creation."""

            return status
        else:
            return "📹 Session recording active but no data collected yet"

    except Exception as e:
        return f"❌ Error getting session status: {str(e)}"

In [ ]:
def view_data():
    """Return a preview of the rent roll data."""
    global app_state  # Use app_state instead of agent_state

    logger.info("View data requested")

    if app_state is None or app_state["df"] is None:  # Note the dictionary access with ["df"]
        logger.warning("View data requested but no data is loaded")
        return "No rent roll data loaded yet."

    # Generate HTML representation of the dataframe
    logger.info(f"Generating HTML preview of data with {len(app_state['df'])} rows")
    html = f"""
    <h3>Rent Roll Data</h3>
    <p>{len(app_state['df'])} rows × {len(app_state['df'].columns)} columns</p>
    {app_state['df'].head(10).fillna('').to_html(index=False)}
    """

    return html

In [ ]:

def clear_chat():
    """Reset the chat history."""
    logger.info("Clearing chat history")
    return []  # Return empty list for Gradio chat history

In [ ]:
def view_dataframe_versions():
    """Return HTML showing all versions of the rent roll dataframe."""
    global app_state
    logger.info("View dataframe versions requested")

    versions_dir = "rent_roll_versions"

    if not os.path.exists(versions_dir):
        logger.warning("No versions directory found")
        return "No version history found. Please save a version first."

    # Get all files in the versions directory
    try:
        all_files = os.listdir(versions_dir)
        # Match any CSV file containing rent_roll in the name
        version_files = [f for f in all_files if f.endswith('.csv') and 'rent_roll' in f]
    except Exception as e:
        logger.error(f"Error reading versions directory: {e}")
        return f"Error listing versions: {str(e)}"

    if not version_files:
        logger.warning("No version files found in directory")
        return f"No version files found in the versions directory ({versions_dir})."

    # Extract version information
    versions = []
    for file in version_files:
        # Extract the version name from the filename
        if file.startswith('rent_roll_v_'):
            version_name = file.replace('rent_roll_', '').replace('.csv', '')
        else:
            version_name = os.path.splitext(file)[0].replace('rent_roll_', '')

        # Get file stats
        try:
            file_path = os.path.join(versions_dir, file)
            file_stats = os.stat(file_path)
            file_size = file_stats.st_size
            modified_time = datetime.fromtimestamp(file_stats.st_mtime).strftime("%Y-%m-%d %H:%M:%S")

            # Try to get row and column counts
            df_info = ""
            try:
                temp_df = pd.read_csv(file_path)
                df_info = f"{len(temp_df)} rows × {len(temp_df.columns)} columns"
            except:
                df_info = "Unable to read file"

            # If we have version info in app_state
            description = ""
            is_original = False

            for v in app_state.get("df_versions", []):
                if v.get("name") == version_name:
                    description = v.get("description", "")
                    is_original = v.get("is_original", False)
                    break

            # If not found in app_state, use fallback description
            if not description and os.path.exists(file_path):
                description = "Found in directory"

            versions.append({
                'version_name': version_name,
                'file_size': file_size,
                'modified_time': modified_time,
                'df_info': df_info,
                'description': description,
                'is_original': is_original,
                'file_path': file_path
            })
        except Exception as e:
            logger.error(f"Error processing version file {file}: {e}")
            versions.append({
                'version_name': version_name,
                'file_size': 0,
                'modified_time': 'Error',
                'df_info': f"Error: {str(e)}",
                'description': '',
                'is_original': False,
                'file_path': os.path.join(versions_dir, file)
            })

    # Sort versions by modification time
    versions.sort(key=lambda x: x['modified_time'])

    # Create basic HTML table without zebra striping
    html = """
    <h3 style="color: white;">Rent Roll Dataframe Version History</h3>
    """

    html += f"""
    <p style="color: white;">Found {len(versions)} version(s) in {versions_dir}</p>
    <table border="1" cellpadding="5" cellspacing="0" style="width: 100%; border-collapse: collapse; color: white;">
        <thead style="background-color: #009879;">
            <tr>
                <th style="text-align: left; padding: 10px;">Version Name</th>
                <th style="text-align: left; padding: 10px;">Status</th>
                <th style="text-align: left; padding: 10px;">Created</th>
                <th style="text-align: left; padding: 10px;">Size</th>
                <th style="text-align: left; padding: 10px;">Data</th>
                <th style="text-align: left; padding: 10px;">Description</th>
            </tr>
        </thead>
        <tbody>
    """

    for i, v in enumerate(versions):
        # No alternating rows - all cells have the same background and text color
        # Always use dark background with white text for all rows

        # Determine status badge
        if i == 0 or v.get('is_original'):
            status_html = '<span style="background-color: #3949ab; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">ORIGINAL</span>'
        elif i == len(versions) - 1:
            status_html = '<span style="background-color: #43a047; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">LATEST</span>'
        else:
            # Middle version with orange badge
            status_html = f'<span style="background-color: #f57c00; color: white; padding: 3px 6px; border-radius: 3px; display: inline-block;">v{i+1}</span>'

        # All rows have dark background and white text
        html += f"""
        <tr style="background-color: #25292e; color: white; border-bottom: 1px solid #333;">
            <td style="padding: 10px;"><code style="font-family: monospace; font-weight: bold;">{v['version_name']}</code></td>
            <td style="padding: 10px;">{status_html}</td>
            <td style="padding: 10px;">{v['modified_time']}</td>
            <td style="padding: 10px;">{round(v['file_size']/1024, 2)} KB</td>
            <td style="padding: 10px;">{v['df_info']}</td>
            <td style="padding: 10px;">{v['description']}</td>
        </tr>
        """

    html += """
        </tbody>
    </table>
    """

    logger.info(f"Generated version history display with {len(versions)} versions")
    return html

In [ ]:
# def analyze_dataframe_changes_with_gpt4(original_df, modified_df, user_description=""):
#     """
#     Use GPT-4.1 to analyze differences between original and modified dataframes,
#     generate a detailed description of changes made, AND test Claude prompts.
#     NOW USES COMPLETE DATAFRAMES FOR ANALYSIS AND TESTS REPLICATION.
#     AUTOMATICALLY SAVES ALL LOGS TO TEXT FILES WHENEVER EXECUTED.
#     ENHANCED WITH GPT-4.1 INTELLIGENT FEEDBACK LOOP FOR REPLICATION TESTING.
#     """
#     import os
#     import json
#     import traceback
#     from datetime import datetime

#     # Create detailed logging directory
#     logs_dir = "manual_edit_analysis_logs"
#     os.makedirs(logs_dir, exist_ok=True)

#     # Generate unique log session ID
#     log_session_id = f"enhanced_edit_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')[:-3]}"  # Include milliseconds for uniqueness
#     log_file_path = os.path.join(logs_dir, f"{log_session_id}_detailed_analysis.txt")

#     # Enhanced logging function that ALWAYS saves to file
#     def log_to_file(content, section_title=""):
#         try:
#             with open(log_file_path, 'a', encoding='utf-8') as f:
#                 if section_title:
#                     f.write(f"\n{'='*80}\n")
#                     f.write(f"{section_title}\n")
#                     f.write(f"{'='*80}\n")
#                 f.write(f"{content}\n")
#                 f.flush()  # Ensure immediate write to disk
#         except Exception as e:
#             print(f"ERROR writing to log file: {e}")

#     # ALWAYS initialize comprehensive log - even if function fails later
#     try:
#         log_to_file(f"""ENHANCED MANUAL EDIT ANALYSIS LOG WITH GPT-4.1 FEEDBACK LOOP
# Session ID: {log_session_id}
# Enhancement: Intelligent AI feedback architecture
# Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# User Description: "{user_description}"
# Log File Path: {log_file_path}

# ANALYSIS OVERVIEW:
# This log contains the complete workflow of analyzing manual dataframe edits:
# 1. Original vs Modified dataframe comparison
# 2. GPT-4.1 prompt generation for Claude
# 3. Enhanced Claude API calls with intelligent feedback loop
# 4. GPT-4.1 gap analysis and prompt improvement
# 5. Progressive replication attempts with learning
# 6. Final analysis with working prompts embedded

# FUNCTION EXECUTION STATUS: STARTING ENHANCED VERSION...
# """, "ENHANCED SESSION INITIALIZATION")

#         print(f"📝 Enhanced manual edit analysis log initialized: {log_file_path}")

#     except Exception as init_error:
#         print(f"CRITICAL: Could not initialize log file: {init_error}")
#         # Continue execution even if logging fails

#     try:
#         # ALWAYS log dataframe information - even if analysis fails later
#         log_to_file(f"""ORIGINAL DATAFRAME COMPLETE ANALYSIS:
# Shape: {original_df.shape}
# Columns: {list(original_df.columns)}
# Data Types: {dict(original_df.dtypes.astype(str))}
# Memory Usage: {original_df.memory_usage(deep=True).sum()} bytes
# Null Counts: {dict(original_df.isnull().sum())}

# FIRST 10 ROWS PREVIEW:
# {original_df.head(10).to_string()}

# COMPLETE ORIGINAL DATAFRAME (ALL ROWS):
# {original_df.to_string(max_rows=None, max_cols=None)}

# ORIGINAL DATAFRAME AS CSV:
# {original_df.to_csv(index=False)}
# """, "ORIGINAL DATAFRAME ANALYSIS")

#         log_to_file(f"""MODIFIED DATAFRAME COMPLETE ANALYSIS:
# Shape: {modified_df.shape}
# Columns: {list(modified_df.columns)}
# Data Types: {dict(modified_df.dtypes.astype(str))}
# Memory Usage: {modified_df.memory_usage(deep=True).sum()} bytes
# Null Counts: {dict(modified_df.isnull().sum())}

# FIRST 10 ROWS PREVIEW:
# {modified_df.head(10).to_string()}

# COMPLETE MODIFIED DATAFRAME (ALL ROWS):
# {modified_df.to_string(max_rows=None, max_cols=None)}

# MODIFIED DATAFRAME AS CSV:
# {modified_df.to_csv(index=False)}
# """, "MODIFIED DATAFRAME ANALYSIS")

#         # Initialize OpenAI client
#         try:
#             client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
#             log_to_file("✅ OpenAI client initialized successfully", "API CLIENT SETUP")
#         except Exception as client_error:
#             log_to_file(f"❌ Failed to initialize OpenAI client: {client_error}", "API CLIENT SETUP ERROR")
#             raise client_error

#         # Prepare comparison data for GPT-4 - NOW WITH COMPLETE DATAFRAMES
#         original_info = {
#             "shape": original_df.shape,
#             "columns": list(original_df.columns),
#             "dtypes": dict(original_df.dtypes.astype(str)),
#             "full_data": original_df.to_string(max_rows=None, max_cols=None),  # COMPLETE dataframe
#             "full_csv": original_df.to_csv(index=False),  # Alternative format
#             "null_counts": dict(original_df.isnull().sum()),
#             "memory_usage": original_df.memory_usage(deep=True).sum(),
#             "summary_stats": original_df.describe(include='all').to_string() if len(original_df) > 0 else "No data"
#         }

#         modified_info = {
#             "shape": modified_df.shape,
#             "columns": list(modified_df.columns),
#             "dtypes": dict(modified_df.dtypes.astype(str)),
#             "full_data": modified_df.to_string(max_rows=None, max_cols=None),  # COMPLETE dataframe
#             "full_csv": modified_df.to_csv(index=False),  # Alternative format
#             "null_counts": dict(modified_df.isnull().sum()),
#             "memory_usage": modified_df.memory_usage(deep=True).sum(),
#             "summary_stats": modified_df.describe(include='all').to_string() if len(modified_df) > 0 else "No data"
#         }

#         # Detect specific changes
#         shape_changed = original_df.shape != modified_df.shape
#         columns_changed = set(original_df.columns) != set(modified_df.columns)

#         log_to_file(f"""STRUCTURAL CHANGES DETECTED:
# Shape Changed: {shape_changed}
# - Original Shape: {original_df.shape}
# - Modified Shape: {modified_df.shape}
# - Rows Added: {max(0, modified_df.shape[0] - original_df.shape[0])}
# - Rows Removed: {max(0, original_df.shape[0] - modified_df.shape[0])}
# - Columns Added: {max(0, modified_df.shape[1] - original_df.shape[1])}
# - Columns Removed: {max(0, original_df.shape[1] - modified_df.shape[1])}

# Columns Changed: {columns_changed}
# - Original Columns: {list(original_df.columns)}
# - Modified Columns: {list(modified_df.columns)}
# - Added Columns: {list(set(modified_df.columns) - set(original_df.columns))}
# - Removed Columns: {list(set(original_df.columns) - set(modified_df.columns))}
# """, "STRUCTURAL CHANGE ANALYSIS")

#         # COMPLETE cell-by-cell comparison - ANALYZE EVERY SINGLE CELL
#         data_changes_detected = False
#         changed_cells = []
#         total_changes = 0
#         changed_rows = set()
#         changed_columns = set()

#         if original_df.shape == modified_df.shape and list(original_df.columns) == list(modified_df.columns):
#             print(f"🔍 Comparing ALL {len(original_df)} rows and {len(original_df.columns)} columns...")
#             log_to_file(f"""STARTING COMPLETE CELL-BY-CELL COMPARISON:
# Total cells to compare: {original_df.shape[0] * original_df.shape[1]}
# Comparing {len(original_df)} rows × {len(original_df.columns)} columns
# This may take time for large dataframes...
# """, "CELL-BY-CELL COMPARISON START")

#             # Compare EVERY cell in the entire dataframe
#             for i in range(len(original_df)):
#                 row_has_changes = False
#                 row_changes = []

#                 for col in original_df.columns:
#                     try:
#                         orig_val = original_df.iloc[i][col]
#                         mod_val = modified_df.iloc[i][col]

#                         # Handle NaN comparisons
#                         if pd.isna(orig_val) and pd.isna(mod_val):
#                             continue
#                         elif pd.isna(orig_val) or pd.isna(mod_val):
#                             data_changes_detected = True
#                             total_changes += 1
#                             row_has_changes = True
#                             changed_columns.add(col)
#                             change_detail = {
#                                 "row": i,
#                                 "column": col,
#                                 "original": str(orig_val),
#                                 "modified": str(mod_val),
#                                 "change_type": "nan_change"
#                             }
#                             row_changes.append(change_detail)
#                             # Store ALL changes, not just first 50
#                             changed_cells.append(change_detail)
#                         elif str(orig_val).strip() != str(mod_val).strip():
#                             data_changes_detected = True
#                             total_changes += 1
#                             row_has_changes = True
#                             changed_columns.add(col)
#                             change_detail = {
#                                 "row": i,
#                                 "column": col,
#                                 "original": str(orig_val),
#                                 "modified": str(mod_val),
#                                 "change_type": "value_change"
#                             }
#                             row_changes.append(change_detail)
#                             # Store ALL changes, not just first 50
#                             changed_cells.append(change_detail)
#                     except Exception as e:
#                         log_to_file(f"ERROR comparing cell at row {i}, column '{col}': {str(e)}")
#                         continue

#                 if row_has_changes:
#                     changed_rows.add(i)
#                     # Log each changed row immediately
#                     log_to_file(f"""ROW {i} CHANGES ({len(row_changes)} changes):
# {json.dumps(row_changes, indent=2)}
# """)

#                 # Log progress every 100 rows for large dataframes
#                 if (i + 1) % 100 == 0:
#                     log_to_file(f"Progress: Processed {i + 1}/{len(original_df)} rows, found {total_changes} changes so far")

#             print(f"✅ Complete comparison finished: {total_changes} total changes detected across {len(changed_rows)} rows")

#             # Log comprehensive change analysis
#             log_to_file(f"""COMPLETE CELL-BY-CELL COMPARISON RESULTS:
# =====================================
# Total Changes Detected: {total_changes}
# Affected Rows: {len(changed_rows)} out of {original_df.shape[0]} ({len(changed_rows)/original_df.shape[0]*100:.1f}%)
# Affected Columns: {len(changed_columns)} out of {len(original_df.columns)} ({len(changed_columns)/len(original_df.columns)*100:.1f}%)

# AFFECTED COLUMNS LIST:
# {list(changed_columns)}

# AFFECTED ROWS LIST:
# {sorted(list(changed_rows))}

# ALL DETECTED CHANGES ({len(changed_cells)} total):
# """, "COMPREHENSIVE CHANGE DETECTION RESULTS")

#             # Log ALL changes, not just a sample
#             for i, change in enumerate(changed_cells):
#                 log_to_file(f"Change {i+1}: Row {change['row']}, Column '{change['column']}' ({change['change_type']}): '{change['original']}' → '{change['modified']}'")

#         else:
#             log_to_file(f"""CANNOT PERFORM CELL-BY-CELL COMPARISON:
# Reason: Shape or column structure differs
# Original shape: {original_df.shape}
# Modified shape: {modified_df.shape}
# Original columns: {list(original_df.columns)}
# Modified columns: {list(modified_df.columns)}
# """, "CELL-BY-CELL COMPARISON SKIPPED")

#         # Calculate change statistics
#         total_cells = original_df.shape[0] * original_df.shape[1] if original_df.size > 0 else 1
#         change_density = total_changes / total_cells

#         log_to_file(f"""COMPREHENSIVE CHANGE STATISTICS:
# ===============================
# Total Cells in Original: {total_cells}
# Total Cells Changed: {total_changes}
# Change Density: {change_density*100:.4f}%
# Percentage of Rows Affected: {len(changed_rows)/original_df.shape[0]*100:.2f}% ({len(changed_rows)}/{original_df.shape[0]})
# Percentage of Columns Affected: {len(changed_columns)/len(original_df.columns)*100:.2f}% ({len(changed_columns)}/{len(original_df.columns)})

# CHANGE PATTERN ANALYSIS:
# - NaN Changes: {len([c for c in changed_cells if c.get('change_type') == 'nan_change'])}
# - Value Changes: {len([c for c in changed_cells if c.get('change_type') == 'value_change'])}
# - Most Affected Columns: {sorted(changed_columns)[:10]}
# - Row Change Distribution: Every {original_df.shape[0]//max(1,len(changed_rows)):.0f} rows on average
# """, "COMPREHENSIVE STATISTICAL ANALYSIS")

#         # STEP 1: Use GPT-4.1 to analyze and generate Claude prompt
#         print("🧠 GPT-4.1: Analyzing changes and generating Claude prompt...")
#         log_to_file("🧠 STARTING GPT-4.1 ANALYSIS TO GENERATE CLAUDE PROMPT...", "GPT-4.1 PROMPT GENERATION START")

#         claude_prompt = _generate_claude_prompt_with_gpt4_logged(
#             client, original_info, modified_info, user_description,
#             total_changes, changed_rows, changed_columns, changed_cells,
#             log_to_file
#         )

#         # STEP 2: Test the Claude prompt with ENHANCED GPT-4.1 FEEDBACK LOOP
#         print("🧠 Testing Claude prompt with GPT-4.1 intelligent feedback loop...")
#         log_to_file("🧠 STARTING ENHANCED CLAUDE REPLICATION WITH AI FEEDBACK LOOP...", "ENHANCED CLAUDE REPLICATION START")

#         replication_results = _test_claude_prompt_replication_logged(
#             original_df, modified_df, claude_prompt, max_attempts=3, log_to_file=log_to_file
#         )

#         # STEP 3: Get final analysis with working prompts embedded
#         log_to_file("📊 GENERATING FINAL ANALYSIS WITH GPT-4.1...", "FINAL ANALYSIS GENERATION START")

#         final_analysis = _get_final_analysis_with_prompts_logged(
#             client, original_info, modified_info, user_description,
#             total_changes, changed_rows, changed_columns, changed_cells,
#             claude_prompt, replication_results, log_to_file
#         )

#         # Add technical metadata and log file reference
#         final_analysis["raw_gpt_response"] = final_analysis.get("raw_gpt_response", "")
#         final_analysis["complete_comparison_performed"] = True
#         final_analysis["log_file_path"] = log_file_path  # ALWAYS include log file path
#         final_analysis["enhancement_method"] = "gpt4_feedback_loop"
#         final_analysis["full_change_statistics"] = {
#             "total_cells": total_cells,
#             "total_changes": total_changes,
#             "change_density": change_density,
#             "affected_rows": list(changed_rows),
#             "affected_columns": list(changed_columns),
#             "replication_tested": True,
#             "replication_success": replication_results["final_success"],
#             "replication_attempts": len(replication_results["attempts"]),
#             "all_changes": changed_cells,  # Include ALL detected changes
#             "progressive_improvement": len([a for a in replication_results["attempts"] if a.get("enhancement_type") == "gpt4_enhanced"]) > 0
#         }

#         # Log final comprehensive results
#         log_to_file(f"""ENHANCED ANALYSIS EXECUTION COMPLETED SUCCESSFULLY
# =====================================================
# Enhancement Method: GPT-4.1 Intelligent Feedback Loop
# Total Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Log File Saved: {log_file_path}
# Log File Size: {os.path.getsize(log_file_path)} bytes

# ENHANCED FINAL RESULTS SUMMARY:
# - Replication Success: {replication_results["final_success"]}
# - Best Match Score: {replication_results.get("best_attempt", {}).get("match_score", 0):.2%}
# - Total Replication Attempts: {len(replication_results["attempts"])}
# - Progressive Improvement: {'Yes' if final_analysis["full_change_statistics"]["progressive_improvement"] else 'No'}
# - Changes Detected: {total_changes}
# - Change Density: {change_density*100:.4f}%

# ENHANCEMENT STATISTICS:
# - GPT-4.1 Enhanced Attempts: {len([a for a in replication_results["attempts"] if a.get("enhancement_type") == "gpt4_enhanced"])}
# - Gap Analysis Performed: {len([a for a in replication_results["attempts"] if a.get("gap_analysis")])}
# - Prompt Improvements: {len([a for a in replication_results["attempts"] if a.get("enhancement_type") == "gpt4_enhanced"])}

# COMPLETE FINAL ANALYSIS OBJECT:
# {json.dumps(final_analysis, indent=2, default=str)}

# ✅ ENHANCED ANALYSIS COMPLETE - ALL LOGS SAVED TO: {log_file_path}
# """, "ENHANCED FINAL EXECUTION RESULTS")

#         print(f"📝 Enhanced complete analysis with all logs saved to: {log_file_path}")
#         return final_analysis

#     except Exception as e:
#         # ALWAYS log errors, even if everything else fails
#         error_details = f"""CRITICAL ERROR DURING ENHANCED ANALYSIS EXECUTION
# ========================================================
# Error Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Error Type: {type(e).__name__}
# Error Message: {str(e)}

# FULL STACK TRACE:
# {traceback.format_exc()}

# EXECUTION CONTEXT:
# - User Description: "{user_description}"
# - Original DF Shape: {original_df.shape if 'original_df' in locals() else 'Unknown'}
# - Modified DF Shape: {modified_df.shape if 'modified_df' in locals() else 'Unknown'}
# - Enhancement Method: GPT-4.1 Feedback Loop
# - Log File: {log_file_path}

# ❌ ENHANCED ANALYSIS FAILED - ERROR LOGGED TO: {log_file_path}
# """

#         log_to_file(error_details, "CRITICAL ENHANCED ERROR")

#         error_msg = f"Error in enhanced GPT-4 dataframe analysis: {e}"
#         logger.error(error_msg)
#         print(f"❌ Enhanced analysis failed but logs saved to: {log_file_path}")

#         return {
#             "change_summary": f"Enhanced dataframe analysis failed: {user_description}",
#             "change_type": "data_edit",
#             "session_description": f"User made changes to entire dataframe. Description: {user_description}. Error in enhanced analysis: {str(e)}",
#             "error": str(e),
#             "complete_comparison_performed": False,
#             "enhancement_method": "gpt4_feedback_loop",
#             "log_file_path": log_file_path,  # ALWAYS include log file path, even on error
#             "error_logged": True
#         }


# def _generate_claude_prompt_with_gpt4_logged(client, original_info, modified_info, user_description,
#                                            total_changes, changed_rows, changed_columns, changed_cells,
#                                            log_to_file):
#     """
#     Enhanced version with comprehensive logging of GPT-4.1 prompt generation
#     """
#     prompt_generation_request = f"""
#     You are an expert at analyzing dataframe changes and generating precise prompts for Claude 3.7 to replicate manual edits.

#     ORIGINAL DATAFRAME:
#     {original_info['full_data']}

#     MODIFIED DATAFRAME:
#     {modified_info['full_data']}

#     CHANGE ANALYSIS:
#     - Total changes: {total_changes}
#     - Changed rows: {list(changed_rows)[:20] if changed_rows else []}
#     - Changed columns: {list(changed_columns)}
#     - Sample changes: {changed_cells[:10]}
#     - User description: "{user_description}"

#     Generate a PRECISE prompt for Claude 3.7 that would replicate these exact changes.
#     Focus on:
#     1. Specific column names and filtering criteria
#     2. Exact transformation logic
#     3. Clear, executable instructions
#     4. Business context for rent roll data

#     Return ONLY the Claude prompt text, nothing else.
#     """

#     # Log the complete GPT-4.1 prompt
#     log_to_file(f"""GPT-4.1 PROMPT TO GENERATE CLAUDE INSTRUCTIONS:
# Model: gpt-4.1
# Temperature: 0.1
# Max Tokens: 3000

# COMPLETE PROMPT SENT TO GPT-4.1:
# {'-'*60}
# {prompt_generation_request}
# {'-'*60}
# """, "GPT-4.1 REQUEST")

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are an expert at generating precise data transformation prompts. Return only the Claude prompt text."},
#                 {"role": "user", "content": prompt_generation_request}
#             ],
#             max_tokens=3000,
#             temperature=0.3
#         )

#         gpt_response = response.choices[0].message.content.strip()

#         # Log GPT-4.1 response
#         log_to_file(f"""GPT-4.1 RESPONSE (CLAUDE PROMPT):
# Response Length: {len(gpt_response)} characters
# Tokens Used: Approximately {len(gpt_response.split())} words

# GENERATED CLAUDE PROMPT:
# {'-'*60}
# {gpt_response}
# {'-'*60}
# """, "GPT-4.1 RESPONSE")

#         return gpt_response

#     except Exception as e:
#         error_msg = f"GPT-4.1 API Error: {str(e)}"
#         log_to_file(f"""GPT-4.1 API CALL FAILED:
# Error: {error_msg}
# Fallback: Using basic prompt
# """)
#         return f"Replicate the manual edits described as: {user_description}"


# def _test_claude_prompt_replication_logged(original_df, target_df, claude_prompt, max_attempts=3, log_to_file=None):
#     """
#     ENHANCED version with GPT-4.1 intelligent feedback loop for Claude replication testing.

#     Architecture:
#     GPT-4.1 (Prompt Generator) → Claude 3.7 (Code Executor) → GPT-4.1 (Gap Analyzer) → GPT-4.1 (Prompt Improver)
#     """
#     attempts = []
#     current_prompt = claude_prompt

#     log_to_file(f"""ENHANCED AI FEEDBACK LOOP REPLICATION TESTING:
# Architecture: GPT-4.1 → Claude 3.7 → GPT-4.1 (Analyzer) → GPT-4.1 (Improver)
# Maximum Attempts: {max_attempts}
# Target DataFrame Shape: {target_df.shape}
# Original DataFrame Shape: {original_df.shape}

# INITIAL CLAUDE PROMPT:
# {'-'*60}
# {claude_prompt}
# {'-'*60}
# """, "ENHANCED REPLICATION TESTING")

#     for attempt in range(max_attempts):
#         print(f"🔄 Enhanced Attempt {attempt + 1}/{max_attempts}")
#         log_to_file(f"Starting enhanced attempt {attempt + 1}/{max_attempts}...", f"ENHANCED ATTEMPT {attempt + 1}")

#         try:
#             # STEP 1: Claude executes the current prompt
#             claude_response = _call_actual_claude_logged(current_prompt, original_df, log_to_file, attempt + 1)
#             code_blocks = _extract_code_blocks(claude_response)

#             if not code_blocks:
#                 attempts.append({
#                     "attempt": attempt + 1,
#                     "success": False,
#                     "error": "No code found in Claude response",
#                     "prompt_used": current_prompt,
#                     "enhancement_type": "no_code_error"
#                 })
#                 log_to_file("No code blocks found in Claude response")
#                 continue

#             # Execute Claude's code
#             test_df = original_df.copy()
#             exec_globals = {"df": test_df, "pd": pd, "np": np, "os": os, "datetime": datetime}

#             execution_output = ""
#             for i, code in enumerate(code_blocks):
#                 try:
#                     exec(code, exec_globals)
#                     execution_output += f"Code block {i+1} executed successfully\n"
#                 except Exception as exec_error:
#                     execution_output += f"Code block {i+1} failed: {str(exec_error)}\n"

#             result_df = exec_globals["df"]
#             match_score = _calculate_match_score(target_df, result_df)

#             log_to_file(f"""CLAUDE EXECUTION RESULTS - ATTEMPT {attempt + 1}:
# Match Score: {match_score:.2%}
# Result Shape: {result_df.shape}
# Target Shape: {target_df.shape}
# Execution Output: {execution_output}
# """)

#             # STEP 2: Check if successful or needs GPT-4.1 enhancement
#             if match_score >= 0.95:
#                 attempts.append({
#                     "attempt": attempt + 1,
#                     "success": True,
#                     "match_score": match_score,
#                     "generated_code": code_blocks,
#                     "prompt_used": current_prompt,
#                     "result_shape": result_df.shape,
#                     "target_shape": target_df.shape,
#                     "execution_output": execution_output,
#                     "enhancement_type": "success"
#                 })
#                 log_to_file("🎉 REPLICATION SUCCESSFUL! Match score >= 95%")
#                 break
#             else:
#                 # STEP 3: GPT-4.1 analyzes why replication failed
#                 gap_analysis = _analyze_replication_gaps_with_gpt4(
#                     target_df, result_df, current_prompt, match_score, log_to_file, attempt + 1
#                 )

#                 # STEP 4: GPT-4.1 improves the prompt based on gap analysis
#                 if attempt < max_attempts - 1:  # Don't improve on last attempt
#                     improved_prompt = _improve_claude_prompt_with_gpt4_analysis(
#                         current_prompt, gap_analysis, attempt + 1, log_to_file
#                     )
#                     current_prompt = improved_prompt

#                 attempts.append({
#                     "attempt": attempt + 1,
#                     "success": False,
#                     "match_score": match_score,
#                     "generated_code": code_blocks,
#                     "prompt_used": current_prompt,
#                     "result_shape": result_df.shape,
#                     "target_shape": target_df.shape,
#                     "execution_output": execution_output,
#                     "gap_analysis": gap_analysis,
#                     "enhancement_type": "gpt4_enhanced"
#                 })

#         except Exception as e:
#             attempts.append({
#                 "attempt": attempt + 1,
#                 "success": False,
#                 "error": str(e),
#                 "prompt_used": current_prompt,
#                 "enhancement_type": "execution_error"
#             })
#             log_to_file(f"ATTEMPT {attempt + 1} FAILED with error: {str(e)}")

#     # Final results
#     final_success = any(attempt["success"] for attempt in attempts)
#     best_attempt = max(attempts, key=lambda x: x.get("match_score", 0)) if attempts else None

#     log_to_file(f"""ENHANCED REPLICATION TESTING COMPLETE:
# Final Success: {final_success}
# Best Match Score: {best_attempt.get('match_score', 0):.2% if best_attempt else 'N/A'}
# Enhancement Method: GPT-4.1 Intelligent Feedback Loop
# Progressive Improvement: {len([a for a in attempts if a.get('enhancement_type') == 'gpt4_enhanced'])} enhanced attempts
# """, "ENHANCED REPLICATION RESULTS")

#     return {
#         "attempts": attempts,
#         "final_success": final_success,
#         "best_attempt": best_attempt,
#         "final_prompt": current_prompt,
#         "enhancement_method": "gpt4_feedback_loop"
#     }


# def _analyze_replication_gaps_with_gpt4(target_df, result_df, original_prompt, match_score, log_to_file, attempt_number):
#     """
#     GPT-4.1 analyzes exactly why the replication failed and identifies specific gaps.
#     """
#     log_to_file(f"""STARTING GPT-4.1 GAP ANALYSIS - ATTEMPT {attempt_number}:
# Analyzing why match score is {match_score:.2%} instead of 95%+
# """, f"GPT-4.1 GAP ANALYSIS {attempt_number}")

#     try:
#         # Calculate specific differences
#         differences = _identify_specific_differences(target_df, result_df)

#         gap_analysis_prompt = f"""You are a data analysis expert. Analyze why this dataframe replication failed.

# TARGET DATAFRAME (what user wanted):
# Shape: {target_df.shape}
# Columns: {list(target_df.columns)}
# First 10 rows:
# {target_df.head(10).to_string()}

# Full target data:
# {target_df.to_string() if len(target_df) <= 50 else target_df.to_string()[:3000] + "... [truncated]"}

# ACTUAL RESULT (what Claude produced):
# Shape: {result_df.shape}
# Columns: {list(result_df.columns)}
# First 10 rows:
# {result_df.head(10).to_string()}

# Full result data:
# {result_df.to_string() if len(result_df) <= 50 else result_df.to_string()[:3000] + "... [truncated]"}

# CLAUDE PROMPT THAT FAILED:
# {original_prompt}

# MATCH SCORE: {match_score:.2%} (Target: 95%+)

# SPECIFIC DIFFERENCES DETECTED:
# {json.dumps(differences, indent=2)}

# ANALYSIS REQUIRED:
# Identify the TOP 3 root causes for this replication failure. For each cause:
# 1. What specific operation failed?
# 2. Why did it fail?
# 3. What should have happened instead?
# 4. What prompt instruction was unclear or missing?

# Focus on actionable insights for improving the Claude prompt.

# Return analysis in this JSON format:
# {{
#     "top_3_issues": [
#         {{
#             "issue": "Brief description",
#             "details": "Detailed explanation",
#             "prompt_problem": "What was wrong with the prompt",
#             "fix_needed": "Specific fix required"
#         }}
#     ],
#     "overall_diagnosis": "Summary of main problem",
#     "confidence": "high|medium|low"
# }}
# """

#         log_to_file(f"""GAP ANALYSIS PROMPT TO GPT-4.1:
# {'-'*60}
# {gap_analysis_prompt}
# {'-'*60}
# """)

#         # Call GPT-4.1 for gap analysis
#         client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are an expert at analyzing dataframe replication failures. Return only valid JSON."},
#                 {"role": "user", "content": gap_analysis_prompt}
#             ],
#             max_tokens=2000,
#             temperature=0.1
#         )

#         gap_analysis_response = response.choices[0].message.content

#         log_to_file(f"""GPT-4.1 GAP ANALYSIS RESPONSE:
# {'-'*60}
# {gap_analysis_response}
# {'-'*60}
# """)

#         # Parse JSON response
#         try:
#             import re
#             json_match = re.search(r'{.*}', gap_analysis_response, re.DOTALL)
#             if json_match:
#                 gap_analysis = json.loads(json_match.group(0))
#                 log_to_file("✅ Successfully parsed gap analysis JSON")
#                 return gap_analysis
#             else:
#                 log_to_file("❌ No JSON found in gap analysis response")
#         except Exception as json_error:
#             log_to_file(f"❌ JSON parsing failed: {str(json_error)}")

#     except Exception as e:
#         log_to_file(f"❌ Gap analysis failed: {str(e)}")

#     # Fallback gap analysis
#     fallback_analysis = {
#         "top_3_issues": [
#             {
#                 "issue": f"Match score only {match_score:.2%}",
#                 "details": "Automated analysis failed, using fallback",
#                 "prompt_problem": "Unknown specific issue",
#                 "fix_needed": "Manual prompt review required"
#             }
#         ],
#         "overall_diagnosis": "Gap analysis failed, using basic feedback",
#         "confidence": "low"
#     }

#     log_to_file(f"Using fallback gap analysis: {json.dumps(fallback_analysis, indent=2)}")
#     return fallback_analysis


# def _improve_claude_prompt_with_gpt4_analysis(current_prompt, gap_analysis, attempt_number, log_to_file):
#     """
#     GPT-4.1 improves the Claude prompt based on intelligent gap analysis.
#     """
#     log_to_file(f"""STARTING GPT-4.1 PROMPT IMPROVEMENT - ATTEMPT {attempt_number}:
# Using intelligent gap analysis to surgically improve the prompt
# """, f"GPT-4.1 PROMPT IMPROVEMENT {attempt_number}")

#     try:
#         improvement_prompt = f"""You are a prompt engineering expert. Improve this Claude prompt based on specific failure analysis.

# CURRENT CLAUDE PROMPT (that failed):
# {current_prompt}

# GAP ANALYSIS FROM PREVIOUS ATTEMPT:
# {json.dumps(gap_analysis, indent=2)}

# ATTEMPT NUMBER: {attempt_number}/3

# TASK: Generate an improved Claude prompt that specifically addresses the identified issues.

# REQUIREMENTS:
# 1. Fix the TOP 3 issues identified in the gap analysis
# 2. Make surgical improvements (don't rewrite everything)
# 3. Add specific instructions for the problem areas
# 4. Maintain the original intent while fixing the failures
# 5. Be more explicit about edge cases and data handling

# Focus on the most impactful improvements that will raise the match score above 95%.

# Return ONLY the improved Claude prompt text, nothing else.
# """

#         log_to_file(f"""PROMPT IMPROVEMENT REQUEST TO GPT-4.1:
# {'-'*60}
# {improvement_prompt}
# {'-'*60}
# """)

#         # Call GPT-4.1 for prompt improvement
#         client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are an expert prompt engineer. Return only the improved Claude prompt."},
#                 {"role": "user", "content": improvement_prompt}
#             ],
#             max_tokens=2000,
#             temperature=0.3
#         )

#         improved_prompt = response.choices[0].message.content.strip()

#         log_to_file(f"""GPT-4.1 IMPROVED PROMPT:
# {'-'*60}
# {improved_prompt}
# {'-'*60}
# """)

#         return improved_prompt

#     except Exception as e:
#         log_to_file(f"❌ Prompt improvement failed: {str(e)}")

#         # Fallback improvement (basic)
#         fallback_improvement = f"""{current_prompt}

# ENHANCED INSTRUCTIONS BASED ON PREVIOUS ATTEMPT:
# - Double-check all filtering conditions
# - Ensure exact data type matching
# - Handle edge cases more carefully
# - Verify all column operations

# Previous attempt had {gap_analysis.get('overall_diagnosis', 'unknown issues')}."""

#         log_to_file(f"Using fallback prompt improvement")
#         return fallback_improvement


# def _identify_specific_differences(target_df, result_df):
#     """
#     Identify specific differences between target and result dataframes.
#     """
#     differences = {
#         "shape_differences": {
#             "target_shape": target_df.shape,
#             "result_shape": result_df.shape,
#             "rows_diff": result_df.shape[0] - target_df.shape[0],
#             "cols_diff": result_df.shape[1] - target_df.shape[1]
#         },
#         "column_differences": {
#             "target_columns": list(target_df.columns),
#             "result_columns": list(result_df.columns),
#             "missing_columns": list(set(target_df.columns) - set(result_df.columns)),
#             "extra_columns": list(set(result_df.columns) - set(target_df.columns))
#         },
#         "data_type_differences": {},
#         "sample_data_differences": [],
#         "statistical_differences": {}
#     }

#     # Data type comparison
#     if list(target_df.columns) == list(result_df.columns):
#         for col in target_df.columns:
#             if str(target_df[col].dtype) != str(result_df[col].dtype):
#                 differences["data_type_differences"][col] = {
#                     "target_dtype": str(target_df[col].dtype),
#                     "result_dtype": str(result_df[col].dtype)
#                 }

#     # Sample data differences (first few mismatches)
#     if target_df.shape == result_df.shape and list(target_df.columns) == list(result_df.columns):
#         mismatch_count = 0
#         for i in range(min(len(target_df), 10)):  # Check first 10 rows
#             for col in target_df.columns:
#                 if mismatch_count >= 5:  # Limit to 5 sample differences
#                     break
#                 try:
#                     target_val = target_df.iloc[i][col]
#                     result_val = result_df.iloc[i][col]

#                     if pd.isna(target_val) != pd.isna(result_val) or (not pd.isna(target_val) and str(target_val).strip() != str(result_val).strip()):
#                         differences["sample_data_differences"].append({
#                             "row": i,
#                             "column": col,
#                             "target_value": str(target_val),
#                             "result_value": str(result_val)
#                         })
#                         mismatch_count += 1
#                 except:
#                     continue

#     # Basic statistical comparison for numeric columns
#     try:
#         for col in target_df.select_dtypes(include=[np.number]).columns:
#             if col in result_df.columns:
#                 differences["statistical_differences"][col] = {
#                     "target_mean": float(target_df[col].mean()) if not target_df[col].empty else None,
#                     "result_mean": float(result_df[col].mean()) if not result_df[col].empty else None,
#                     "target_count": int(target_df[col].count()),
#                     "result_count": int(result_df[col].count())
#                 }
#     except:
#         pass

#     return differences


# def _call_actual_claude_logged(prompt, original_df, log_to_file, attempt_number):
#     """
#     Enhanced version with comprehensive logging of Claude API calls
#     """
#     log_to_file(f"""CALLING CLAUDE API - ATTEMPT {attempt_number}:
# Model: claude-sonnet-4-20250514
# Temperature: 0.2
# Max Tokens: 2000

# PROMPT BEING SENT TO CLAUDE:
# {'-'*60}
# {prompt}
# {'-'*60}

# DATAFRAME CONTEXT BEING SENT:
# Shape: {original_df.shape}
# Columns: {list(original_df.columns)}
# First 10 rows:
# {original_df.head(10).to_string()}
# """, f"CLAUDE API CALL {attempt_number}")

#     try:
#         # Get Anthropic client
#         anthropic_client = Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

#         # Prepare dataframe context for Claude
#         df_summary = f"""
#         DATAFRAME CONTENT:
#         {original_df.to_string(max_rows=50)}

#         DATAFRAME STATISTICS:
#         - Shape: {original_df.shape}
#         - Columns: {list(original_df.columns)}
#         - Data types: {dict(original_df.dtypes)}
#         - Null values per column: {dict(original_df.isnull().sum())}
#         """

#         # System prompt for Claude
#         claude_system_prompt = """You are an expert Python data analyst.
#         You will receive a dataframe that is already loaded as 'df' and a specific task to perform.
#         Generate Python pandas code to accomplish the task.

#         IMPORTANT RULES:
#         1. The dataframe 'df' is already loaded - do not load or import it
#         2. Wrap all code in ```python and ``` blocks
#         3. Do not use try-except blocks - let errors propagate naturally
#         4. Be precise and specific in your transformations
#         5. Provide working, executable code
#         6. Focus on the exact transformation requested"""

#         # Prepare messages for Claude
#         claude_messages = [
#             {
#                 "role": "user",
#                 "content": f"Here is the dataframe that's already loaded as 'df':\n\n{df_summary}\n\nTASK: {prompt}\n\nGenerate Python code to accomplish this task."
#             }
#         ]

#         # Call Claude
#         claude_response = anthropic_client.messages.create(
#             # model="claude-sonnet-4-20250514",  # Use the latest Claude model
#             model="claude-3-7-sonnet-20250219",  # Use the latest Claude model
#             system=claude_system_prompt,
#             messages=claude_messages,
#             max_tokens=2000,
#             temperature=0.2
#         )

#         # Extract response text
#         response_text = claude_response.content[0].text

#         log_to_file(f"""CLAUDE API RESPONSE - ATTEMPT {attempt_number}:
# Response Length: {len(response_text)} characters
# Response Received Successfully: ✅

# FULL CLAUDE RESPONSE:
# {'-'*60}
# {response_text}
# {'-'*60}
# """)

#         return response_text

#     except Exception as e:
#         error_msg = f"Claude API Error: {str(e)}"
#         log_to_file(f"""CLAUDE API CALL FAILED - ATTEMPT {attempt_number}:
# Error: {error_msg}
# Using fallback response
# """)

#         # Fallback response if Claude API fails
#         fallback_response = f"""
#         I'll help you with this task. Here's the code:

#         ```python
#         # Error calling Claude API: {str(e)}
#         # Fallback code
#         print("Claude API error, using fallback")
#         print(f"Dataframe shape: {{df.shape}}")
#         print(df.head())
#         ```
#         """

#         log_to_file(f"Fallback response generated:\n{fallback_response}")
#         return fallback_response


# def _get_final_analysis_with_prompts_logged(client, original_info, modified_info, user_description,
#                                           total_changes, changed_rows, changed_columns, changed_cells,
#                                           claude_prompt, replication_results, log_to_file):
#     """
#     Enhanced version with comprehensive logging of final analysis generation
#     """
#     best_attempt = replication_results.get("best_attempt", {})
#     final_claude_prompt = replication_results.get("final_prompt", claude_prompt)
#     success_status = "SUCCESS" if replication_results["final_success"] else "PARTIAL"

#     # Calculate statistics
#     total_cells = original_info["shape"][0] * original_info["shape"][1] if original_info["shape"][0] > 0 else 1
#     change_density = total_changes / total_cells

#     analysis_prompt = f"""
#     Analyze this dataframe change and provide a comprehensive summary.

#     ORIGINAL: {original_info['shape']} with data:
#     {original_info['full_data'][:2000]}...

#     MODIFIED: {modified_info['shape']} with data:
#     {modified_info['full_data'][:2000]}...

#     CHANGES: {total_changes} cells changed ({change_density*100:.1f}%)
#     USER DESCRIPTION: {user_description}

#     TESTED REPLICATION: {success_status}
#     - Attempts: {len(replication_results['attempts'])}
#     - Best match: {best_attempt.get('match_score', 0):.1%}
#     - Enhancement method: {replication_results.get('enhancement_method', 'standard')}

#     Provide analysis in this exact JSON format:
#     {{
#         "change_summary": "Detailed technical summary including WORKING_CLAUDE_PROMPT: {final_claude_prompt} and GPT4_ANALYSIS_PROMPT: [the prompt used to generate the Claude prompt] - describe what changed and how replication performed with enhanced feedback loop",
#         "change_type": "data_edit|structure_change|mixed",
#         "structural_changes": {{
#             "rows_added": {max(0, modified_info['shape'][0] - original_info['shape'][0])},
#             "rows_removed": {max(0, original_info['shape'][0] - modified_info['shape'][0])},
#             "columns_added": [],
#             "columns_removed": []
#         }},
#         "data_modifications": {{
#             "cells_changed": {total_changes},
#             "total_cells": {total_cells},
#             "change_percentage": {change_density*100:.2f},
#             "rows_affected": {len(changed_rows)},
#             "columns_affected": {list(changed_columns)},
#             "common_patterns": ["Patterns identified"],
#             "data_quality_impact": "improved|degraded|neutral"
#         }},
#         "business_impact": {{
#             "rent_calculations_affected": "Analysis of rent impact",
#             "tenant_information_updated": "Analysis of tenant data changes",
#             "occupancy_status_changed": "Analysis of occupancy changes"
#         }},
#         "recommendations": ["Specific recommendations"],
#         "session_description": "ENHANCEMENT: GPT-4.1_FEEDBACK_LOOP | REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | REPLICATION_ATTEMPTS: {len(replication_results['attempts'])} | USER_EDIT: {user_description} | Changes: {total_changes} cells ({change_density*100:.1f}%) | Best match: {best_attempt.get('match_score', 0):.1%}"
#     }}
#     """

#     log_to_file(f"""FINAL ANALYSIS GENERATION:
# Sending final analysis request to GPT-4.1...

# PROMPT FOR FINAL ANALYSIS:
# {'-'*60}
# {analysis_prompt}
# {'-'*60}
# """, "FINAL ANALYSIS GENERATION")

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are a data analyst. Return only valid JSON."},
#                 {"role": "user", "content": analysis_prompt}
#             ],
#             max_tokens=3000,
#             temperature=0.2
#         )

#         gpt_final_response = response.choices[0].message.content
#         log_to_file(f"""FINAL ANALYSIS GPT-4.1 RESPONSE:
# {'-'*60}
# {gpt_final_response}
# {'-'*60}
# """)

#         try:
#             import json
#             import re
#             json_match = re.search(r'{.*}', gpt_final_response, re.DOTALL)
#             if json_match:
#                 parsed_analysis = json.loads(json_match.group(0))
#                 log_to_file("✅ Successfully parsed JSON from final analysis response")
#                 return parsed_analysis
#             else:
#                 log_to_file("❌ No JSON found in final analysis response")
#         except Exception as json_error:
#             log_to_file(f"❌ JSON parsing failed: {str(json_error)}")

#     except Exception as api_error:
#         log_to_file(f"❌ Final analysis API call failed: {str(api_error)}")

#     # Fallback
#     fallback_analysis = {
#         "change_summary": f"ENHANCEMENT: GPT-4.1_FEEDBACK_LOOP | WORKING_CLAUDE_PROMPT: {final_claude_prompt} | GPT4_ANALYSIS_PROMPT: [embedded] | {user_description} - {total_changes} changes with {replication_results['final_success']} replication",
#         "change_type": "data_edit",
#         "session_description": f"ENHANCEMENT: GPT-4.1_FEEDBACK_LOOP | REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | USER_EDIT: {user_description} | Changes: {total_changes} cells",
#         "data_modifications": {
#             "cells_changed": total_changes,
#             "total_cells": total_cells,
#             "change_percentage": change_density*100
#         }
#     }

#     log_to_file(f"Using fallback analysis:\n{json.dumps(fallback_analysis, indent=2)}")

#     return fallback_analysis


# # Additional helper functions for the enhanced logging system

# def _extract_code_blocks(response_text):
#     """
#     Extract Python code blocks from response text
#     """
#     import re
#     code_pattern = r'```(?:python)?\s*(.*?)```'
#     code_blocks = re.findall(code_pattern, response_text, re.DOTALL)
#     return [block.strip() for block in code_blocks if block.strip()]


# def _calculate_match_score(target_df, result_df):
#     """
#     Calculate how closely the result matches the target with improved scoring.
#     Now gives partial credit for near-matches instead of strict 0/1 scoring.
#     """
#     import pandas as pd

#     # Handle edge cases
#     if target_df.empty and result_df.empty:
#         return 1.0
#     if target_df.empty or result_df.empty:
#         return 0.0

#     # 1. Shape similarity score (30% weight)
#     target_rows, target_cols = target_df.shape
#     result_rows, result_cols = result_df.shape

#     # Row similarity (allows for minor filtering differences)
#     if target_rows == 0:
#         row_score = 1.0 if result_rows == 0 else 0.0
#     else:
#         row_diff = abs(target_rows - result_rows)
#         row_score = max(0, 1.0 - (row_diff / target_rows))

#     # Column similarity (should be exact for most use cases)
#     col_score = 1.0 if target_cols == result_cols else 0.0

#     shape_score = 0.7 * row_score + 0.3 * col_score

#     # 2. Column structure score (20% weight)
#     target_columns = list(target_df.columns)
#     result_columns = list(result_df.columns)

#     if target_columns == result_columns:
#         column_score = 1.0
#     else:
#         # Partial credit for column overlap
#         common_cols = set(target_columns) & set(result_columns)
#         total_unique_cols = set(target_columns) | set(result_columns)
#         column_score = len(common_cols) / len(total_unique_cols) if total_unique_cols else 0.0

#     # 3. Content similarity score (50% weight)
#     content_score = 0.0

#     if target_columns == result_columns and len(target_columns) > 0:
#         # Compare overlapping rows when columns match
#         min_rows = min(len(target_df), len(result_df))

#         if min_rows > 0:
#             total_cells = 0
#             matching_cells = 0

#             # Compare cell by cell for overlapping area
#             for i in range(min_rows):
#                 for col in target_columns:
#                     total_cells += 1

#                     try:
#                         target_val = target_df.iloc[i][col]
#                         result_val = result_df.iloc[i][col]

#                         # Handle NaN comparisons
#                         if pd.isna(target_val) and pd.isna(result_val):
#                             matching_cells += 1
#                         elif pd.isna(target_val) or pd.isna(result_val):
#                             # NaN vs non-NaN = no match
#                             continue
#                         else:
#                             # String comparison with whitespace handling
#                             target_str = str(target_val).strip()
#                             result_str = str(result_val).strip()

#                             if target_str == result_str:
#                                 matching_cells += 1
#                             else:
#                                 # Try numeric comparison for potential formatting differences
#                                 try:
#                                     target_num = float(target_str.replace(',', '').replace(', ', ''))
#                                     result_num = float(result_str.replace(',', '').replace(', ', ''))
#                                     if abs(target_num - result_num) < 0.01:  # Allow small floating point differences
#                                         matching_cells += 1
#                                 except (ValueError, TypeError):
#                                     # Not numeric, keep as non-match
#                                     continue
#                     except (IndexError, KeyError):
#                         # Skip invalid cell references
#                         continue

#             content_score = matching_cells / total_cells if total_cells > 0 else 0.0

#             # Bonus for exact row count match when content is high
#             if len(target_df) == len(result_df) and content_score > 0.9:
#                 content_score = min(1.0, content_score * 1.05)

#     else:
#         # Different column structures - can only do basic comparison
#         if min(len(target_df), len(result_df)) > 0:
#             # Give small credit for having some data with similar row count
#             row_similarity = 1.0 - abs(len(target_df) - len(result_df)) / max(len(target_df), len(result_df))
#             content_score = 0.2 * row_similarity  # Low score for structure mismatch

#     # 4. Calculate weighted final score
#     final_score = (0.30 * shape_score +
#                   0.20 * column_score +
#                   0.50 * content_score)

#     # Ensure score is between 0 and 1
#     return max(0.0, min(1.0, final_score))


# def _improve_claude_prompt(current_prompt, target_df, result_df, match_score):
#     """
#     Improve the Claude prompt based on the mismatch (FALLBACK - now replaced by GPT-4.1 enhancement)
#     """
#     feedback = f"\nPREVIOUS ATTEMPT FEEDBACK:\n"
#     feedback += f"Match score: {match_score:.2%}\n"
#     feedback += f"Target shape: {target_df.shape}, Result shape: {result_df.shape}\n"

#     if target_df.shape != result_df.shape:
#         feedback += "Shape mismatch detected. Please check row filtering logic.\n"

#     return current_prompt + feedback


# # Enhanced save function that also uses the detailed logging
# def save_edited_dataframe_enhanced_with_logging(edited_df, description):
#     """
#     Enhanced version that uses the new comprehensive logging system
#     for analyzing manual dataframe changes with GPT-4.1 feedback loop.
#     """
#     global app_state, session_recorder

#     if edited_df is None or edited_df.empty:
#         return "No data to save", gr.update()

#     try:
#         # Convert the edited dataframe to proper pandas DataFrame if needed
#         if not isinstance(edited_df, pd.DataFrame):
#             edited_df = pd.DataFrame(edited_df)

#         # Get the original dataframe for comparison
#         original_df = app_state["df"].copy()

#         logger.info("Analyzing dataframe changes with enhanced GPT-4.1 feedback loop...")
#         print("🧠 Analyzing changes with enhanced GPT-4.1 intelligent feedback loop system...")

#         # Use the enhanced analysis function with comprehensive logging and feedback loop
#         change_analysis = analyze_dataframe_changes_with_gpt4(
#             original_df=original_df,
#             modified_df=edited_df,
#             user_description=description
#         )

#         # Generate a meaningful description if not provided
#         if not description:
#             description = change_analysis.get("change_summary", "Manual edits via data editor")

#         # Save as new version
#         version_name = save_dataframe_version(edited_df, description)

#         # Update the app state with the edited dataframe
#         app_state["df"] = edited_df

#         # Record this in the copiloting session if active
#         if session_recorder.current_session_file:
#             session_description = change_analysis.get("session_description", f"Enhanced manual data edits: {description}")

#             # Create detailed session entry with log file reference
#             session_entry = f"""
# MANUAL DATA EDIT SESSION WITH ENHANCED GPT-4.1 FEEDBACK LOOP
# ============================================================
# Timestamp: {datetime.now().strftime('%H:%M:%S')}
# Edit Description: {description}
# Version Saved: {version_name}
# Enhancement Method: GPT-4.1 Intelligent Feedback Loop
# Log File: {change_analysis.get('log_file_path', 'N/A')}

# ENHANCED GPT-4.1 CHANGE ANALYSIS:
# {'-' * 40}
# Change Summary: {change_analysis.get('change_summary', 'N/A')}
# Change Type: {change_analysis.get('change_type', 'N/A')}
# Enhancement Method: {change_analysis.get('enhancement_method', 'gpt4_feedback_loop')}
# Replication Success: {change_analysis.get('full_change_statistics', {}).get('replication_success', 'Unknown')}
# Replication Attempts: {change_analysis.get('full_change_statistics', {}).get('replication_attempts', 'Unknown')}
# Progressive Improvement: {change_analysis.get('full_change_statistics', {}).get('progressive_improvement', 'Unknown')}

# Structural Changes:
# {json.dumps(change_analysis.get('structural_changes', {}), indent=2)}

# Data Modifications:
# {json.dumps(change_analysis.get('data_modifications', {}), indent=2)}

# Business Impact:
# {json.dumps(change_analysis.get('business_impact', {}), indent=2)}

# Recommendations:
# {chr(10).join([f"• {rec}" for rec in change_analysis.get('recommendations', [])])}

# Enhanced Technical Statistics:
# - Total Cells: {change_analysis.get('full_change_statistics', {}).get('total_cells', 'Unknown')}
# - Changed Cells: {change_analysis.get('full_change_statistics', {}).get('total_changes', 'Unknown')}
# - Change Density: {change_analysis.get('full_change_statistics', {}).get('change_density', 0)*100:.2f}%
# - Affected Rows: {len(change_analysis.get('full_change_statistics', {}).get('affected_rows', []))}
# - Affected Columns: {len(change_analysis.get('full_change_statistics', {}).get('affected_columns', []))}
# - GPT-4.1 Enhanced Attempts: {len([a for a in change_analysis.get('full_change_statistics', {}).get('replication_attempts', []) if isinstance(a, dict) and a.get('enhancement_type') == 'gpt4_enhanced'])}

# Original DataFrame Shape: {original_df.shape}
# Modified DataFrame Shape: {edited_df.shape}
# {'-' * 80}
# """

#             # Append to session file
#             with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
#                 f.write(session_entry + "\n")

#             # Record in session data structure
#             session_recorder.record_conversation_turn(
#                 user_message=f"MANUAL EDIT (Enhanced GPT-4.1 Feedback): {description}",
#                 ai_response=session_description,
#                 action_type="manual_data_edit_enhanced_gpt4",
#                 code_executed=None,
#                 version_saved=version_name
#             )

#             # Record the dataframe version change
#             session_recorder.record_dataframe_version(
#                 version_name=version_name,
#                 description=description,
#                 shape=list(edited_df.shape),
#                 columns=list(edited_df.columns)
#             )

#             # Record any issues found by GPT-4
#             replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
#             if not replication_success:
#                 session_recorder.record_issue_found(
#                     f"Enhanced manual edit replication failed: {description}. Check detailed log for GPT-4.1 feedback analysis.",
#                     severity="medium"
#                 )

#             logger.info("Enhanced manual edit analysis with GPT-4.1 feedback loop recorded in copiloting session")

#         # Log the changes
#         logger.info(f"Saved edited dataframe as version {version_name}")

#         # Create detailed success message with log file information
#         log_file_path = change_analysis.get('log_file_path', 'N/A')
#         replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
#         replication_attempts = change_analysis.get('full_change_statistics', {}).get('replication_attempts', 0)
#         progressive_improvement = change_analysis.get('full_change_statistics', {}).get('progressive_improvement', False)

#         success_message = f"""✅ Successfully saved as version {version_name}

# 🧠 Enhanced GPT-4.1 Feedback Loop Analysis Summary:
# {change_analysis.get('change_summary', 'Changes analyzed')[:200]}...

# 📊 Enhanced Change Details:
# • Change Type: {change_analysis.get('change_type', 'Unknown')}
# • Enhancement Method: {change_analysis.get('enhancement_method', 'gpt4_feedback_loop')}
# • Original Shape: {original_df.shape}
# • New Shape: {edited_df.shape}
# • Replication Success: {'✅ Yes' if replication_success else '❌ Partial/Failed'}
# • Replication Attempts: {replication_attempts}
# • Progressive Improvement: {'✅ Yes' if progressive_improvement else '❌ No'}

# 📝 Session Recording: {'✅ Recorded' if session_recorder.current_session_file else '❌ No active session'}

# 📁 Enhanced Analysis Log:
# {log_file_path}

# This enhanced log contains:
# • Complete GPT-4.1 prompts and responses
# • All Claude API calls and generated code
# • GPT-4.1 gap analysis for failed attempts
# • Intelligent prompt improvements between attempts
# • Progressive learning from failures
# • Step-by-step replication attempts
# • Cell-by-cell change analysis
# • Business impact assessment
# """

#         # Add recommendations if available
#         if change_analysis.get('recommendations'):
#             success_message += f"\n💡 Enhanced Recommendations:\n"
#             for rec in change_analysis['recommendations'][:3]:  # Show first 3
#                 success_message += f"• {rec}\n"

#         # Add note about enhanced capabilities
#         success_message += f"\n🚀 Enhancement Features Used:"
#         success_message += f"\n• GPT-4.1 Gap Analysis: {'✅' if progressive_improvement else '❌'}"
#         success_message += f"\n• Intelligent Prompt Improvement: {'✅' if progressive_improvement else '❌'}"
#         success_message += f"\n• Progressive Learning: {'✅' if progressive_improvement else '❌'}"

#         # Add note about log file location
#         success_message += f"\n\n📄 For complete debugging and enhancement information, check: {log_file_path}"

#         return success_message, gr.update(value=edited_df)

#     except Exception as e:
#         error_msg = f"❌ Error saving with enhanced analysis: {str(e)}"
#         logger.error(f"Error saving edited dataframe with enhanced analysis: {e}")
#         logger.error(traceback.format_exc())

#         # Still try to record the error in session
#         if session_recorder.current_session_file:
#             session_recorder.record_conversation_turn(
#                 user_message=f"MANUAL EDIT FAILED (Enhanced GPT-4.1): {description}",
#                 ai_response=error_msg,
#                 action_type="manual_edit_error_enhanced_gpt4",
#                 code_executed=None,
#                 version_saved=None
#             )

#         return error_msg, gr.update()


# # Function to view and summarize all log files
# def get_manual_edit_logs_summary():
#     """
#     Generate a summary of all manual edit analysis log files including enhanced versions
#     """
#     logs_dir = "manual_edit_analysis_logs"

#     if not os.path.exists(logs_dir):
#         return "No manual edit analysis logs found yet."

#     try:
#         log_files = [f for f in os.listdir(logs_dir) if f.endswith('_detailed_analysis.txt')]

#         if not log_files:
#             return "No detailed analysis log files found."

#         # Sort by creation time (newest first)
#         log_files.sort(key=lambda x: os.path.getctime(os.path.join(logs_dir, x)), reverse=True)

#         # Count enhanced vs standard logs
#         enhanced_logs = [f for f in log_files if f.startswith('enhanced_edit_')]
#         standard_logs = [f for f in log_files if f.startswith('manual_edit_')]

#         summary = f"""📁 Enhanced Manual Edit Analysis Logs Summary
# Found {len(log_files)} detailed analysis log files:
# • Enhanced (GPT-4.1 Feedback Loop): {len(enhanced_logs)}
# • Standard: {len(standard_logs)}

# Recent Logs:
# """

#         for i, log_file in enumerate(log_files[:10], 1):  # Show last 10
#             file_path = os.path.join(logs_dir, log_file)
#             file_size = os.path.getsize(file_path)
#             created_time = datetime.fromtimestamp(os.path.getctime(file_path))

#             # Try to extract session info from filename
#             session_id = log_file.replace('_detailed_analysis.txt', '')
#             log_type = "🧠 Enhanced" if log_file.startswith('enhanced_edit_') else "📝 Standard"

#             summary += f"""{i}. {log_type} - {session_id}
#    📄 File: {log_file}
#    📅 Created: {created_time.strftime('%Y-%m-%d %H:%M:%S')}
#    💾 Size: {file_size:,} bytes
#    📁 Path: {file_path}

# """

#         if len(log_files) > 10:
#             summary += f"... and {len(log_files) - 10} more log files\n"

#         summary += f"""
# 📋 Enhanced Log File Contents Include:
# • Complete GPT-4.1 prompts and responses
# • All Claude API calls and code generation
# • GPT-4.1 gap analysis for replication failures
# • Intelligent prompt improvement between attempts
# • Progressive learning and enhancement tracking
# • Step-by-step replication testing results
# • Cell-by-cell dataframe comparison analysis
# • Business impact and recommendation analysis
# • Detailed error messages and debugging information

# 🚀 Enhancement Features:
# • Gap Analysis: GPT-4.1 identifies specific failure reasons
# • Prompt Improvement: Surgical fixes based on analysis
# • Progressive Learning: Each attempt gets smarter
# • Success Tracking: Monitors improvement across attempts

# 📂 All logs are saved in: {logs_dir}/
# """

#         return summary

#     except Exception as e:
#         return f"Error reading log files: {str(e)}"


# # Function to read a specific log file
# def read_manual_edit_log(session_id):
#     """
#     Read and return the contents of a specific manual edit analysis log
#     """
#     logs_dir = "manual_edit_analysis_logs"
#     log_file_path = os.path.join(logs_dir, f"{session_id}_detailed_analysis.txt")

#     if not os.path.exists(log_file_path):
#         return f"Log file not found: {log_file_path}"

#     try:
#         with open(log_file_path, 'r', encoding='utf-8') as f:
#             content = f.read()

#         # Determine if this is an enhanced log
#         is_enhanced = "GPT-4.1 FEEDBACK LOOP" in content or "ENHANCED" in content

#         log_type = "🧠 Enhanced GPT-4.1 Feedback Loop" if is_enhanced else "📝 Standard"

#         return f"""📄 Manual Edit Analysis Log ({log_type}): {session_id}
# {'='*80}

# {content}

# {'='*80}
# End of log file: {log_file_path}

# Log Type: {log_type}
# Enhancement Features: {'Gap Analysis, Prompt Improvement, Progressive Learning' if is_enhanced else 'Basic replication testing'}
# """

#     except Exception as e:
#         return f"Error reading log file: {str(e)}"


# # Function to analyze enhancement performance across logs
# def analyze_enhancement_performance():
#     """
#     Analyze the performance difference between standard and enhanced logs
#     """
#     logs_dir = "manual_edit_analysis_logs"

#     if not os.path.exists(logs_dir):
#         return "No manual edit analysis logs found yet."

#     try:
#         log_files = [f for f in os.listdir(logs_dir) if f.endswith('_detailed_analysis.txt')]

#         if not log_files:
#             return "No detailed analysis log files found."

#         enhanced_stats = []
#         standard_stats = []

#         for log_file in log_files:
#             try:
#                 file_path = os.path.join(logs_dir, log_file)
#                 with open(file_path, 'r', encoding='utf-8') as f:
#                     content = f.read()

#                 is_enhanced = "GPT-4.1 FEEDBACK LOOP" in content or "ENHANCED" in content

#                 # Extract success rate
#                 if "REPLICATION SUCCESSFUL" in content:
#                     success = True
#                 elif "REPLICATION TESTING COMPLETE" in content:
#                     success = False
#                 else:
#                     continue

#                 # Extract match score
#                 import re
#                 match_scores = re.findall(r'Best Match Score: ([\d.]+)%', content)
#                 if match_scores:
#                     best_score = float(match_scores[-1])
#                 else:
#                     best_score = 0.0

#                 # Extract attempt count
#                 attempt_matches = re.findall(r'Total Attempts: (\d+)', content)
#                 if attempt_matches:
#                     attempts = int(attempt_matches[-1])
#                 else:
#                     attempts = 3

#                 stat = {
#                     'success': success,
#                     'best_score': best_score,
#                     'attempts': attempts,
#                     'file': log_file
#                 }

#                 if is_enhanced:
#                     enhanced_stats.append(stat)
#                 else:
#                     standard_stats.append(stat)

#             except Exception as e:
#                 continue

#         # Calculate performance metrics
#         def calc_metrics(stats):
#             if not stats:
#                 return {'success_rate': 0, 'avg_score': 0, 'avg_attempts': 0, 'count': 0}

#             success_rate = sum(1 for s in stats if s['success']) / len(stats)
#             avg_score = sum(s['best_score'] for s in stats) / len(stats)
#             avg_attempts = sum(s['attempts'] for s in stats) / len(stats)

#             return {
#                 'success_rate': success_rate,
#                 'avg_score': avg_score,
#                 'avg_attempts': avg_attempts,
#                 'count': len(stats)
#             }

#         enhanced_metrics = calc_metrics(enhanced_stats)
#         standard_metrics = calc_metrics(standard_stats)

#         summary = f"""🧠 Enhancement Performance Analysis
# {'='*50}

# 📊 Enhanced GPT-4.1 Feedback Loop:
# • Total Sessions: {enhanced_metrics['count']}
# • Success Rate: {enhanced_metrics['success_rate']:.1%}
# • Average Best Score: {enhanced_metrics['avg_score']:.1f}%
# • Average Attempts: {enhanced_metrics['avg_attempts']:.1f}

# 📝 Standard Approach:
# • Total Sessions: {standard_metrics['count']}
# • Success Rate: {standard_metrics['success_rate']:.1%}
# • Average Best Score: {standard_metrics['avg_score']:.1f}%
# • Average Attempts: {standard_metrics['avg_attempts']:.1f}

# 🚀 Improvement Metrics:
# """

#         if standard_metrics['count'] > 0 and enhanced_metrics['count'] > 0:
#             success_improvement = enhanced_metrics['success_rate'] - standard_metrics['success_rate']
#             score_improvement = enhanced_metrics['avg_score'] - standard_metrics['avg_score']

#             summary += f"""• Success Rate Improvement: {success_improvement:+.1%}
# • Average Score Improvement: {score_improvement:+.1f}%
# • Enhanced Success Rate: {enhanced_metrics['success_rate']:.1%} vs Standard: {standard_metrics['success_rate']:.1%}

# 💡 Analysis:
# The enhanced GPT-4.1 feedback loop shows {'significant improvement' if success_improvement > 0.1 else 'some improvement' if success_improvement > 0 else 'similar performance'}
# over the standard approach with {'higher' if score_improvement > 5 else 'similar'} match scores and
# {'better' if success_improvement > 0.05 else 'equivalent'} success rates.
# """
#         else:
#             summary += "• Insufficient data for comparison\n"

#         return summary

#     except Exception as e:
#         return f"Error analyzing enhancement performance: {str(e)}"

In [ ]:
# def analyze_dataframe_changes_with_gpt4(original_df, modified_df, user_description=""):
#     """
#     Use GPT-4.1 to analyze differences between original and modified dataframes,
#     generate a detailed description of changes made, AND test Claude prompts.
#     NOW USES COMPLETE DATAFRAMES FOR ANALYSIS AND TESTS REPLICATION.
#     AUTOMATICALLY SAVES ALL LOGS TO TEXT FILES WHENEVER EXECUTED.
#     """
#     import os
#     import json
#     import traceback
#     from datetime import datetime

#     # Create detailed logging directory
#     logs_dir = "manual_edit_analysis_logs"
#     os.makedirs(logs_dir, exist_ok=True)

#     # Generate unique log session ID
#     log_session_id = f"manual_edit_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')[:-3]}"  # Include milliseconds for uniqueness
#     log_file_path = os.path.join(logs_dir, f"{log_session_id}_detailed_analysis.txt")

#     # Enhanced logging function that ALWAYS saves to file
#     def log_to_file(content, section_title=""):
#         try:
#             with open(log_file_path, 'a', encoding='utf-8') as f:
#                 if section_title:
#                     f.write(f"\n{'='*80}\n")
#                     f.write(f"{section_title}\n")
#                     f.write(f"{'='*80}\n")
#                 f.write(f"{content}\n")
#                 f.flush()  # Ensure immediate write to disk
#         except Exception as e:
#             print(f"ERROR writing to log file: {e}")

#     # ALWAYS initialize comprehensive log - even if function fails later
#     try:
#         log_to_file(f"""COMPREHENSIVE MANUAL EDIT ANALYSIS LOG
# Session ID: {log_session_id}
# Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# User Description: "{user_description}"
# Log File Path: {log_file_path}

# ANALYSIS OVERVIEW:
# This log contains the complete workflow of analyzing manual dataframe edits:
# 1. Original vs Modified dataframe comparison
# 2. GPT-4.1 prompt generation for Claude
# 3. Claude API calls and responses
# 4. Code execution attempts and results
# 5. Replication success/failure analysis

# FUNCTION EXECUTION STATUS: STARTING...
# """, "SESSION INITIALIZATION")

#         print(f"📝 Manual edit analysis log initialized: {log_file_path}")

#     except Exception as init_error:
#         print(f"CRITICAL: Could not initialize log file: {init_error}")
#         # Continue execution even if logging fails

#     try:
#         # ALWAYS log dataframe information - even if analysis fails later
#         log_to_file(f"""ORIGINAL DATAFRAME COMPLETE ANALYSIS:
# Shape: {original_df.shape}
# Columns: {list(original_df.columns)}
# Data Types: {dict(original_df.dtypes.astype(str))}
# Memory Usage: {original_df.memory_usage(deep=True).sum()} bytes
# Null Counts: {dict(original_df.isnull().sum())}

# FIRST 10 ROWS PREVIEW:
# {original_df.head(10).to_string()}

# COMPLETE ORIGINAL DATAFRAME (ALL ROWS):
# {original_df.to_string(max_rows=None, max_cols=None)}

# ORIGINAL DATAFRAME AS CSV:
# {original_df.to_csv(index=False)}
# """, "ORIGINAL DATAFRAME ANALYSIS")

#         log_to_file(f"""MODIFIED DATAFRAME COMPLETE ANALYSIS:
# Shape: {modified_df.shape}
# Columns: {list(modified_df.columns)}
# Data Types: {dict(modified_df.dtypes.astype(str))}
# Memory Usage: {modified_df.memory_usage(deep=True).sum()} bytes
# Null Counts: {dict(modified_df.isnull().sum())}

# FIRST 10 ROWS PREVIEW:
# {modified_df.head(10).to_string()}

# COMPLETE MODIFIED DATAFRAME (ALL ROWS):
# {modified_df.to_string(max_rows=None, max_cols=None)}

# MODIFIED DATAFRAME AS CSV:
# {modified_df.to_csv(index=False)}
# """, "MODIFIED DATAFRAME ANALYSIS")

#         # Initialize OpenAI client
#         try:
#             client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
#             log_to_file("✅ OpenAI client initialized successfully", "API CLIENT SETUP")
#         except Exception as client_error:
#             log_to_file(f"❌ Failed to initialize OpenAI client: {client_error}", "API CLIENT SETUP ERROR")
#             raise client_error

#         # Prepare comparison data for GPT-4 - NOW WITH COMPLETE DATAFRAMES
#         original_info = {
#             "shape": original_df.shape,
#             "columns": list(original_df.columns),
#             "dtypes": dict(original_df.dtypes.astype(str)),
#             "full_data": original_df.to_string(max_rows=None, max_cols=None),  # COMPLETE dataframe
#             "full_csv": original_df.to_csv(index=False),  # Alternative format
#             "null_counts": dict(original_df.isnull().sum()),
#             "memory_usage": original_df.memory_usage(deep=True).sum(),
#             "summary_stats": original_df.describe(include='all').to_string() if len(original_df) > 0 else "No data"
#         }

#         modified_info = {
#             "shape": modified_df.shape,
#             "columns": list(modified_df.columns),
#             "dtypes": dict(modified_df.dtypes.astype(str)),
#             "full_data": modified_df.to_string(max_rows=None, max_cols=None),  # COMPLETE dataframe
#             "full_csv": modified_df.to_csv(index=False),  # Alternative format
#             "null_counts": dict(modified_df.isnull().sum()),
#             "memory_usage": modified_df.memory_usage(deep=True).sum(),
#             "summary_stats": modified_df.describe(include='all').to_string() if len(modified_df) > 0 else "No data"
#         }

#         # Detect specific changes
#         shape_changed = original_df.shape != modified_df.shape
#         columns_changed = set(original_df.columns) != set(modified_df.columns)

#         log_to_file(f"""STRUCTURAL CHANGES DETECTED:
# Shape Changed: {shape_changed}
# - Original Shape: {original_df.shape}
# - Modified Shape: {modified_df.shape}
# - Rows Added: {max(0, modified_df.shape[0] - original_df.shape[0])}
# - Rows Removed: {max(0, original_df.shape[0] - modified_df.shape[0])}
# - Columns Added: {max(0, modified_df.shape[1] - original_df.shape[1])}
# - Columns Removed: {max(0, original_df.shape[1] - modified_df.shape[1])}

# Columns Changed: {columns_changed}
# - Original Columns: {list(original_df.columns)}
# - Modified Columns: {list(modified_df.columns)}
# - Added Columns: {list(set(modified_df.columns) - set(original_df.columns))}
# - Removed Columns: {list(set(original_df.columns) - set(modified_df.columns))}
# """, "STRUCTURAL CHANGE ANALYSIS")

#         # COMPLETE cell-by-cell comparison - ANALYZE EVERY SINGLE CELL
#         data_changes_detected = False
#         changed_cells = []
#         total_changes = 0
#         changed_rows = set()
#         changed_columns = set()

#         if original_df.shape == modified_df.shape and list(original_df.columns) == list(modified_df.columns):
#             print(f"🔍 Comparing ALL {len(original_df)} rows and {len(original_df.columns)} columns...")
#             log_to_file(f"""STARTING COMPLETE CELL-BY-CELL COMPARISON:
# Total cells to compare: {original_df.shape[0] * original_df.shape[1]}
# Comparing {len(original_df)} rows × {len(original_df.columns)} columns
# This may take time for large dataframes...
# """, "CELL-BY-CELL COMPARISON START")

#             # Compare EVERY cell in the entire dataframe
#             for i in range(len(original_df)):
#                 row_has_changes = False
#                 row_changes = []

#                 for col in original_df.columns:
#                     try:
#                         orig_val = original_df.iloc[i][col]
#                         mod_val = modified_df.iloc[i][col]

#                         # Handle NaN comparisons
#                         if pd.isna(orig_val) and pd.isna(mod_val):
#                             continue
#                         elif pd.isna(orig_val) or pd.isna(mod_val):
#                             data_changes_detected = True
#                             total_changes += 1
#                             row_has_changes = True
#                             changed_columns.add(col)
#                             change_detail = {
#                                 "row": i,
#                                 "column": col,
#                                 "original": str(orig_val),
#                                 "modified": str(mod_val),
#                                 "change_type": "nan_change"
#                             }
#                             row_changes.append(change_detail)
#                             # Store ALL changes, not just first 50
#                             changed_cells.append(change_detail)
#                         elif str(orig_val).strip() != str(mod_val).strip():
#                             data_changes_detected = True
#                             total_changes += 1
#                             row_has_changes = True
#                             changed_columns.add(col)
#                             change_detail = {
#                                 "row": i,
#                                 "column": col,
#                                 "original": str(orig_val),
#                                 "modified": str(mod_val),
#                                 "change_type": "value_change"
#                             }
#                             row_changes.append(change_detail)
#                             # Store ALL changes, not just first 50
#                             changed_cells.append(change_detail)
#                     except Exception as e:
#                         log_to_file(f"ERROR comparing cell at row {i}, column '{col}': {str(e)}")
#                         continue

#                 if row_has_changes:
#                     changed_rows.add(i)
#                     # Log each changed row immediately
#                     log_to_file(f"""ROW {i} CHANGES ({len(row_changes)} changes):
# {json.dumps(row_changes, indent=2)}
# """)

#                 # Log progress every 100 rows for large dataframes
#                 if (i + 1) % 100 == 0:
#                     log_to_file(f"Progress: Processed {i + 1}/{len(original_df)} rows, found {total_changes} changes so far")

#             print(f"✅ Complete comparison finished: {total_changes} total changes detected across {len(changed_rows)} rows")

#             # Log comprehensive change analysis
#             log_to_file(f"""COMPLETE CELL-BY-CELL COMPARISON RESULTS:
# =====================================
# Total Changes Detected: {total_changes}
# Affected Rows: {len(changed_rows)} out of {original_df.shape[0]} ({len(changed_rows)/original_df.shape[0]*100:.1f}%)
# Affected Columns: {len(changed_columns)} out of {len(original_df.columns)} ({len(changed_columns)/len(original_df.columns)*100:.1f}%)

# AFFECTED COLUMNS LIST:
# {list(changed_columns)}

# AFFECTED ROWS LIST:
# {sorted(list(changed_rows))}

# ALL DETECTED CHANGES ({len(changed_cells)} total):
# """, "COMPREHENSIVE CHANGE DETECTION RESULTS")

#             # Log ALL changes, not just a sample
#             for i, change in enumerate(changed_cells):
#                 log_to_file(f"Change {i+1}: Row {change['row']}, Column '{change['column']}' ({change['change_type']}): '{change['original']}' → '{change['modified']}'")

#         else:
#             log_to_file(f"""CANNOT PERFORM CELL-BY-CELL COMPARISON:
# Reason: Shape or column structure differs
# Original shape: {original_df.shape}
# Modified shape: {modified_df.shape}
# Original columns: {list(original_df.columns)}
# Modified columns: {list(modified_df.columns)}
# """, "CELL-BY-CELL COMPARISON SKIPPED")

#         # Calculate change statistics
#         total_cells = original_df.shape[0] * original_df.shape[1] if original_df.size > 0 else 1
#         change_density = total_changes / total_cells

#         log_to_file(f"""COMPREHENSIVE CHANGE STATISTICS:
# ===============================
# Total Cells in Original: {total_cells}
# Total Cells Changed: {total_changes}
# Change Density: {change_density*100:.4f}%
# Percentage of Rows Affected: {len(changed_rows)/original_df.shape[0]*100:.2f}% ({len(changed_rows)}/{original_df.shape[0]})
# Percentage of Columns Affected: {len(changed_columns)/len(original_df.columns)*100:.2f}% ({len(changed_columns)}/{len(original_df.columns)})

# CHANGE PATTERN ANALYSIS:
# - NaN Changes: {len([c for c in changed_cells if c.get('change_type') == 'nan_change'])}
# - Value Changes: {len([c for c in changed_cells if c.get('change_type') == 'value_change'])}
# - Most Affected Columns: {sorted(changed_columns)[:10]}
# - Row Change Distribution: Every {original_df.shape[0]//max(1,len(changed_rows)):.0f} rows on average
# """, "COMPREHENSIVE STATISTICAL ANALYSIS")

#         # STEP 1: Use GPT-4.1 to analyze and generate Claude prompt
#         print("🧠 GPT-4.1: Analyzing changes and generating Claude prompt...")
#         log_to_file("🧠 STARTING GPT-4.1 ANALYSIS TO GENERATE CLAUDE PROMPT...", "GPT-4.1 PROMPT GENERATION START")

#         claude_prompt = _generate_claude_prompt_with_gpt4_logged(
#             client, original_info, modified_info, user_description,
#             total_changes, changed_rows, changed_columns, changed_cells,
#             log_to_file
#         )

#         # STEP 2: Test the Claude prompt by actually running it
#         print("🤖 Testing Claude prompt replication...")
#         log_to_file("🤖 STARTING CLAUDE PROMPT REPLICATION TESTING...", "CLAUDE REPLICATION TESTING START")

#         replication_results = _test_claude_prompt_replication_logged(
#             original_df, modified_df, claude_prompt, max_attempts=3, log_to_file=log_to_file
#         )

#         # STEP 3: Get final analysis with working prompts embedded
#         log_to_file("📊 GENERATING FINAL ANALYSIS WITH GPT-4.1...", "FINAL ANALYSIS GENERATION START")

#         final_analysis = _get_final_analysis_with_prompts_logged(
#             client, original_info, modified_info, user_description,
#             total_changes, changed_rows, changed_columns, changed_cells,
#             claude_prompt, replication_results, log_to_file
#         )

#         # Add technical metadata and log file reference
#         final_analysis["raw_gpt_response"] = final_analysis.get("raw_gpt_response", "")
#         final_analysis["complete_comparison_performed"] = True
#         final_analysis["log_file_path"] = log_file_path  # ALWAYS include log file path
#         final_analysis["full_change_statistics"] = {
#             "total_cells": total_cells,
#             "total_changes": total_changes,
#             "change_density": change_density,
#             "affected_rows": list(changed_rows),
#             "affected_columns": list(changed_columns),
#             "replication_tested": True,
#             "replication_success": replication_results["final_success"],
#             "replication_attempts": len(replication_results["attempts"]),
#             "all_changes": changed_cells  # Include ALL detected changes
#         }

#         # Log final comprehensive results
#         log_to_file(f"""ANALYSIS EXECUTION COMPLETED SUCCESSFULLY
# ==========================================
# Total Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Log File Saved: {log_file_path}
# Log File Size: {os.path.getsize(log_file_path)} bytes

# FINAL RESULTS SUMMARY:
# - Replication Success: {replication_results["final_success"]}
# - Best Match Score: {replication_results.get("best_attempt", {}).get("match_score", 0):.2%}
# - Total Replication Attempts: {len(replication_results["attempts"])}
# - Changes Detected: {total_changes}
# - Change Density: {change_density*100:.4f}%

# COMPLETE FINAL ANALYSIS OBJECT:
# {json.dumps(final_analysis, indent=2, default=str)}

# ✅ ANALYSIS COMPLETE - ALL LOGS SAVED TO: {log_file_path}
# """, "FINAL EXECUTION RESULTS")

#         print(f"📝 Complete analysis with all logs saved to: {log_file_path}")
#         return final_analysis

#     except Exception as e:
#         # ALWAYS log errors, even if everything else fails
#         error_details = f"""CRITICAL ERROR DURING ANALYSIS EXECUTION
# ========================================
# Error Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
# Error Type: {type(e).__name__}
# Error Message: {str(e)}

# FULL STACK TRACE:
# {traceback.format_exc()}

# EXECUTION CONTEXT:
# - User Description: "{user_description}"
# - Original DF Shape: {original_df.shape if 'original_df' in locals() else 'Unknown'}
# - Modified DF Shape: {modified_df.shape if 'modified_df' in locals() else 'Unknown'}
# - Log File: {log_file_path}

# ❌ ANALYSIS FAILED - ERROR LOGGED TO: {log_file_path}
# """

#         log_to_file(error_details, "CRITICAL ERROR")

#         error_msg = f"Error in complete GPT-4 dataframe analysis: {e}"
#         logger.error(error_msg)
#         print(f"❌ Analysis failed but logs saved to: {log_file_path}")

#         return {
#             "change_summary": f"Complete dataframe analysis failed: {user_description}",
#             "change_type": "data_edit",
#             "session_description": f"User made changes to entire dataframe. Description: {user_description}. Error in analysis: {str(e)}",
#             "error": str(e),
#             "complete_comparison_performed": False,
#             "log_file_path": log_file_path,  # ALWAYS include log file path, even on error
#             "error_logged": True
#         }
#         error_msg = f"Error in complete GPT-4 dataframe analysis: {e}"
#         log_to_file(f"""CRITICAL ERROR:
# Error Message: {str(e)}
# Error Type: {type(e).__name__}
# Stack Trace:
# {traceback.format_exc()}
# """, "ERROR ANALYSIS")

#         logger.error(error_msg)
#         return {
#             "change_summary": f"Complete dataframe analysis failed: {user_description}",
#             "change_type": "data_edit",
#             "session_description": f"User made changes to entire dataframe. Description: {user_description}. Error in analysis: {str(e)}",
#             "error": str(e),
#             "complete_comparison_performed": False,
#             "log_file": log_file_path
#         }


# def _generate_claude_prompt_with_gpt4_logged(client, original_info, modified_info, user_description,
#                                            total_changes, changed_rows, changed_columns, changed_cells,
#                                            log_to_file):
#     """
#     Enhanced version with comprehensive logging of GPT-4.1 prompt generation
#     """
#     prompt_generation_request = f"""
#     You are an expert at analyzing dataframe changes and generating precise prompts for Claude 3.7 to replicate manual edits.

#     ORIGINAL DATAFRAME:
#     {original_info['full_data']}

#     MODIFIED DATAFRAME:
#     {modified_info['full_data']}

#     CHANGE ANALYSIS:
#     - Total changes: {total_changes}
#     - Changed rows: {list(changed_rows)[:20] if changed_rows else []}
#     - Changed columns: {list(changed_columns)}
#     - Sample changes: {changed_cells[:10]}
#     - User description: "{user_description}"

#     Generate a PRECISE prompt for Claude 3.7 that would replicate these exact changes.
#     Focus on:
#     1. Specific column names and filtering criteria
#     2. Exact transformation logic
#     3. Clear, executable instructions
#     4. Business context for rent roll data

#     Return ONLY the Claude prompt text, nothing else.
#     """

#     # Log the complete GPT-4.1 prompt
#     log_to_file(f"""GPT-4.1 PROMPT TO GENERATE CLAUDE INSTRUCTIONS:
# Model: gpt-4.1
# Temperature: 0.1
# Max Tokens: 3000

# COMPLETE PROMPT SENT TO GPT-4.1:
# {'-'*60}
# {prompt_generation_request}
# {'-'*60}
# """, "GPT-4.1 REQUEST")

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are an expert at generating precise data transformation prompts. Return only the Claude prompt text."},
#                 {"role": "user", "content": prompt_generation_request}
#             ],
#             max_tokens=3000,
#             temperature=0.1
#         )

#         gpt_response = response.choices[0].message.content.strip()

#         # Log GPT-4.1 response
#         log_to_file(f"""GPT-4.1 RESPONSE (CLAUDE PROMPT):
# Response Length: {len(gpt_response)} characters
# Tokens Used: Approximately {len(gpt_response.split())} words

# GENERATED CLAUDE PROMPT:
# {'-'*60}
# {gpt_response}
# {'-'*60}
# """, "GPT-4.1 RESPONSE")

#         return gpt_response

#     except Exception as e:
#         error_msg = f"GPT-4.1 API Error: {str(e)}"
#         log_to_file(f"""GPT-4.1 API CALL FAILED:
# Error: {error_msg}
# Fallback: Using basic prompt
# """)
#         return f"Replicate the manual edits described as: {user_description}"


# def _test_claude_prompt_replication_logged(original_df, target_df, claude_prompt, max_attempts=3, log_to_file=None):
#     """
#     Enhanced version with comprehensive logging of Claude replication attempts
#     """
#     attempts = []
#     current_prompt = claude_prompt

#     log_to_file(f"""CLAUDE REPLICATION TESTING STARTED:
# Maximum Attempts: {max_attempts}
# Target DataFrame Shape: {target_df.shape}
# Original DataFrame Shape: {original_df.shape}

# INITIAL CLAUDE PROMPT TO TEST:
# {'-'*60}
# {claude_prompt}
# {'-'*60}
# """, "REPLICATION TESTING INITIALIZATION")

#     for attempt in range(max_attempts):
#         print(f"🔄 Attempt {attempt + 1}/{max_attempts}")
#         log_to_file(f"Starting attempt {attempt + 1}/{max_attempts}...", f"ATTEMPT {attempt + 1}")

#         try:
#             # Call actual Claude API with the generated prompt
#             claude_response = _call_actual_claude_logged(current_prompt, original_df, log_to_file, attempt + 1)

#             # Extract and execute code
#             code_blocks = _extract_code_blocks(claude_response)

#             log_to_file(f"""CODE EXTRACTION RESULTS:
# Found {len(code_blocks)} code blocks
# Code Blocks:
# """)

#             for i, code in enumerate(code_blocks):
#                 log_to_file(f"Code Block {i+1}:\n```python\n{code}\n```\n")

#             if not code_blocks:
#                 attempts.append({
#                     "attempt": attempt + 1,
#                     "success": False,
#                     "error": "No code found",
#                     "prompt_used": current_prompt
#                 })
#                 log_to_file("No code blocks found in Claude response")
#                 continue

#             # Execute the code with proper globals
#             test_df = original_df.copy()
#             exec_globals = {
#                 "df": test_df,
#                 "pd": pd,
#                 "np": np,
#                 "os": os,
#                 "datetime": datetime
#             }

#             log_to_file("Starting code execution...")
#             execution_output = ""

#             for i, code in enumerate(code_blocks):
#                 try:
#                     log_to_file(f"Executing code block {i+1}...")
#                     exec(code, exec_globals)
#                     execution_output += f"Code block {i+1} executed successfully\n"
#                 except Exception as exec_error:
#                     execution_output += f"Code block {i+1} failed: {str(exec_error)}\n"
#                     log_to_file(f"Code block {i+1} execution error: {str(exec_error)}")

#             result_df = exec_globals["df"]
#             log_to_file(f"Execution completed. Result DataFrame shape: {result_df.shape}")

#             # Calculate match score
#             match_score = _calculate_match_score(target_df, result_df)

#             attempts.append({
#                 "attempt": attempt + 1,
#                 "success": match_score >= 0.95,
#                 "match_score": match_score,
#                 "generated_code": code_blocks,
#                 "prompt_used": current_prompt,
#                 "result_shape": result_df.shape,
#                 "target_shape": target_df.shape,
#                 "execution_output": execution_output
#             })

#             log_to_file(f"""ATTEMPT {attempt + 1} RESULTS:
# Match Score: {match_score:.2%}
# Success Threshold (95%): {'✅ PASSED' if match_score >= 0.95 else '❌ FAILED'}
# Result Shape: {result_df.shape}
# Target Shape: {target_df.shape}
# Execution Output:
# {execution_output}
# """)

#             print(f"📊 Match score: {match_score:.2%}")

#             if match_score >= 0.95:
#                 print("✅ Replication successful!")
#                 log_to_file("🎉 REPLICATION SUCCESSFUL! Stopping attempts.")
#                 break
#             else:
#                 # Improve prompt for next attempt
#                 current_prompt = _improve_claude_prompt(current_prompt, target_df, result_df, match_score)
#                 log_to_file(f"Match score below threshold. Improving prompt for next attempt:\n{current_prompt}")

#         except Exception as e:
#             error_msg = str(e)
#             attempts.append({
#                 "attempt": attempt + 1,
#                 "success": False,
#                 "error": error_msg,
#                 "prompt_used": current_prompt
#             })
#             log_to_file(f"ATTEMPT {attempt + 1} FAILED with error: {error_msg}")
#             print(f"❌ Error: {error_msg}")

#     final_success = any(attempt["success"] for attempt in attempts)
#     best_attempt = max(attempts, key=lambda x: x.get("match_score", 0)) if attempts else None

#     log_to_file(f"""REPLICATION TESTING COMPLETE:
# Final Success: {final_success}
# Best Match Score: {best_attempt.get('match_score', 0):.2% if best_attempt else 'N/A'}
# Total Attempts: {len(attempts)}

# ALL ATTEMPTS SUMMARY:
# """, "REPLICATION TESTING RESULTS")

#     for i, attempt in enumerate(attempts, 1):
#         log_to_file(f"Attempt {i}: Success={attempt.get('success', False)}, Score={attempt.get('match_score', 0):.2%}, Error={attempt.get('error', 'None')}")

#     return {
#         "attempts": attempts,
#         "final_success": final_success,
#         "best_attempt": best_attempt,
#         "final_prompt": best_attempt["prompt_used"] if best_attempt else claude_prompt
#     }


# def _call_actual_claude_logged(prompt, original_df, log_to_file, attempt_number):
#     """
#     Enhanced version with comprehensive logging of Claude API calls
#     """
#     log_to_file(f"""CALLING CLAUDE API - ATTEMPT {attempt_number}:
# Model: claude-3-7-sonnet-20250219
# Temperature: 0.2
# Max Tokens: 2000

# PROMPT BEING SENT TO CLAUDE:
# {'-'*60}
# {prompt}
# {'-'*60}

# DATAFRAME CONTEXT BEING SENT:
# Shape: {original_df.shape}
# Columns: {list(original_df.columns)}
# First 10 rows:
# {original_df.head(10).to_string()}
# """, f"CLAUDE API CALL {attempt_number}")

#     try:
#         # Get Anthropic client
#         anthropic_client = Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

#         # Prepare dataframe context for Claude
#         df_summary = f"""
#         DATAFRAME CONTENT:
#         {original_df.to_string(max_rows=50)}

#         DATAFRAME STATISTICS:
#         - Shape: {original_df.shape}
#         - Columns: {list(original_df.columns)}
#         - Data types: {dict(original_df.dtypes)}
#         - Null values per column: {dict(original_df.isnull().sum())}
#         """

#         # System prompt for Claude
#         claude_system_prompt = """You are an expert Python data analyst.
#         You will receive a dataframe that is already loaded as 'df' and a specific task to perform.
#         Generate Python pandas code to accomplish the task.

#         IMPORTANT RULES:
#         1. The dataframe 'df' is already loaded - do not load or import it
#         2. Wrap all code in ```python and ``` blocks
#         3. Do not use try-except blocks - let errors propagate naturally
#         4. Be precise and specific in your transformations
#         5. Provide working, executable code
#         6. Focus on the exact transformation requested"""

#         # Prepare messages for Claude
#         claude_messages = [
#             {
#                 "role": "user",
#                 "content": f"Here is the dataframe that's already loaded as 'df':\n\n{df_summary}\n\nTASK: {prompt}\n\nGenerate Python code to accomplish this task."
#             }
#         ]

#         # Call Claude
#         claude_response = anthropic_client.messages.create(
#             model="claude-sonnet-4-20250514",  # Use the latest Claude model
#             system=claude_system_prompt,
#             messages=claude_messages,
#             max_tokens=2000,
#             temperature=0.2
#         )

#         # Extract response text
#         response_text = claude_response.content[0].text

#         log_to_file(f"""CLAUDE API RESPONSE - ATTEMPT {attempt_number}:
# Response Length: {len(response_text)} characters
# Response Received Successfully: ✅

# FULL CLAUDE RESPONSE:
# {'-'*60}
# {response_text}
# {'-'*60}
# """)

#         return response_text

#     except Exception as e:
#         error_msg = f"Claude API Error: {str(e)}"
#         log_to_file(f"""CLAUDE API CALL FAILED - ATTEMPT {attempt_number}:
# Error: {error_msg}
# Using fallback response
# """)

#         # Fallback response if Claude API fails
#         fallback_response = f"""
#         I'll help you with this task. Here's the code:

#         ```python
#         # Error calling Claude API: {str(e)}
#         # Fallback code
#         print("Claude API error, using fallback")
#         print(f"Dataframe shape: {{df.shape}}")
#         print(df.head())
#         ```
#         """

#         log_to_file(f"Fallback response generated:\n{fallback_response}")
#         return fallback_response


# def _get_final_analysis_with_prompts_logged(client, original_info, modified_info, user_description,
#                                           total_changes, changed_rows, changed_columns, changed_cells,
#                                           claude_prompt, replication_results, log_to_file):
#     """
#     Enhanced version with comprehensive logging of final analysis generation
#     """
#     best_attempt = replication_results.get("best_attempt", {})
#     final_claude_prompt = replication_results.get("final_prompt", claude_prompt)
#     success_status = "SUCCESS" if replication_results["final_success"] else "PARTIAL"

#     # Calculate statistics
#     total_cells = original_info["shape"][0] * original_info["shape"][1] if original_info["shape"][0] > 0 else 1
#     change_density = total_changes / total_cells

#     analysis_prompt = f"""
#     Analyze this dataframe change and provide a comprehensive summary.

#     ORIGINAL: {original_info['shape']} with data:
#     {original_info['full_data'][:2000]}...

#     MODIFIED: {modified_info['shape']} with data:
#     {modified_info['full_data'][:2000]}...

#     CHANGES: {total_changes} cells changed ({change_density*100:.1f}%)
#     USER DESCRIPTION: {user_description}

#     TESTED REPLICATION: {success_status}
#     - Attempts: {len(replication_results['attempts'])}
#     - Best match: {best_attempt.get('match_score', 0):.1%}

#     Provide analysis in this exact JSON format:
#     {{
#         "change_summary": "Detailed technical summary including WORKING_CLAUDE_PROMPT: {final_claude_prompt} and GPT4_ANALYSIS_PROMPT: [the prompt used to generate the Claude prompt] - describe what changed and how replication performed",
#         "change_type": "data_edit|structure_change|mixed",
#         "structural_changes": {{
#             "rows_added": {max(0, modified_info['shape'][0] - original_info['shape'][0])},
#             "rows_removed": {max(0, original_info['shape'][0] - modified_info['shape'][0])},
#             "columns_added": [],
#             "columns_removed": []
#         }},
#         "data_modifications": {{
#             "cells_changed": {total_changes},
#             "total_cells": {total_cells},
#             "change_percentage": {change_density*100:.2f},
#             "rows_affected": {len(changed_rows)},
#             "columns_affected": {list(changed_columns)},
#             "common_patterns": ["Patterns identified"],
#             "data_quality_impact": "improved|degraded|neutral"
#         }},
#         "business_impact": {{
#             "rent_calculations_affected": "Analysis of rent impact",
#             "tenant_information_updated": "Analysis of tenant data changes",
#             "occupancy_status_changed": "Analysis of occupancy changes"
#         }},
#         "recommendations": ["Specific recommendations"],
#         "session_description": "REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | REPLICATION_ATTEMPTS: {len(replication_results['attempts'])} | USER_EDIT: {user_description} | Changes: {total_changes} cells ({change_density*100:.1f}%) | Best match: {best_attempt.get('match_score', 0):.1%}"
#     }}
#     """

#     log_to_file(f"""FINAL ANALYSIS GENERATION:
# Sending final analysis request to GPT-4.1...

# PROMPT FOR FINAL ANALYSIS:
# {'-'*60}
# {analysis_prompt}
# {'-'*60}
# """, "FINAL ANALYSIS GENERATION")

#     try:
#         response = client.chat.completions.create(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": "You are a data analyst. Return only valid JSON."},
#                 {"role": "user", "content": analysis_prompt}
#             ],
#             max_tokens=3000,
#             temperature=0.2
#         )

#         gpt_final_response = response.choices[0].message.content
#         log_to_file(f"""FINAL ANALYSIS GPT-4.1 RESPONSE:
# {'-'*60}
# {gpt_final_response}
# {'-'*60}
# """)

#         try:
#             import json
#             import re
#             json_match = re.search(r'{.*}', gpt_final_response, re.DOTALL)
#             if json_match:
#                 parsed_analysis = json.loads(json_match.group(0))
#                 log_to_file("✅ Successfully parsed JSON from final analysis response")
#                 return parsed_analysis
#             else:
#                 log_to_file("❌ No JSON found in final analysis response")
#         except Exception as json_error:
#             log_to_file(f"❌ JSON parsing failed: {str(json_error)}")

#     except Exception as api_error:
#         log_to_file(f"❌ Final analysis API call failed: {str(api_error)}")

#     # Fallback
#     fallback_analysis = {
#         "change_summary": f"WORKING_CLAUDE_PROMPT: {final_claude_prompt} | GPT4_ANALYSIS_PROMPT: [embedded] | {user_description} - {total_changes} changes with {replication_results['final_success']} replication",
#         "change_type": "data_edit",
#         "session_description": f"REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | USER_EDIT: {user_description} | Changes: {total_changes} cells",
#         "data_modifications": {
#             "cells_changed": total_changes,
#             "total_cells": total_cells,
#             "change_percentage": change_density*100
#         }
#     }

#     log_to_file(f"Using fallback analysis:\n{json.dumps(fallback_analysis, indent=2)}")

#     return fallback_analysis


# # Additional helper functions for the enhanced logging system

# def _extract_code_blocks(response_text):
#     """
#     Extract Python code blocks from response text
#     """
#     import re
#     code_pattern = r'```(?:python)?\s*(.*?)```'
#     code_blocks = re.findall(code_pattern, response_text, re.DOTALL)
#     return [block.strip() for block in code_blocks if block.strip()]


# def _calculate_match_score(target_df, result_df):
#     """
#     Calculate how closely the result matches the target with improved scoring.
#     Now gives partial credit for near-matches instead of strict 0/1 scoring.
#     """
#     import pandas as pd

#     # Handle edge cases
#     if target_df.empty and result_df.empty:
#         return 1.0
#     if target_df.empty or result_df.empty:
#         return 0.0

#     # 1. Shape similarity score (30% weight)
#     target_rows, target_cols = target_df.shape
#     result_rows, result_cols = result_df.shape

#     # Row similarity (allows for minor filtering differences)
#     if target_rows == 0:
#         row_score = 1.0 if result_rows == 0 else 0.0
#     else:
#         row_diff = abs(target_rows - result_rows)
#         row_score = max(0, 1.0 - (row_diff / target_rows))

#     # Column similarity (should be exact for most use cases)
#     col_score = 1.0 if target_cols == result_cols else 0.0

#     shape_score = 0.7 * row_score + 0.3 * col_score

#     # 2. Column structure score (20% weight)
#     target_columns = list(target_df.columns)
#     result_columns = list(result_df.columns)

#     if target_columns == result_columns:
#         column_score = 1.0
#     else:
#         # Partial credit for column overlap
#         common_cols = set(target_columns) & set(result_columns)
#         total_unique_cols = set(target_columns) | set(result_columns)
#         column_score = len(common_cols) / len(total_unique_cols) if total_unique_cols else 0.0

#     # 3. Content similarity score (50% weight)
#     content_score = 0.0

#     if target_columns == result_columns and len(target_columns) > 0:
#         # Compare overlapping rows when columns match
#         min_rows = min(len(target_df), len(result_df))

#         if min_rows > 0:
#             total_cells = 0
#             matching_cells = 0

#             # Compare cell by cell for overlapping area
#             for i in range(min_rows):
#                 for col in target_columns:
#                     total_cells += 1

#                     try:
#                         target_val = target_df.iloc[i][col]
#                         result_val = result_df.iloc[i][col]

#                         # Handle NaN comparisons
#                         if pd.isna(target_val) and pd.isna(result_val):
#                             matching_cells += 1
#                         elif pd.isna(target_val) or pd.isna(result_val):
#                             # NaN vs non-NaN = no match
#                             continue
#                         else:
#                             # String comparison with whitespace handling
#                             target_str = str(target_val).strip()
#                             result_str = str(result_val).strip()

#                             if target_str == result_str:
#                                 matching_cells += 1
#                             else:
#                                 # Try numeric comparison for potential formatting differences
#                                 try:
#                                     target_num = float(target_str.replace(',', '').replace('$', ''))
#                                     result_num = float(result_str.replace(',', '').replace('$', ''))
#                                     if abs(target_num - result_num) < 0.01:  # Allow small floating point differences
#                                         matching_cells += 1
#                                 except (ValueError, TypeError):
#                                     # Not numeric, keep as non-match
#                                     continue
#                     except (IndexError, KeyError):
#                         # Skip invalid cell references
#                         continue

#             content_score = matching_cells / total_cells if total_cells > 0 else 0.0

#             # Bonus for exact row count match when content is high
#             if len(target_df) == len(result_df) and content_score > 0.9:
#                 content_score = min(1.0, content_score * 1.05)

#     else:
#         # Different column structures - can only do basic comparison
#         if min(len(target_df), len(result_df)) > 0:
#             # Give small credit for having some data with similar row count
#             row_similarity = 1.0 - abs(len(target_df) - len(result_df)) / max(len(target_df), len(result_df))
#             content_score = 0.2 * row_similarity  # Low score for structure mismatch

#     # 4. Calculate weighted final score
#     final_score = (0.30 * shape_score +
#                   0.20 * column_score +
#                   0.50 * content_score)

#     # Ensure score is between 0 and 1
#     return max(0.0, min(1.0, final_score))



# def _improve_claude_prompt(current_prompt, target_df, result_df, match_score):
#     """
#     Improve the Claude prompt based on the mismatch
#     """
#     feedback = f"\nPREVIOUS ATTEMPT FEEDBACK:\n"
#     feedback += f"Match score: {match_score:.2%}\n"
#     feedback += f"Target shape: {target_df.shape}, Result shape: {result_df.shape}\n"

#     if target_df.shape != result_df.shape:
#         feedback += "Shape mismatch detected. Please check row filtering logic.\n"

#     return current_prompt + feedback


# # Enhanced save function that also uses the detailed logging
# def save_edited_dataframe_enhanced_with_logging(edited_df, description):
#     """
#     Enhanced version that uses the new comprehensive logging system
#     for analyzing manual dataframe changes.
#     """
#     global app_state, session_recorder

#     if edited_df is None or edited_df.empty:
#         return "No data to save", gr.update()

#     try:
#         # Convert the edited dataframe to proper pandas DataFrame if needed
#         if not isinstance(edited_df, pd.DataFrame):
#             edited_df = pd.DataFrame(edited_df)

#         # Get the original dataframe for comparison
#         original_df = app_state["df"].copy()

#         logger.info("Analyzing dataframe changes with enhanced GPT-4.1 logging...")
#         print("🤖 Analyzing changes with enhanced GPT-4.1 logging system...")

#         # Use the enhanced analysis function with comprehensive logging
#         change_analysis = analyze_dataframe_changes_with_gpt4(
#             original_df=original_df,
#             modified_df=edited_df,
#             user_description=description
#         )

#         # Generate a meaningful description if not provided
#         if not description:
#             description = change_analysis.get("change_summary", "Manual edits via data editor")

#         # Save as new version
#         version_name = save_dataframe_version(edited_df, description)

#         # Update the app state with the edited dataframe
#         app_state["df"] = edited_df

#         # Record this in the copiloting session if active
#         if session_recorder.current_session_file:
#             session_description = change_analysis.get("session_description", f"Manual data edits: {description}")

#             # Create detailed session entry with log file reference
#             session_entry = f"""
# MANUAL DATA EDIT SESSION WITH ENHANCED LOGGING
# ==============================================
# Timestamp: {datetime.now().strftime('%H:%M:%S')}
# Edit Description: {description}
# Version Saved: {version_name}
# Log File: {change_analysis.get('log_file', 'N/A')}

# ENHANCED GPT-4.1 CHANGE ANALYSIS:
# {'-' * 40}
# Change Summary: {change_analysis.get('change_summary', 'N/A')}
# Change Type: {change_analysis.get('change_type', 'N/A')}
# Replication Success: {change_analysis.get('full_change_statistics', {}).get('replication_success', 'Unknown')}
# Replication Attempts: {change_analysis.get('full_change_statistics', {}).get('replication_attempts', 'Unknown')}

# Structural Changes:
# {json.dumps(change_analysis.get('structural_changes', {}), indent=2)}

# Data Modifications:
# {json.dumps(change_analysis.get('data_modifications', {}), indent=2)}

# Business Impact:
# {json.dumps(change_analysis.get('business_impact', {}), indent=2)}

# Recommendations:
# {chr(10).join([f"• {rec}" for rec in change_analysis.get('recommendations', [])])}

# Technical Statistics:
# - Total Cells: {change_analysis.get('full_change_statistics', {}).get('total_cells', 'Unknown')}
# - Changed Cells: {change_analysis.get('full_change_statistics', {}).get('total_changes', 'Unknown')}
# - Change Density: {change_analysis.get('full_change_statistics', {}).get('change_density', 0)*100:.2f}%
# - Affected Rows: {len(change_analysis.get('full_change_statistics', {}).get('affected_rows', []))}
# - Affected Columns: {len(change_analysis.get('full_change_statistics', {}).get('affected_columns', []))}

# Original DataFrame Shape: {original_df.shape}
# Modified DataFrame Shape: {edited_df.shape}
# {'-' * 80}
# """

#             # Append to session file
#             with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
#                 f.write(session_entry + "\n")

#             # Record in session data structure
#             session_recorder.record_conversation_turn(
#                 user_message=f"MANUAL EDIT (Enhanced Logging): {description}",
#                 ai_response=session_description,
#                 action_type="manual_data_edit_enhanced",
#                 code_executed=None,
#                 version_saved=version_name
#             )

#             # Record the dataframe version change
#             session_recorder.record_dataframe_version(
#                 version_name=version_name,
#                 description=description,
#                 shape=list(edited_df.shape),
#                 columns=list(edited_df.columns)
#             )

#             # Record any issues found by GPT-4
#             replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
#             if not replication_success:
#                 session_recorder.record_issue_found(
#                     f"Manual edit replication failed: {description}. Check detailed log for analysis.",
#                     severity="medium"
#                 )

#             logger.info("Enhanced manual edit analysis recorded in copiloting session")

#         # Log the changes
#         logger.info(f"Saved edited dataframe as version {version_name}")

#         # Create detailed success message with log file information
#         log_file_path = change_analysis.get('log_file', 'N/A')
#         replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
#         replication_attempts = change_analysis.get('full_change_statistics', {}).get('replication_attempts', 0)

#         success_message = f"""✅ Successfully saved as version {version_name}

# 🤖 Enhanced GPT-4.1 Analysis Summary:
# {change_analysis.get('change_summary', 'Changes analyzed')[:200]}...

# 📊 Change Details:
# • Change Type: {change_analysis.get('change_type', 'Unknown')}
# • Original Shape: {original_df.shape}
# • New Shape: {edited_df.shape}
# • Replication Success: {'✅ Yes' if replication_success else '❌ Partial/Failed'}
# • Replication Attempts: {replication_attempts}

# 📝 Session Recording: {'✅ Recorded' if session_recorder.current_session_file else '❌ No active session'}

# 📁 Detailed Analysis Log:
# {log_file_path}

# This log contains:
# • Complete GPT-4.1 prompts and responses
# • All Claude API calls and generated code
# • Step-by-step replication attempts
# • Cell-by-cell change analysis
# • Business impact assessment
# """

#         # Add recommendations if available
#         if change_analysis.get('recommendations'):
#             success_message += f"\n💡 Recommendations:\n"
#             for rec in change_analysis['recommendations'][:3]:  # Show first 3
#                 success_message += f"• {rec}\n"

#         # Add note about log file location
#         success_message += f"\n📄 For complete debugging information, check: {log_file_path}"

#         return success_message, gr.update(value=edited_df)

#     except Exception as e:
#         error_msg = f"❌ Error saving: {str(e)}"
#         logger.error(f"Error saving edited dataframe: {e}")
#         logger.error(traceback.format_exc())

#         # Still try to record the error in session
#         if session_recorder.current_session_file:
#             session_recorder.record_conversation_turn(
#                 user_message=f"MANUAL EDIT FAILED (Enhanced): {description}",
#                 ai_response=error_msg,
#                 action_type="manual_edit_error_enhanced",
#                 code_executed=None,
#                 version_saved=None
#             )

#         return error_msg, gr.update()


# # Function to view and summarize all log files
# def get_manual_edit_logs_summary():
#     """
#     Generate a summary of all manual edit analysis log files
#     """
#     logs_dir = "manual_edit_analysis_logs"

#     if not os.path.exists(logs_dir):
#         return "No manual edit analysis logs found yet."

#     try:
#         log_files = [f for f in os.listdir(logs_dir) if f.endswith('_detailed_analysis.txt')]

#         if not log_files:
#             return "No detailed analysis log files found."

#         # Sort by creation time (newest first)
#         log_files.sort(key=lambda x: os.path.getctime(os.path.join(logs_dir, x)), reverse=True)

#         summary = f"""📁 Manual Edit Analysis Logs Summary
# Found {len(log_files)} detailed analysis log files:

# """

#         for i, log_file in enumerate(log_files[:10], 1):  # Show last 10
#             file_path = os.path.join(logs_dir, log_file)
#             file_size = os.path.getsize(file_path)
#             created_time = datetime.fromtimestamp(os.path.getctime(file_path))

#             # Try to extract session info from filename
#             session_id = log_file.replace('_detailed_analysis.txt', '')

#             summary += f"""{i}. {session_id}
#    📄 File: {log_file}
#    📅 Created: {created_time.strftime('%Y-%m-%d %H:%M:%S')}
#    💾 Size: {file_size:,} bytes
#    📁 Path: {file_path}

# """

#         if len(log_files) > 10:
#             summary += f"... and {len(log_files) - 10} more log files\n"

#         summary += f"""
# 📋 Log File Contents Include:
# • Complete GPT-4.1 prompts and responses
# • All Claude API calls and code generation
# • Step-by-step replication testing results
# • Cell-by-cell dataframe comparison analysis
# • Business impact and recommendation analysis
# • Detailed error messages and debugging information

# 📂 All logs are saved in: {logs_dir}/
# """

#         return summary

#     except Exception as e:
#         return f"Error reading log files: {str(e)}"


# # Function to read a specific log file
# def read_manual_edit_log(session_id):
#     """
#     Read and return the contents of a specific manual edit analysis log
#     """
#     logs_dir = "manual_edit_analysis_logs"
#     log_file_path = os.path.join(logs_dir, f"{session_id}_detailed_analysis.txt")

#     if not os.path.exists(log_file_path):
#         return f"Log file not found: {log_file_path}"

#     try:
#         with open(log_file_path, 'r', encoding='utf-8') as f:
#             content = f.read()

#         return f"""📄 Manual Edit Analysis Log: {session_id}
# {'='*80}

# {content}

# {'='*80}
# End of log file: {log_file_path}
# """

#     except Exception as e:
#         return f"Error reading log file: {str(e)}"

In [ ]:
def analyze_dataframe_changes_with_gpt4(original_df, modified_df, user_description=""):
    """
    Enhanced version with ALL CRITICAL FIXES APPLIED:
    1. Fixed logging bug with proper string formatting
    2. Focus on data type/formatting precision
    3. Implement attempt-to-attempt learning
    4. Add data type validation for exact target schema matching
    """
    import os
    import json
    import traceback
    from datetime import datetime

    # Create detailed logging directory
    logs_dir = "manual_edit_analysis_logs"
    os.makedirs(logs_dir, exist_ok=True)

    # Generate unique log session ID
    log_session_id = f"manual_edit_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')[:-3]}"
    log_file_path = os.path.join(logs_dir, f"{log_session_id}_detailed_analysis.txt")

    # FIXED LOGGING FUNCTION - Proper string formatting
    def log_to_file(content, section_title=""):
        try:
            with open(log_file_path, 'a', encoding='utf-8') as f:
                if section_title:
                    f.write(f"\n{'='*80}\n")
                    f.write(f"{section_title}\n")
                    f.write(f"{'='*80}\n")
                f.write(f"{content}\n")
                f.flush()
        except Exception as e:
            print(f"ERROR writing to log file: {e}")

    # Initialize comprehensive log
    try:
        log_to_file(f"""COMPREHENSIVE MANUAL EDIT ANALYSIS LOG - ENHANCED VERSION
Session ID: {log_session_id}
Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
User Description: "{user_description}"
Log File Path: {log_file_path}

ENHANCEMENTS APPLIED:
1. ✅ Fixed logging string formatting bug
2. ✅ Enhanced data type/formatting precision matching
3. ✅ Implemented attempt-to-attempt learning system
4. ✅ Added comprehensive data type validation

ANALYSIS OVERVIEW:
This log contains the complete workflow of analyzing manual dataframe edits:
1. Original vs Modified dataframe comparison with data type analysis
2. GPT-4.1 prompt generation with schema validation
3. Claude API calls with learning feedback loops
4. Code execution with precision validation
5. Replication success/failure analysis with detailed diagnostics

FUNCTION EXECUTION STATUS: STARTING...
""", "SESSION INITIALIZATION")

        print(f"📝 Enhanced manual edit analysis log initialized: {log_file_path}")

    except Exception as init_error:
        print(f"CRITICAL: Could not initialize log file: {init_error}")

    try:
        # ENHANCED DATAFRAME ANALYSIS with data type focus
        log_to_file(f"""ORIGINAL DATAFRAME COMPLETE ANALYSIS:
Shape: {original_df.shape}
Columns: {list(original_df.columns)}
Data Types: {dict(original_df.dtypes.astype(str))}
Data Type Details: {json.dumps({col: str(dtype) for col, dtype in original_df.dtypes.items()}, indent=2)}
Memory Usage: {original_df.memory_usage(deep=True).sum()} bytes
Null Counts: {dict(original_df.isnull().sum())}
Index Type: {type(original_df.index).__name__}
Column Order: {list(original_df.columns)}

FIRST 10 ROWS PREVIEW:
{original_df.head(10).to_string()}

COMPLETE ORIGINAL DATAFRAME (ALL ROWS):
{original_df.to_string(max_rows=None, max_cols=None)}

ORIGINAL DATAFRAME AS CSV:
{original_df.to_csv(index=False)}

DATA TYPE ANALYSIS:
{_analyze_dataframe_schema(original_df, "ORIGINAL")}
""", "ORIGINAL DATAFRAME ANALYSIS")

        log_to_file(f"""MODIFIED DATAFRAME COMPLETE ANALYSIS:
Shape: {modified_df.shape}
Columns: {list(modified_df.columns)}
Data Types: {dict(modified_df.dtypes.astype(str))}
Data Type Details: {json.dumps({col: str(dtype) for col, dtype in modified_df.dtypes.items()}, indent=2)}
Memory Usage: {modified_df.memory_usage(deep=True).sum()} bytes
Null Counts: {dict(modified_df.isnull().sum())}
Index Type: {type(modified_df.index).__name__}
Column Order: {list(modified_df.columns)}

FIRST 10 ROWS PREVIEW:
{modified_df.head(10).to_string()}

COMPLETE MODIFIED DATAFRAME (ALL ROWS):
{modified_df.to_string(max_rows=None, max_cols=None)}

MODIFIED DATAFRAME AS CSV:
{modified_df.to_csv(index=False)}

DATA TYPE ANALYSIS:
{_analyze_dataframe_schema(modified_df, "MODIFIED")}
""", "MODIFIED DATAFRAME ANALYSIS")

        # Initialize OpenAI client
        try:
            client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)
            log_to_file("✅ OpenAI client initialized successfully", "API CLIENT SETUP")
        except Exception as client_error:
            log_to_file(f"❌ Failed to initialize OpenAI client: {client_error}", "API CLIENT SETUP ERROR")
            raise client_error

        # ENHANCED SCHEMA COMPARISON
        schema_comparison = _compare_dataframe_schemas(original_df, modified_df)
        log_to_file(f"""ENHANCED SCHEMA COMPARISON:
{json.dumps(schema_comparison, indent=2)}
""", "SCHEMA COMPARISON ANALYSIS")

        # Prepare enhanced comparison data for GPT-4
        original_info = {
            "shape": original_df.shape,
            "columns": list(original_df.columns),
            "dtypes": dict(original_df.dtypes.astype(str)),
            "dtype_details": {col: str(dtype) for col, dtype in original_df.dtypes.items()},
            "full_data": original_df.to_string(max_rows=None, max_cols=None),
            "full_csv": original_df.to_csv(index=False),
            "null_counts": dict(original_df.isnull().sum()),
            "memory_usage": original_df.memory_usage(deep=True).sum(),
            "schema_analysis": _analyze_dataframe_schema(original_df, "ORIGINAL"),
            "summary_stats": original_df.describe(include='all').to_string() if len(original_df) > 0 else "No data"
        }

        modified_info = {
            "shape": modified_df.shape,
            "columns": list(modified_df.columns),
            "dtypes": dict(modified_df.dtypes.astype(str)),
            "dtype_details": {col: str(dtype) for col, dtype in modified_df.dtypes.items()},
            "full_data": modified_df.to_string(max_rows=None, max_cols=None),
            "full_csv": modified_df.to_csv(index=False),
            "null_counts": dict(modified_df.isnull().sum()),
            "memory_usage": modified_df.memory_usage(deep=True).sum(),
            "schema_analysis": _analyze_dataframe_schema(modified_df, "MODIFIED"),
            "summary_stats": modified_df.describe(include='all').to_string() if len(modified_df) > 0 else "No data"
        }

        # Enhanced structural change detection
        shape_changed = original_df.shape != modified_df.shape
        columns_changed = set(original_df.columns) != set(modified_df.columns)
        schema_changes = schema_comparison

        log_to_file(f"""ENHANCED STRUCTURAL CHANGE ANALYSIS:
Shape Changed: {shape_changed}
- Original Shape: {original_df.shape}
- Modified Shape: {modified_df.shape}
- Rows Added: {max(0, modified_df.shape[0] - original_df.shape[0])}
- Rows Removed: {max(0, original_df.shape[0] - modified_df.shape[0])}
- Columns Added: {max(0, modified_df.shape[1] - original_df.shape[1])}
- Columns Removed: {max(0, original_df.shape[1] - modified_df.shape[1])}

Columns Changed: {columns_changed}
- Original Columns: {list(original_df.columns)}
- Modified Columns: {list(modified_df.columns)}
- Added Columns: {list(set(modified_df.columns) - set(original_df.columns))}
- Removed Columns: {list(set(original_df.columns) - set(modified_df.columns))}

SCHEMA CHANGES DETECTED:
{json.dumps(schema_changes, indent=2)}
""", "ENHANCED STRUCTURAL CHANGE ANALYSIS")

        # Enhanced cell-by-cell comparison with data type focus
        data_changes_detected = False
        changed_cells = []
        total_changes = 0
        changed_rows = set()
        changed_columns = set()
        data_type_mismatches = []

        if original_df.shape == modified_df.shape and list(original_df.columns) == list(modified_df.columns):
            print(f"🔍 Enhanced comparison: {len(original_df)} rows × {len(original_df.columns)} columns...")
            log_to_file(f"""STARTING ENHANCED CELL-BY-CELL COMPARISON:
Total cells to compare: {original_df.shape[0] * original_df.shape[1]}
Comparing {len(original_df)} rows × {len(original_df.columns)} columns
Enhanced features: Data type validation, formatting precision, null handling
This may take time for large dataframes...
""", "ENHANCED CELL-BY-CELL COMPARISON START")

            # Enhanced cell comparison with data type awareness
            for i in range(len(original_df)):
                row_has_changes = False
                row_changes = []

                for col in original_df.columns:
                    try:
                        orig_val = original_df.iloc[i][col]
                        mod_val = modified_df.iloc[i][col]

                        # Enhanced comparison with data type awareness
                        change_detected, change_detail = _enhanced_cell_comparison(
                            orig_val, mod_val, i, col,
                            original_df.dtypes[col], modified_df.dtypes[col]
                        )

                        if change_detected:
                            data_changes_detected = True
                            total_changes += 1
                            row_has_changes = True
                            changed_columns.add(col)
                            row_changes.append(change_detail)
                            changed_cells.append(change_detail)

                            # Track data type mismatches
                            if change_detail.get('data_type_changed'):
                                data_type_mismatches.append(change_detail)

                    except Exception as e:
                        log_to_file(f"ERROR comparing cell at row {i}, column '{col}': {str(e)}")
                        continue

                if row_has_changes:
                    changed_rows.add(i)

                # Log progress every 100 rows
                if (i + 1) % 100 == 0:
                    log_to_file(f"Progress: Processed {i + 1}/{len(original_df)} rows, found {total_changes} changes so far")

            print(f"✅ Enhanced comparison finished: {total_changes} changes, {len(data_type_mismatches)} data type mismatches")

            # Log comprehensive enhanced change analysis
            log_to_file(f"""ENHANCED CELL-BY-CELL COMPARISON RESULTS:
=====================================
Total Changes Detected: {total_changes}
Data Type Mismatches: {len(data_type_mismatches)}
Affected Rows: {len(changed_rows)} out of {original_df.shape[0]} ({len(changed_rows)/original_df.shape[0]*100:.1f}%)
Affected Columns: {len(changed_columns)} out of {len(original_df.columns)} ({len(changed_columns)/len(original_df.columns)*100:.1f}%)

AFFECTED COLUMNS LIST:
{list(changed_columns)}

AFFECTED ROWS LIST:
{sorted(list(changed_rows))[:50]}  # Show first 50 rows

DATA TYPE MISMATCHES:
{json.dumps(data_type_mismatches, indent=2) if data_type_mismatches else "None detected"}

ALL DETECTED CHANGES ({len(changed_cells)} total):
""", "ENHANCED COMPREHENSIVE CHANGE DETECTION RESULTS")

            # Log ALL changes with enhanced details
            for i, change in enumerate(changed_cells[:100]):  # Show first 100 changes
                change_type = change.get('change_type', 'unknown')
                data_type_info = f" [DType: {change.get('original_dtype', 'unknown')} → {change.get('modified_dtype', 'unknown')}]" if change.get('data_type_changed') else ""
                log_to_file(f"Change {i+1}: Row {change['row']}, Column '{change['column']}' ({change_type}){data_type_info}: '{change['original']}' → '{change['modified']}'")

        else:
            log_to_file(f"""CANNOT PERFORM CELL-BY-CELL COMPARISON:
Reason: Shape or column structure differs
Original shape: {original_df.shape}
Modified shape: {modified_df.shape}
Original columns: {list(original_df.columns)}
Modified columns: {list(modified_df.columns)}
""", "CELL-BY-CELL COMPARISON SKIPPED")

        # Enhanced change statistics
        total_cells = original_df.shape[0] * original_df.shape[1] if original_df.size > 0 else 1
        change_density = total_changes / total_cells

        log_to_file(f"""ENHANCED COMPREHENSIVE STATISTICAL ANALYSIS:
===============================
Total Cells in Original: {total_cells}
Total Cells Changed: {total_changes}
Change Density: {change_density*100:.4f}%
Data Type Mismatches: {len(data_type_mismatches)}
Percentage of Rows Affected: {len(changed_rows)/original_df.shape[0]*100:.2f}% ({len(changed_rows)}/{original_df.shape[0]})
Percentage of Columns Affected: {len(changed_columns)/len(original_df.columns)*100:.2f}% ({len(changed_columns)}/{len(original_df.columns)})

ENHANCED CHANGE PATTERN ANALYSIS:
- NaN Changes: {len([c for c in changed_cells if c.get('change_type') == 'nan_change'])}
- Value Changes: {len([c for c in changed_cells if c.get('change_type') == 'value_change'])}
- Data Type Changes: {len([c for c in changed_cells if c.get('data_type_changed')])}
- Formatting Changes: {len([c for c in changed_cells if c.get('change_type') == 'formatting_change'])}
- Most Affected Columns: {sorted(changed_columns)[:10]}
- Row Change Distribution: Every {original_df.shape[0]//max(1,len(changed_rows)):.0f} rows on average

SCHEMA COMPATIBILITY ANALYSIS:
{json.dumps(schema_comparison, indent=2)}
""", "ENHANCED COMPREHENSIVE STATISTICAL ANALYSIS")

        # STEP 1: Enhanced GPT-4.1 prompt generation with schema awareness
        print("🧠 Enhanced GPT-4.1: Analyzing changes with schema validation...")
        log_to_file("🧠 STARTING ENHANCED GPT-4.1 ANALYSIS WITH SCHEMA VALIDATION...", "ENHANCED GPT-4.1 PROMPT GENERATION START")

        claude_prompt = _generate_enhanced_claude_prompt_with_gpt4_logged(
            client, original_info, modified_info, user_description,
            total_changes, changed_rows, changed_columns, changed_cells,
            schema_comparison, data_type_mismatches, log_to_file
        )

        # STEP 2: Enhanced Claude prompt testing with learning system
        print("🤖 Enhanced Claude testing with learning system...")
        log_to_file("🤖 STARTING ENHANCED CLAUDE REPLICATION WITH LEARNING SYSTEM...", "ENHANCED CLAUDE REPLICATION TESTING START")

        replication_results = _test_claude_prompt_replication_with_learning_logged(
            original_df, modified_df, claude_prompt, schema_comparison, max_attempts=3, log_to_file=log_to_file
        )

        # STEP 3: Enhanced final analysis
        log_to_file("📊 GENERATING ENHANCED FINAL ANALYSIS...", "ENHANCED FINAL ANALYSIS GENERATION START")

        final_analysis = _get_enhanced_final_analysis_with_prompts_logged(
            client, original_info, modified_info, user_description,
            total_changes, changed_rows, changed_columns, changed_cells,
            schema_comparison, data_type_mismatches, claude_prompt, replication_results, log_to_file
        )

        # Enhanced metadata
        final_analysis["raw_gpt_response"] = final_analysis.get("raw_gpt_response", "")
        final_analysis["complete_comparison_performed"] = True
        final_analysis["enhanced_features_applied"] = True
        final_analysis["log_file_path"] = log_file_path
        final_analysis["full_change_statistics"] = {
            "total_cells": total_cells,
            "total_changes": total_changes,
            "change_density": change_density,
            "data_type_mismatches": len(data_type_mismatches),
            "affected_rows": list(changed_rows),
            "affected_columns": list(changed_columns),
            "schema_changes": schema_comparison,
            "replication_tested": True,
            "replication_success": replication_results["final_success"],
            "replication_attempts": len(replication_results["attempts"]),
            "learning_applied": True,
            "all_changes": changed_cells
        }

        # FIXED: Proper string formatting in final log
        best_attempt_score = replication_results.get("best_attempt", {}).get("match_score", 0)
        best_score_formatted = f"{best_attempt_score:.2%}" if best_attempt_score > 0 else "N/A"

        log_to_file(f"""ENHANCED ANALYSIS EXECUTION COMPLETED SUCCESSFULLY
==========================================
Total Execution Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Log File Saved: {log_file_path}
Log File Size: {os.path.getsize(log_file_path)} bytes

ENHANCED FINAL RESULTS SUMMARY:
- Replication Success: {replication_results["final_success"]}
- Best Match Score: {best_score_formatted}
- Total Replication Attempts: {len(replication_results["attempts"])}
- Changes Detected: {total_changes}
- Data Type Mismatches: {len(data_type_mismatches)}
- Change Density: {change_density*100:.4f}%
- Learning System Applied: ✅
- Schema Validation Applied: ✅

ENHANCEMENTS SUCCESSFULLY APPLIED:
1. ✅ Fixed logging string formatting bug
2. ✅ Enhanced data type/formatting precision matching
3. ✅ Implemented attempt-to-attempt learning system
4. ✅ Added comprehensive data type validation

COMPLETE ENHANCED FINAL ANALYSIS OBJECT:
{json.dumps(final_analysis, indent=2, default=str)}

✅ ENHANCED ANALYSIS COMPLETE - ALL LOGS SAVED TO: {log_file_path}
""", "ENHANCED FINAL EXECUTION RESULTS")

        print(f"📝 Enhanced analysis with all fixes applied - logs saved to: {log_file_path}")
        return final_analysis

    except Exception as e:
        # FIXED: Proper error logging without string formatting issues
        error_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        error_type = type(e).__name__
        error_message = str(e)
        stack_trace = traceback.format_exc()

        original_shape = original_df.shape if 'original_df' in locals() else 'Unknown'
        modified_shape = modified_df.shape if 'modified_df' in locals() else 'Unknown'

        error_details = f"""CRITICAL ERROR DURING ENHANCED ANALYSIS EXECUTION
========================================
Error Time: {error_time}
Error Type: {error_type}
Error Message: {error_message}

FULL STACK TRACE:
{stack_trace}

EXECUTION CONTEXT:
- User Description: "{user_description}"
- Original DF Shape: {original_shape}
- Modified DF Shape: {modified_shape}
- Log File: {log_file_path}

❌ ENHANCED ANALYSIS FAILED - ERROR LOGGED TO: {log_file_path}
"""

        log_to_file(error_details, "CRITICAL ERROR")

        error_msg = f"Error in enhanced GPT-4 dataframe analysis: {e}"
        logger.error(error_msg)
        print(f"❌ Enhanced analysis failed but logs saved to: {log_file_path}")

        return {
            "change_summary": f"Enhanced dataframe analysis failed: {user_description}",
            "change_type": "data_edit",
            "session_description": f"User made changes to entire dataframe. Description: {user_description}. Error in enhanced analysis: {error_message}",
            "error": error_message,
            "enhanced_features_applied": False,
            "complete_comparison_performed": False,
            "log_file_path": log_file_path,
            "error_logged": True
        }


# ENHANCEMENT 1: Data Type Schema Analysis Functions
def _analyze_dataframe_schema(df, label=""):
    """
    Analyze dataframe schema with detailed data type information
    """
    schema_analysis = {
        "label": label,
        "shape": df.shape,
        "column_count": len(df.columns),
        "row_count": len(df),
        "columns": {}
    }

    for col in df.columns:
        col_data = df[col]
        schema_analysis["columns"][col] = {
            "dtype": str(col_data.dtype),
            "python_type": str(type(col_data.dtype)),
            "null_count": int(col_data.isnull().sum()),
            "null_percentage": float(col_data.isnull().sum() / len(df) * 100) if len(df) > 0 else 0,
            "unique_count": int(col_data.nunique()),
            "memory_usage": int(col_data.memory_usage(deep=True)),
            "sample_values": [str(val) for val in col_data.dropna().head(3).tolist()],
            "is_numeric": pd.api.types.is_numeric_dtype(col_data),
            "is_datetime": pd.api.types.is_datetime64_any_dtype(col_data),
            "is_categorical": pd.api.types.is_categorical_dtype(col_data),
            "is_object": pd.api.types.is_object_dtype(col_data)
        }

    return schema_analysis


def _compare_dataframe_schemas(original_df, modified_df):
    """
    Compare schemas between original and modified dataframes
    """
    orig_schema = _analyze_dataframe_schema(original_df, "ORIGINAL")
    mod_schema = _analyze_dataframe_schema(modified_df, "MODIFIED")

    comparison = {
        "shape_changed": orig_schema["shape"] != mod_schema["shape"],
        "column_count_changed": orig_schema["column_count"] != mod_schema["column_count"],
        "row_count_changed": orig_schema["row_count"] != mod_schema["row_count"],
        "column_changes": {},
        "schema_compatibility_score": 0.0,
        "critical_issues": []
    }

    # Analyze column-by-column changes
    all_columns = set(original_df.columns) | set(modified_df.columns)
    compatible_columns = 0

    for col in all_columns:
        if col in original_df.columns and col in modified_df.columns:
            orig_col = orig_schema["columns"][col]
            mod_col = mod_schema["columns"][col]

            column_change = {
                "exists_in_both": True,
                "dtype_changed": orig_col["dtype"] != mod_col["dtype"],
                "original_dtype": orig_col["dtype"],
                "modified_dtype": mod_col["dtype"],
                "null_count_changed": orig_col["null_count"] != mod_col["null_count"],
                "type_compatibility": _check_type_compatibility(orig_col["dtype"], mod_col["dtype"])
            }

            if column_change["type_compatibility"]:
                compatible_columns += 1
            else:
                comparison["critical_issues"].append(f"Column '{col}': Incompatible type change {orig_col['dtype']} → {mod_col['dtype']}")

        elif col in original_df.columns:
            column_change = {
                "exists_in_both": False,
                "removed": True,
                "original_dtype": orig_schema["columns"][col]["dtype"]
            }
            comparison["critical_issues"].append(f"Column '{col}' was removed")

        else:
            column_change = {
                "exists_in_both": False,
                "added": True,
                "modified_dtype": mod_schema["columns"][col]["dtype"]
            }
            comparison["critical_issues"].append(f"Column '{col}' was added")

        comparison["column_changes"][col] = column_change

    # Calculate compatibility score
    total_columns = len(all_columns)
    comparison["schema_compatibility_score"] = compatible_columns / total_columns if total_columns > 0 else 1.0

    return comparison


def _check_type_compatibility(orig_dtype, mod_dtype):
    """
    Check if two data types are compatible for replication
    """
    # Exact match
    if str(orig_dtype) == str(mod_dtype):
        return True

    # Compatible numeric types
    numeric_types = ['int64', 'int32', 'float64', 'float32', 'number']
    if any(t in str(orig_dtype).lower() for t in numeric_types) and any(t in str(mod_dtype).lower() for t in numeric_types):
        return True

    # Compatible string/object types
    string_types = ['object', 'string', 'str']
    if any(t in str(orig_dtype).lower() for t in string_types) and any(t in str(mod_dtype).lower() for t in string_types):
        return True

    # Compatible datetime types
    datetime_types = ['datetime', 'timestamp']
    if any(t in str(orig_dtype).lower() for t in datetime_types) and any(t in str(mod_dtype).lower() for t in datetime_types):
        return True

    return False


# ENHANCEMENT 2: Enhanced Cell Comparison with Data Type Awareness
def _enhanced_cell_comparison(orig_val, mod_val, row_idx, col_name, orig_dtype, mod_dtype):
    """
    Enhanced cell comparison with data type awareness and precision handling
    """
    change_detail = {
        "row": row_idx,
        "column": col_name,
        "original": str(orig_val),
        "modified": str(mod_val),
        "original_dtype": str(orig_dtype),
        "modified_dtype": str(mod_dtype),
        "data_type_changed": str(orig_dtype) != str(mod_dtype),
        "change_type": "no_change"
    }

    # Handle NaN comparisons
    if pd.isna(orig_val) and pd.isna(mod_val):
        return False, change_detail
    elif pd.isna(orig_val) or pd.isna(mod_val):
        change_detail["change_type"] = "nan_change"
        return True, change_detail

    # Data type change detection
    if str(orig_dtype) != str(mod_dtype):
        change_detail["change_type"] = "data_type_change"
        # Still check if values are equivalent despite type change
        if _are_values_equivalent(orig_val, mod_val):
            change_detail["change_type"] = "data_type_change_equivalent_value"
            return True, change_detail
        else:
            return True, change_detail

    # Enhanced value comparison with precision handling
    if _are_values_equivalent(orig_val, mod_val):
        return False, change_detail

    # Check for formatting differences
    if _are_formatting_differences_only(orig_val, mod_val):
        change_detail["change_type"] = "formatting_change"
        return True, change_detail

    # Significant value change
    change_detail["change_type"] = "value_change"
    return True, change_detail


def _are_values_equivalent(val1, val2):
    """
    Check if two values are equivalent despite potential formatting differences
    """
    # Exact string match after stripping
    if str(val1).strip() == str(val2).strip():
        return True

    # Numeric equivalence check
    try:
        # Handle currency and comma formatting
        clean_val1 = str(val1).replace(',', '').replace('$', '').strip()
        clean_val2 = str(val2).replace(',', '').replace('$', '').strip()

        num1 = float(clean_val1)
        num2 = float(clean_val2)

        # Allow small floating point differences
        return abs(num1 - num2) < 1e-10

    except (ValueError, TypeError):
        pass

    # Date equivalence check
    try:
        import pandas as pd
        date1 = pd.to_datetime(val1)
        date2 = pd.to_datetime(val2)
        return date1 == date2
    except:
        pass

    return False


def _are_formatting_differences_only(val1, val2):
    """
    Check if differences are only formatting (whitespace, case, etc.)
    """
    try:
        # Case-insensitive comparison
        if str(val1).strip().lower() == str(val2).strip().lower():
            return True

        # Numeric formatting differences
        clean_val1 = str(val1).replace(',', '').replace('$', '').replace(' ', '').strip()
        clean_val2 = str(val2).replace(',', '').replace('$', '').replace(' ', '').strip()

        if clean_val1 == clean_val2:
            return True

    except:
        pass

    return False


# ENHANCEMENT 3: Enhanced GPT-4.1 Prompt Generation with Schema Awareness
def _generate_enhanced_claude_prompt_with_gpt4_logged(client, original_info, modified_info, user_description,
                                                     total_changes, changed_rows, changed_columns, changed_cells,
                                                     schema_comparison, data_type_mismatches, log_to_file):
    """
    Enhanced GPT-4.1 prompt generation with comprehensive schema awareness and data type focus
    """
    # Prepare detailed schema information for GPT-4.1
    schema_issues = schema_comparison.get("critical_issues", [])
    compatibility_score = schema_comparison.get("schema_compatibility_score", 0)

    sample_changes = changed_cells[:15] if changed_cells else []
    sample_type_mismatches = data_type_mismatches[:5] if data_type_mismatches else []

    enhanced_prompt_request = f"""
    You are an expert at analyzing dataframe changes and generating PRECISE prompts for Claude 3.7 to replicate manual edits with EXACT data type and formatting precision.

    CRITICAL REQUIREMENTS:
    1. Focus on DATA TYPE PRECISION - ensure exact dtype matching
    2. Handle FORMATTING with precision (decimals, currency, dates)
    3. Provide SPECIFIC transformation logic, not general filtering
    4. Include EXPLICIT data type conversion instructions
    5. Address schema compatibility issues

    ORIGINAL DATAFRAME:
    Shape: {original_info['shape']}
    Columns: {original_info['columns']}
    Data Types: {original_info['dtype_details']}

    Data Sample:
    {original_info['full_data'][:3000]}

    MODIFIED DATAFRAME:
    Shape: {modified_info['shape']}
    Columns: {modified_info['columns']}
    Data Types: {modified_info['dtype_details']}

    Data Sample:
    {modified_info['full_data'][:3000]}

    ENHANCED CHANGE ANALYSIS:
    - Total changes: {total_changes}
    - Schema compatibility score: {compatibility_score:.2f}
    - Data type mismatches: {len(data_type_mismatches)}
    - Critical schema issues: {schema_issues}
    - Changed rows: {list(changed_rows)[:20] if changed_rows else []}
    - Changed columns: {list(changed_columns)}

    SAMPLE DETAILED CHANGES:
    {json.dumps(sample_changes, indent=2)}

    DATA TYPE MISMATCHES DETECTED:
    {json.dumps(sample_type_mismatches, indent=2)}

    USER DESCRIPTION: "{user_description}"

    Generate a PRECISE prompt for Claude 3.7 that will replicate these exact changes with:

    1. EXACT DATA TYPE SPECIFICATIONS:
       - Include explicit dtype conversion commands
       - Specify decimal precision for floats
       - Handle string formatting requirements

    2. SPECIFIC TRANSFORMATION LOGIC:
       - Exact cell selection criteria
       - Precise value transformation formulas
       - Row-by-row or column-by-column instructions if needed

    3. SCHEMA VALIDATION:
       - Ensure output matches target schema exactly
       - Include data type verification steps

    4. BUSINESS CONTEXT for rent roll data:
       - Consider tenant information updates
       - Handle rent calculations precisely
       - Manage occupancy status changes

    The prompt should be executable Python pandas code that produces the EXACT modified dataframe.

    Return ONLY the Claude prompt text that focuses on precision and data type accuracy.
    """

    # Log the enhanced GPT-4.1 prompt
    log_to_file(f"""ENHANCED GPT-4.1 PROMPT TO GENERATE CLAUDE INSTRUCTIONS:
Model: gpt-4.1
Temperature: 0.05 (Lower for precision)
Max Tokens: 4000

ENHANCED PROMPT FEATURES:
- Schema compatibility analysis included
- Data type mismatch focus
- Formatting precision requirements
- Sample change details provided

COMPLETE ENHANCED PROMPT SENT TO GPT-4.1:
{'-'*60}
{enhanced_prompt_request}
{'-'*60}
""", "ENHANCED GPT-4.1 REQUEST")

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You are an expert at generating precise data transformation prompts with exact data type and formatting specifications. Focus on precision and schema accuracy."},
                {"role": "user", "content": enhanced_prompt_request}
            ],
            max_tokens=4000,
            temperature=0.05  # Lower temperature for more precision
        )

        enhanced_gpt_response = response.choices[0].message.content.strip()

        # Log enhanced GPT-4.1 response
        log_to_file(f"""ENHANCED GPT-4.1 RESPONSE (CLAUDE PROMPT):
Response Length: {len(enhanced_gpt_response)} characters
Tokens Used: Approximately {len(enhanced_gpt_response.split())} words
Schema Focus: ✅ Applied
Data Type Precision: ✅ Applied

GENERATED ENHANCED CLAUDE PROMPT:
{'-'*60}
{enhanced_gpt_response}
{'-'*60}
""", "ENHANCED GPT-4.1 RESPONSE")

        return enhanced_gpt_response

    except Exception as e:
        error_msg = f"Enhanced GPT-4.1 API Error: {str(e)}"
        log_to_file(f"""ENHANCED GPT-4.1 API CALL FAILED:
Error: {error_msg}
Fallback: Using enhanced basic prompt with schema awareness
""")

        # Enhanced fallback prompt with schema awareness
        fallback_prompt = f"""Replicate the manual edits described as: {user_description}

CRITICAL REQUIREMENTS:
- Maintain exact data types: {modified_info['dtype_details']}
- Total changes to make: {total_changes}
- Focus on columns: {list(changed_columns)}
- Ensure precise formatting and data type conversion
- Output must match target schema exactly

Sample changes for reference:
{json.dumps(sample_changes[:5], indent=2)}
"""
        return fallback_prompt


# ENHANCEMENT 4: Claude Testing with Learning System
def _test_claude_prompt_replication_with_learning_logged(original_df, target_df, claude_prompt, schema_comparison, max_attempts=3, log_to_file=None):
    """
    Enhanced Claude testing with attempt-to-attempt learning system
    """
    attempts = []
    current_prompt = claude_prompt
    learning_history = []

    log_to_file(f"""ENHANCED CLAUDE REPLICATION TESTING WITH LEARNING STARTED:
Maximum Attempts: {max_attempts}
Target DataFrame Shape: {target_df.shape}
Original DataFrame Shape: {original_df.shape}
Schema Compatibility Score: {schema_comparison.get('schema_compatibility_score', 0):.2f}
Critical Schema Issues: {len(schema_comparison.get('critical_issues', []))}

LEARNING SYSTEM FEATURES:
✅ Attempt-to-attempt feedback loop
✅ Schema validation focus
✅ Data type precision tracking
✅ Progressive prompt refinement

INITIAL ENHANCED CLAUDE PROMPT TO TEST:
{'-'*60}
{claude_prompt}
{'-'*60}
""", "ENHANCED REPLICATION TESTING INITIALIZATION")

    for attempt in range(max_attempts):
        print(f"🔄 Enhanced Attempt {attempt + 1}/{max_attempts} with learning system")
        log_to_file(f"Starting enhanced attempt {attempt + 1}/{max_attempts} with accumulated learning...", f"ENHANCED ATTEMPT {attempt + 1}")

        try:
            # Apply learning from previous attempts
            if attempt > 0:
                current_prompt = _apply_learning_to_prompt(current_prompt, learning_history, target_df, log_to_file)

            # Call Claude with enhanced logging
            claude_response = _call_enhanced_claude_logged(current_prompt, original_df, target_df, schema_comparison, log_to_file, attempt + 1)

            # Enhanced code extraction
            code_blocks = _extract_enhanced_code_blocks(claude_response)

            log_to_file(f"""ENHANCED CODE EXTRACTION RESULTS:
Found {len(code_blocks)} code blocks
Enhanced Features: Schema validation, data type checking

Code Blocks Extracted:
""")

            for i, code in enumerate(code_blocks):
                log_to_file(f"Enhanced Code Block {i+1}:\n```python\n{code}\n```\n")

            if not code_blocks:
                attempt_result = {
                    "attempt": attempt + 1,
                    "success": False,
                    "error": "No code found in Claude response",
                    "prompt_used": current_prompt,
                    "learning_applied": len(learning_history) > 0
                }
                attempts.append(attempt_result)
                learning_history.append({
                    "attempt": attempt + 1,
                    "issue": "no_code_generated",
                    "resolution": "request_explicit_code_blocks"
                })
                log_to_file("No code blocks found - adding to learning history")
                continue

            # Enhanced code execution with schema validation
            test_df = original_df.copy()
            exec_globals = {
                "df": test_df,
                "pd": pd,
                "np": np,
                "os": os,
                "datetime": datetime
            }

            log_to_file("Starting enhanced code execution with schema validation...")
            execution_output = ""
            execution_success = True

            for i, code in enumerate(code_blocks):
                try:
                    log_to_file(f"Executing enhanced code block {i+1}...")
                    exec(code, exec_globals)
                    execution_output += f"Enhanced code block {i+1} executed successfully\n"
                except Exception as exec_error:
                    execution_output += f"Enhanced code block {i+1} failed: {str(exec_error)}\n"
                    log_to_file(f"Enhanced code block {i+1} execution error: {str(exec_error)}")
                    execution_success = False
                    learning_history.append({
                        "attempt": attempt + 1,
                        "issue": f"execution_error_{i+1}",
                        "error": str(exec_error),
                        "resolution": "fix_syntax_and_logic"
                    })

            result_df = exec_globals["df"]
            log_to_file(f"Enhanced execution completed. Result DataFrame shape: {result_df.shape}")

            # Enhanced validation with schema checking
            enhanced_validation = _enhanced_result_validation(target_df, result_df, schema_comparison)
            match_score = enhanced_validation["overall_score"]
            schema_match = enhanced_validation["schema_match"]
            data_type_match = enhanced_validation["data_type_match"]

            attempt_result = {
                "attempt": attempt + 1,
                "success": match_score >= 0.95 and schema_match,
                "match_score": match_score,
                "schema_match": schema_match,
                "data_type_match": data_type_match,
                "enhanced_validation": enhanced_validation,
                "generated_code": code_blocks,
                "prompt_used": current_prompt,
                "result_shape": result_df.shape,
                "target_shape": target_df.shape,
                "execution_output": execution_output,
                "execution_success": execution_success,
                "learning_applied": len(learning_history) > 0
            }

            attempts.append(attempt_result)

            log_to_file(f"""ENHANCED ATTEMPT {attempt + 1} RESULTS:
Overall Match Score: {match_score:.2%}
Schema Match: {'✅ PASSED' if schema_match else '❌ FAILED'}
Data Type Match: {'✅ PASSED' if data_type_match else '❌ FAILED'}
Success Threshold (95% + Schema): {'✅ PASSED' if (match_score >= 0.95 and schema_match) else '❌ FAILED'}
Result Shape: {result_df.shape}
Target Shape: {target_df.shape}
Execution Success: {'✅ YES' if execution_success else '❌ NO'}
Learning Applied: {'✅ YES' if len(learning_history) > 0 else '❌ NO'}

ENHANCED VALIDATION DETAILS:
{json.dumps(enhanced_validation, indent=2)}

Execution Output:
{execution_output}
""")

            print(f"📊 Enhanced Match Score: {match_score:.2%}, Schema: {'✅' if schema_match else '❌'}")

            # Add learning insights for next attempt
            if match_score < 0.95 or not schema_match:
                learning_insights = _analyze_failure_for_learning(target_df, result_df, enhanced_validation)
                learning_history.extend(learning_insights)
                log_to_file(f"Learning insights added: {json.dumps(learning_insights, indent=2)}")

            if match_score >= 0.95 and schema_match:
                print("✅ Enhanced replication successful!")
                log_to_file("🎉 ENHANCED REPLICATION SUCCESSFUL! Stopping attempts.")
                break

        except Exception as e:
            error_msg = str(e)
            attempt_result = {
                "attempt": attempt + 1,
                "success": False,
                "error": error_msg,
                "prompt_used": current_prompt,
                "learning_applied": len(learning_history) > 0
            }
            attempts.append(attempt_result)
            learning_history.append({
                "attempt": attempt + 1,
                "issue": "execution_exception",
                "error": error_msg,
                "resolution": "improve_error_handling"
            })
            log_to_file(f"ENHANCED ATTEMPT {attempt + 1} FAILED with error: {error_msg}")
            print(f"❌ Enhanced Error: {error_msg}")

    final_success = any(attempt["success"] for attempt in attempts)
    best_attempt = max(attempts, key=lambda x: x.get("match_score", 0)) if attempts else None

    log_to_file(f"""ENHANCED REPLICATION TESTING COMPLETE:
Final Success: {final_success}
Best Match Score: {best_attempt.get('match_score', 0):.2% if best_attempt else 'N/A'}
Best Schema Match: {best_attempt.get('schema_match', False) if best_attempt else False}
Total Attempts: {len(attempts)}
Total Learning Insights: {len(learning_history)}

LEARNING HISTORY SUMMARY:
{json.dumps(learning_history, indent=2)}

ALL ENHANCED ATTEMPTS SUMMARY:
""", "ENHANCED REPLICATION TESTING RESULTS")

    for i, attempt in enumerate(attempts, 1):
        success = attempt.get('success', False)
        score = attempt.get('match_score', 0)
        schema_match = attempt.get('schema_match', False)
        learning = attempt.get('learning_applied', False)
        error = attempt.get('error', 'None')

        log_to_file(f"Enhanced Attempt {i}: Success={success}, Score={score:.2%}, Schema={schema_match}, Learning={learning}, Error={error}")

    return {
        "attempts": attempts,
        "final_success": final_success,
        "best_attempt": best_attempt,
        "final_prompt": best_attempt["prompt_used"] if best_attempt else claude_prompt,
        "learning_history": learning_history,
        "learning_applied": len(learning_history) > 0
    }


# Learning System Helper Functions
def _apply_learning_to_prompt(base_prompt, learning_history, target_df, log_to_file):
    """
    Apply accumulated learning to improve the prompt
    """
    if not learning_history:
        return base_prompt

    # Analyze learning patterns
    common_issues = {}
    for learning in learning_history:
        issue = learning.get("issue", "unknown")
        if issue in common_issues:
            common_issues[issue] += 1
        else:
            common_issues[issue] = 1

    # Generate improvement suggestions based on learning
    improvements = []

    if "execution_error" in [l.get("issue", "") for l in learning_history]:
        improvements.append("Focus on syntactically correct pandas operations")

    if "schema_mismatch" in [l.get("issue", "") for l in learning_history]:
        improvements.append(f"Ensure exact data types: {dict(target_df.dtypes)}")

    if "data_type_mismatch" in [l.get("issue", "") for l in learning_history]:
        improvements.append("Include explicit dtype conversions using .astype()")

    if "no_code_generated" in [l.get("issue", "") for l in learning_history]:
        improvements.append("Generate code in ```python ``` blocks")

    # Apply improvements to prompt
    if improvements:
        learning_section = f"""

LEARNING FROM PREVIOUS ATTEMPTS:
Based on previous attempts, please pay special attention to:
{chr(10).join([f"• {improvement}" for improvement in improvements])}

COMMON ISSUES TO AVOID:
{chr(10).join([f"• {issue}: occurred {count} time(s)" for issue, count in common_issues.items()])}
"""
        improved_prompt = base_prompt + learning_section

        log_to_file(f"""LEARNING APPLIED TO PROMPT:
Improvements Added: {len(improvements)}
Common Issues Addressed: {len(common_issues)}

LEARNING SECTION ADDED:
{learning_section}
""")

        return improved_prompt

    return base_prompt


def _analyze_failure_for_learning(target_df, result_df, enhanced_validation):
    """
    Analyze failure to extract learning insights for next attempt
    """
    insights = []

    # Schema issues
    if not enhanced_validation.get("schema_match", True):
        if enhanced_validation.get("shape_mismatch", False):
            insights.append({
                "issue": "shape_mismatch",
                "details": f"Target: {target_df.shape}, Got: {result_df.shape}",
                "resolution": "check_row_filtering_and_column_selection"
            })

        if enhanced_validation.get("column_mismatch", False):
            insights.append({
                "issue": "column_mismatch",
                "details": f"Target cols: {list(target_df.columns)}, Got cols: {list(result_df.columns)}",
                "resolution": "ensure_exact_column_names_and_order"
            })

    # Data type issues
    if not enhanced_validation.get("data_type_match", True):
        type_mismatches = enhanced_validation.get("data_type_details", {})
        for col, details in type_mismatches.items():
            if details.get("mismatch", False):
                insights.append({
                    "issue": "data_type_mismatch",
                    "column": col,
                    "expected": details.get("target_dtype"),
                    "got": details.get("result_dtype"),
                    "resolution": f"add_explicit_conversion: df['{col}'] = df['{col}'].astype('{details.get('target_dtype')}')"
                })

    # Content issues
    content_score = enhanced_validation.get("content_score", 1.0)
    if content_score < 0.9:
        insights.append({
            "issue": "content_mismatch",
            "score": content_score,
            "resolution": "review_transformation_logic_for_cell_values"
        })

    return insights


# Enhanced Result Validation
def _enhanced_result_validation(target_df, result_df, schema_comparison):
    """
    Enhanced validation with comprehensive schema and data type checking
    """
    validation = {
        "overall_score": 0.0,
        "schema_match": False,
        "data_type_match": False,
        "content_score": 0.0,
        "shape_mismatch": False,
        "column_mismatch": False,
        "data_type_details": {}
    }

    # Shape validation
    if target_df.shape != result_df.shape:
        validation["shape_mismatch"] = True
        validation["overall_score"] = 0.0
        return validation

    # Column validation
    if list(target_df.columns) != list(result_df.columns):
        validation["column_mismatch"] = True
        validation["overall_score"] = 0.1
        return validation

    validation["schema_match"] = True

    # Data type validation
    type_match_count = 0
    total_columns = len(target_df.columns)

    for col in target_df.columns:
        target_dtype = str(target_df[col].dtype)
        result_dtype = str(result_df[col].dtype)

        type_compatible = _check_type_compatibility(target_dtype, result_dtype)

        validation["data_type_details"][col] = {
            "target_dtype": target_dtype,
            "result_dtype": result_dtype,
            "exact_match": target_dtype == result_dtype,
            "compatible": type_compatible,
            "mismatch": not type_compatible
        }

        if type_compatible:
            type_match_count += 1

    validation["data_type_match"] = type_match_count == total_columns

    # Content validation using the enhanced comparison
    if validation["schema_match"] and len(target_df) > 0:
        validation["content_score"] = _calculate_enhanced_match_score(target_df, result_df)
    else:
        validation["content_score"] = 0.0

    # Overall score calculation
    schema_weight = 0.3
    dtype_weight = 0.3
    content_weight = 0.4

    schema_score = 1.0 if validation["schema_match"] else 0.0
    dtype_score = type_match_count / total_columns if total_columns > 0 else 1.0

    validation["overall_score"] = (
        schema_weight * schema_score +
        dtype_weight * dtype_score +
        content_weight * validation["content_score"]
    )

    return validation


def _calculate_enhanced_match_score(target_df, result_df):
    """
    Enhanced match score calculation with improved precision
    """
    if target_df.empty and result_df.empty:
        return 1.0
    if target_df.empty or result_df.empty:
        return 0.0
    if target_df.shape != result_df.shape:
        return 0.0
    if list(target_df.columns) != list(result_df.columns):
        return 0.0

    total_cells = 0
    matching_cells = 0

    for i in range(len(target_df)):
        for col in target_df.columns:
            total_cells += 1

            try:
                target_val = target_df.iloc[i][col]
                result_val = result_df.iloc[i][col]

                if _are_values_equivalent(target_val, result_val):
                    matching_cells += 1

            except (IndexError, KeyError):
                continue

    return matching_cells / total_cells if total_cells > 0 else 0.0


# Enhanced Claude API Call
def _call_enhanced_claude_logged(prompt, original_df, target_df, schema_comparison, log_to_file, attempt_number):
    """
    Enhanced Claude API call with schema context and improved prompting
    """
    target_schema_info = {
        "shape": target_df.shape,
        "dtypes": dict(target_df.dtypes),
        "columns": list(target_df.columns),
        "sample_data": target_df.head(3).to_string()
    }

    log_to_file(f"""CALLING ENHANCED CLAUDE API - ATTEMPT {attempt_number}:
Model: claude-sonnet-4-20250514
Temperature: 0.1 (Optimized for precision)
Max Tokens: 3000

ENHANCED FEATURES:
✅ Target schema context provided
✅ Data type precision focus
✅ Schema compatibility warnings included

PROMPT BEING SENT TO CLAUDE:
{'-'*60}
{prompt}
{'-'*60}

ENHANCED CONTEXT PROVIDED:
Original DataFrame Shape: {original_df.shape}
Target DataFrame Shape: {target_df.shape}
Schema Compatibility Score: {schema_comparison.get('schema_compatibility_score', 0):.2f}
Critical Issues: {len(schema_comparison.get('critical_issues', []))}

TARGET SCHEMA REQUIREMENTS:
{json.dumps(target_schema_info, indent=2, default=str)}
""", f"ENHANCED CLAUDE API CALL {attempt_number}")

    try:
        # Get Anthropic client
        anthropic_client = Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

        # Enhanced dataframe context with target schema
        enhanced_df_summary = f"""
        CURRENT DATAFRAME CONTENT:
        {original_df.to_string(max_rows=50)}

        CURRENT DATAFRAME STATISTICS:
        - Shape: {original_df.shape}
        - Columns: {list(original_df.columns)}
        - Data types: {dict(original_df.dtypes)}
        - Null values per column: {dict(original_df.isnull().sum())}

        TARGET SCHEMA REQUIREMENTS (CRITICAL):
        - Target Shape: {target_df.shape}
        - Target Columns: {list(target_df.columns)}
        - Target Data Types: {dict(target_df.dtypes)}
        - Schema Compatibility Issues: {schema_comparison.get('critical_issues', [])}

        SAMPLE TARGET DATA:
        {target_df.head(3).to_string()}
        """

        # Enhanced system prompt
        enhanced_claude_system_prompt = """You are an expert Python data analyst with specialization in precise dataframe transformations.
        You will receive a dataframe that is already loaded as 'df' and specific transformation requirements.

        CRITICAL PRECISION REQUIREMENTS:
        1. The dataframe 'df' is already loaded - do not load or import it
        2. Generate code that produces EXACTLY the target schema and data types
        3. Wrap all code in ```python and ``` blocks
        4. Include explicit data type conversions using .astype() when needed
        5. Focus on precision over filtering - make exact transformations
        6. Validate your output matches the target schema exactly
        7. Handle null values, formatting, and data types with precision

        SCHEMA MATCHING PRIORITY:
        - Exact column names and order
        - Exact data types for each column
        - Exact row count and content
        - Proper handling of null values and edge cases"""

        # Enhanced messages for Claude
        enhanced_claude_messages = [
            {
                "role": "user",
                "content": f"Here is the enhanced context:\n\n{enhanced_df_summary}\n\nTASK WITH PRECISION REQUIREMENTS: {prompt}\n\nGenerate precise Python code to accomplish this exact transformation with schema validation."
            }
        ]

        # Call Claude
        claude_response = anthropic_client.messages.create(
            model="claude-3-7-sonnet-20250219",
            system=enhanced_claude_system_prompt,
            messages=enhanced_claude_messages,
            max_tokens=3000,
            temperature=0.1  # Lower temperature for more precision
        )

        # Extract response text
        response_text = claude_response.content[0].text

        log_to_file(f"""ENHANCED CLAUDE API RESPONSE - ATTEMPT {attempt_number}:
Response Length: {len(response_text)} characters
Enhanced Features Applied: ✅
Response Received Successfully: ✅

FULL ENHANCED CLAUDE RESPONSE:
{'-'*60}
{response_text}
{'-'*60}
""")

        return response_text

    except Exception as e:
        error_msg = f"Enhanced Claude API Error: {str(e)}"
        log_to_file(f"""ENHANCED CLAUDE API CALL FAILED - ATTEMPT {attempt_number}:
Error: {error_msg}
Using enhanced fallback response
""")

        # Enhanced fallback response
        enhanced_fallback_response = f"""
        I'll help you with this enhanced transformation task. Here's the precise code:

        ```python
        # Enhanced error calling Claude API: {str(e)}
        # Enhanced fallback code with schema awareness
        print("Enhanced Claude API error, using schema-aware fallback")
        print(f"Current dataframe shape: {{df.shape}}")
        print(f"Target shape should be: {target_df.shape}")
        print(f"Target dtypes: {dict(target_df.dtypes)}")
        print("Please review transformation logic manually")

        # Basic schema validation
        if df.shape != {target_df.shape}:
            print("Warning: Shape mismatch detected")

        # Show current vs target schema
        print("\\nCurrent dtypes:", dict(df.dtypes))
        print("Target dtypes:", {dict(target_df.dtypes)})
        ```
        """

        log_to_file(f"Enhanced fallback response generated:\n{enhanced_fallback_response}")
        return enhanced_fallback_response


def _extract_enhanced_code_blocks(response_text):
    """
    Enhanced code block extraction with validation
    """
    import re

    # Enhanced pattern matching for code blocks
    patterns = [
        r'```python\s*(.*?)```',
        r'```\s*python\s*(.*?)```',
        r'```\s*(.*?)```',
        r'`([^`]+)`'  # Single backticks as fallback
    ]

    code_blocks = []

    for pattern in patterns:
        matches = re.findall(pattern, response_text, re.DOTALL | re.IGNORECASE)
        for match in matches:
            cleaned_code = match.strip()
            if cleaned_code and len(cleaned_code) > 10:  # Filter out very short matches
                code_blocks.append(cleaned_code)

    # Remove duplicates while preserving order
    seen = set()
    unique_blocks = []
    for block in code_blocks:
        if block not in seen:
            seen.add(block)
            unique_blocks.append(block)

    return unique_blocks


# Enhanced Final Analysis Generation
def _get_enhanced_final_analysis_with_prompts_logged(client, original_info, modified_info, user_description,
                                                   total_changes, changed_rows, changed_columns, changed_cells,
                                                   schema_comparison, data_type_mismatches, claude_prompt,
                                                   replication_results, log_to_file):
    """
    Enhanced final analysis with comprehensive learning and schema insights
    """
    best_attempt = replication_results.get("best_attempt", {})
    final_claude_prompt = replication_results.get("final_prompt", claude_prompt)
    learning_applied = replication_results.get("learning_applied", False)
    learning_history = replication_results.get("learning_history", [])

    success_status = "SUCCESS" if replication_results["final_success"] else "PARTIAL"

    # Enhanced statistics
    total_cells = original_info["shape"][0] * original_info["shape"][1] if original_info["shape"][0] > 0 else 1
    change_density = total_changes / total_cells

    enhanced_analysis_prompt = f"""
    Analyze this enhanced dataframe change with comprehensive schema and learning insights.

    ORIGINAL DATAFRAME:
    Shape: {original_info['shape']}
    Schema: {original_info['dtype_details']}
    Data Sample: {original_info['full_data'][:2000]}...

    MODIFIED DATAFRAME:
    Shape: {modified_info['shape']}
    Schema: {modified_info['dtype_details']}
    Data Sample: {modified_info['full_data'][:2000]}...

    ENHANCED CHANGE ANALYSIS:
    - Total changes: {total_changes} cells ({change_density*100:.1f}%)
    - Data type mismatches: {len(data_type_mismatches)}
    - Schema compatibility: {schema_comparison.get('schema_compatibility_score', 0):.1%}
    - Critical schema issues: {len(schema_comparison.get('critical_issues', []))}
    - User description: "{user_description}"

    REPLICATION TESTING RESULTS:
    - Status: {success_status}
    - Attempts: {len(replication_results['attempts'])}
    - Best match: {best_attempt.get('match_score', 0):.1%}
    - Schema match: {best_attempt.get('schema_match', False)}
    - Learning applied: {learning_applied}
    - Learning insights: {len(learning_history)}

    FINAL WORKING PROMPT: {final_claude_prompt}

    Provide comprehensive analysis in this exact JSON format:
    {{
        "change_summary": "ENHANCED ANALYSIS: [comprehensive summary including WORKING_CLAUDE_PROMPT: {final_claude_prompt}] and detailed technical findings with schema validation results",
        "change_type": "data_edit|structure_change|mixed",
        "structural_changes": {{
            "rows_added": {max(0, modified_info['shape'][0] - original_info['shape'][0])},
            "rows_removed": {max(0, original_info['shape'][0] - modified_info['shape'][0])},
            "columns_added": {list(set(modified_info['columns']) - set(original_info['columns']))},
            "columns_removed": {list(set(original_info['columns']) - set(modified_info['columns']))}
        }},
        "enhanced_data_modifications": {{
            "cells_changed": {total_changes},
            "total_cells": {total_cells},
            "change_percentage": {change_density*100:.2f},
            "data_type_mismatches": {len(data_type_mismatches)},
            "rows_affected": {len(changed_rows)},
            "columns_affected": {list(changed_columns)},
            "schema_compatibility_score": {schema_comparison.get('schema_compatibility_score', 0):.2f},
            "critical_schema_issues": {len(schema_comparison.get('critical_issues', []))},
            "precision_patterns": ["Enhanced patterns identified"],
            "data_quality_impact": "improved|degraded|neutral"
        }},
        "replication_analysis": {{
            "final_success": {replication_results['final_success']},
            "best_match_score": {best_attempt.get('match_score', 0):.2f},
            "schema_validation_passed": {best_attempt.get('schema_match', False)},
            "learning_system_applied": {learning_applied},
            "total_learning_insights": {len(learning_history)},
            "common_issues_resolved": ["Issues identified and resolved"]
        }},
        "business_impact": {{
            "rent_calculations_affected": "Enhanced analysis of rent impact with precision focus",
            "tenant_information_updated": "Enhanced analysis of tenant data changes",
            "occupancy_status_changed": "Enhanced analysis of occupancy changes",
            "data_integrity_maintained": "Assessment of data integrity after changes"
        }},
        "enhanced_recommendations": [
            "Enhanced recommendations based on learning insights",
            "Schema validation recommendations",
            "Data type precision improvements"
        ],
        "session_description": "ENHANCED_REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | LEARNING_APPLIED: {learning_applied} | REPLICATION_ATTEMPTS: {len(replication_results['attempts'])} | USER_EDIT: {user_description} | Changes: {total_changes} cells ({change_density*100:.1f}%) | Best match: {best_attempt.get('match_score', 0):.1%} | Schema: {best_attempt.get('schema_match', False)}"
    }}
    """

    log_to_file(f"""ENHANCED FINAL ANALYSIS GENERATION:
Sending enhanced final analysis request to GPT-4.1...
Features: Schema insights, learning history, precision focus

ENHANCED PROMPT FOR FINAL ANALYSIS:
{'-'*60}
{enhanced_analysis_prompt}
{'-'*60}
""", "ENHANCED FINAL ANALYSIS GENERATION")

    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[
                {"role": "system", "content": "You are an enhanced data analyst with schema validation expertise. Return only valid JSON with comprehensive insights."},
                {"role": "user", "content": enhanced_analysis_prompt}
            ],
            max_tokens=4000,
            temperature=0.1
        )

        enhanced_gpt_final_response = response.choices[0].message.content
        log_to_file(f"""ENHANCED FINAL ANALYSIS GPT-4.1 RESPONSE:
{'-'*60}
{enhanced_gpt_final_response}
{'-'*60}
""")

        try:
            import json
            import re
            json_match = re.search(r'{.*}', enhanced_gpt_final_response, re.DOTALL)
            if json_match:
                enhanced_parsed_analysis = json.loads(json_match.group(0))
                log_to_file("✅ Successfully parsed JSON from enhanced final analysis response")
                return enhanced_parsed_analysis
            else:
                log_to_file("❌ No JSON found in enhanced final analysis response")
        except Exception as json_error:
            log_to_file(f"❌ Enhanced JSON parsing failed: {str(json_error)}")

    except Exception as api_error:
        log_to_file(f"❌ Enhanced final analysis API call failed: {str(api_error)}")

    # Enhanced fallback analysis
    enhanced_fallback_analysis = {
        "change_summary": f"ENHANCED_WORKING_CLAUDE_PROMPT: {final_claude_prompt} | ENHANCED_GPT4_ANALYSIS: [embedded] | {user_description} - {total_changes} changes with {replication_results['final_success']} replication and {learning_applied} learning",
        "change_type": "data_edit",
        "enhanced_data_modifications": {
            "cells_changed": total_changes,
            "total_cells": total_cells,
            "change_percentage": change_density*100,
            "data_type_mismatches": len(data_type_mismatches),
            "schema_compatibility_score": schema_comparison.get('schema_compatibility_score', 0)
        },
        "replication_analysis": {
            "final_success": replication_results['final_success'],
            "best_match_score": best_attempt.get('match_score', 0),
            "learning_system_applied": learning_applied,
            "total_learning_insights": len(learning_history)
        },
        "session_description": f"ENHANCED_REPLICATION_STATUS: {success_status} | FINAL_CLAUDE_PROMPT: {final_claude_prompt} | LEARNING_APPLIED: {learning_applied} | USER_EDIT: {user_description} | Changes: {total_changes} cells"
    }

    log_to_file(f"Using enhanced fallback analysis:\n{json.dumps(enhanced_fallback_analysis, indent=2)}")
    return enhanced_fallback_analysis


# Enhanced Save Function with All Fixes Applied
def save_edited_dataframe_enhanced_with_all_fixes(edited_df, description):
    """
    ULTIMATE enhanced version with ALL CRITICAL FIXES APPLIED:
    1. ✅ Fixed logging string formatting bug
    2. ✅ Enhanced data type/formatting precision matching
    3. ✅ Implemented attempt-to-attempt learning system
    4. ✅ Added comprehensive data type validation
    """
    global app_state, session_recorder

    if edited_df is None or edited_df.empty:
        return "No data to save", gr.update()

    try:
        # Convert the edited dataframe to proper pandas DataFrame if needed
        if not isinstance(edited_df, pd.DataFrame):
            edited_df = pd.DataFrame(edited_df)

        # Get the original dataframe for comparison
        original_df = app_state["df"].copy()

        logger.info("Analyzing dataframe changes with ALL ENHANCED FIXES...")
        print("🚀 Analyzing changes with ALL CRITICAL FIXES APPLIED...")

        # Use the ULTIMATE enhanced analysis function with all fixes
        change_analysis = analyze_dataframe_changes_with_gpt4(
            original_df=original_df,
            modified_df=edited_df,
            user_description=description
        )

        # Generate meaningful description if not provided
        if not description:
            description = change_analysis.get("change_summary", "Manual edits via enhanced data editor")

        # Save as new version
        version_name = save_dataframe_version(edited_df, description)

        # Update app state
        app_state["df"] = edited_df

        # Enhanced session recording with all fixes
        if session_recorder.current_session_file:
            session_description = change_analysis.get("session_description", f"Enhanced manual data edits: {description}")

            # Create comprehensive session entry with all enhancements
            enhanced_session_entry = f"""
ULTIMATE ENHANCED MANUAL DATA EDIT SESSION - ALL FIXES APPLIED
============================================================
Timestamp: {datetime.now().strftime('%H:%M:%S')}
Edit Description: {description}
Version Saved: {version_name}
Log File: {change_analysis.get('log_file_path', 'N/A')}

ALL CRITICAL FIXES APPLIED:
✅ 1. Fixed logging string formatting bug
✅ 2. Enhanced data type/formatting precision matching
✅ 3. Implemented attempt-to-attempt learning system
✅ 4. Added comprehensive data type validation

ULTIMATE ENHANCED GPT-4.1 CHANGE ANALYSIS:
{'-' * 50}
Change Summary: {change_analysis.get('change_summary', 'N/A')}
Change Type: {change_analysis.get('change_type', 'N/A')}
Enhanced Features Applied: {change_analysis.get('enhanced_features_applied', False)}

Replication Analysis:
{json.dumps(change_analysis.get('replication_analysis', {}), indent=2)}

Enhanced Data Modifications:
{json.dumps(change_analysis.get('enhanced_data_modifications', {}), indent=2)}

Structural Changes:
{json.dumps(change_analysis.get('structural_changes', {}), indent=2)}

Business Impact:
{json.dumps(change_analysis.get('business_impact', {}), indent=2)}

Enhanced Recommendations:
{chr(10).join([f"• {rec}" for rec in change_analysis.get('enhanced_recommendations', [])])}

Ultimate Technical Statistics:
- Total Cells: {change_analysis.get('full_change_statistics', {}).get('total_cells', 'Unknown')}
- Changed Cells: {change_analysis.get('full_change_statistics', {}).get('total_changes', 'Unknown')}
- Change Density: {change_analysis.get('full_change_statistics', {}).get('change_density', 0)*100:.2f}%
- Data Type Mismatches: {change_analysis.get('full_change_statistics', {}).get('data_type_mismatches', 'Unknown')}
- Schema Compatibility: {change_analysis.get('full_change_statistics', {}).get('schema_changes', {}).get('schema_compatibility_score', 'Unknown')}
- Learning Applied: {change_analysis.get('full_change_statistics', {}).get('learning_applied', False)}
- Affected Rows: {len(change_analysis.get('full_change_statistics', {}).get('affected_rows', []))}
- Affected Columns: {len(change_analysis.get('full_change_statistics', {}).get('affected_columns', []))}

Original DataFrame Shape: {original_df.shape}
Modified DataFrame Shape: {edited_df.shape}
{'-' * 80}
"""

            # Append to session file
            with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
                f.write(enhanced_session_entry + "\n")

            # Record in session data structure
            session_recorder.record_conversation_turn(
                user_message=f"ULTIMATE ENHANCED MANUAL EDIT: {description}",
                ai_response=session_description,
                action_type="manual_data_edit_ultimate_enhanced",
                code_executed=None,
                version_saved=version_name
            )

            # Record dataframe version change
            session_recorder.record_dataframe_version(
                version_name=version_name,
                description=description,
                shape=list(edited_df.shape),
                columns=list(edited_df.columns)
            )

            # Record issues with enhanced analysis
            replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
            learning_applied = change_analysis.get('full_change_statistics', {}).get('learning_applied', False)

            if not replication_success:
                session_recorder.record_issue_found(
                    f"Enhanced manual edit replication failed: {description}. Check detailed log for comprehensive analysis.",
                    severity="medium"
                )
            elif learning_applied:
                session_recorder.record_issue_found(
                    f"Manual edit required learning system intervention: {description}. Replication succeeded after learning.",
                    severity="low"
                )

            logger.info("Ultimate enhanced manual edit analysis recorded in copiloting session")

        # Log the changes
        logger.info(f"Saved edited dataframe as version {version_name} with all enhancements")

        # Create comprehensive success message with all enhancements
        log_file_path = change_analysis.get('log_file_path', 'N/A')
        replication_success = change_analysis.get('full_change_statistics', {}).get('replication_success', False)
        replication_attempts = change_analysis.get('full_change_statistics', {}).get('replication_attempts', 0)
        learning_applied = change_analysis.get('full_change_statistics', {}).get('learning_applied', False)
        schema_score = change_analysis.get('full_change_statistics', {}).get('schema_changes', {}).get('schema_compatibility_score', 0)
        data_type_mismatches = change_analysis.get('full_change_statistics', {}).get('data_type_mismatches', 0)

        ultimate_success_message = f"""✅ Successfully saved as version {version_name}

🚀 ULTIMATE ENHANCED GPT-4.1 Analysis Summary:
{change_analysis.get('change_summary', 'Changes analyzed with all enhancements')[:300]}...

📊 Enhanced Change Details:
• Change Type: {change_analysis.get('change_type', 'Unknown')}
• Original Shape: {original_df.shape}
• New Shape: {edited_df.shape}
• Replication Success: {'✅ Yes' if replication_success else '❌ Partial/Failed'}
• Replication Attempts: {replication_attempts}
• Learning System Applied: {'✅ Yes' if learning_applied else '❌ No'}
• Schema Compatibility: {schema_score:.1%}
• Data Type Mismatches: {data_type_mismatches}

🔧 ALL CRITICAL FIXES APPLIED:
✅ 1. Fixed logging string formatting bug
✅ 2. Enhanced data type/formatting precision matching
✅ 3. Implemented attempt-to-attempt learning system
✅ 4. Added comprehensive data type validation

📝 Session Recording: {'✅ Recorded' if session_recorder.current_session_file else '❌ No active session'}

📁 Comprehensive Analysis Log:
{log_file_path}

This ultimate enhanced log contains:
• Complete GPT-4.1 prompts and responses with schema focus
• All Claude API calls with learning system feedback
• Step-by-step replication attempts with precision validation
• Cell-by-cell change analysis with data type awareness
• Business impact assessment with enhanced insights
• Learning system evolution and improvements
• Comprehensive error handling and debugging information
"""

        # Add enhanced recommendations if available
        if change_analysis.get('enhanced_recommendations'):
            ultimate_success_message += f"\n💡 Enhanced Recommendations:\n"
            for rec in change_analysis['enhanced_recommendations'][:3]:
                ultimate_success_message += f"• {rec}\n"

        # Add comprehensive log file information
        ultimate_success_message += f"\n📄 For complete enhanced debugging and learning insights: {log_file_path}"

        # Add replication analysis summary
        replication_analysis = change_analysis.get('replication_analysis', {})
        if replication_analysis:
            ultimate_success_message += f"\n\n🤖 Replication Analysis:"
            ultimate_success_message += f"\n• Best Match Score: {replication_analysis.get('best_match_score', 0):.1%}"
            ultimate_success_message += f"\n• Schema Validation: {'✅ Passed' if replication_analysis.get('schema_validation_passed', False) else '❌ Failed'}"
            ultimate_success_message += f"\n• Learning Insights Generated: {replication_analysis.get('total_learning_insights', 0)}"

        return ultimate_success_message, gr.update(value=edited_df)

    except Exception as e:
        error_msg = f"❌ Error in ultimate enhanced save: {str(e)}"
        logger.error(f"Error in ultimate enhanced save: {e}")
        logger.error(traceback.format_exc())

        # Enhanced error recording in session
        if session_recorder.current_session_file:
            session_recorder.record_conversation_turn(
                user_message=f"ULTIMATE ENHANCED MANUAL EDIT FAILED: {description}",
                ai_response=error_msg,
                action_type="manual_edit_error_ultimate_enhanced",
                code_executed=None,
                version_saved=None
            )

        return error_msg, gr.update()


# Enhanced Log Summary with All Fixes
def get_enhanced_manual_edit_logs_summary_with_all_fixes():
    """
    Generate enhanced summary of all manual edit analysis log files with comprehensive insights
    """
    logs_dir = "manual_edit_analysis_logs"

    if not os.path.exists(logs_dir):
        return "No enhanced manual edit analysis logs found yet."

    try:
        log_files = [f for f in os.listdir(logs_dir) if f.endswith('_detailed_analysis.txt')]

        if not log_files:
            return "No enhanced detailed analysis log files found."

        # Sort by creation time (newest first)
        log_files.sort(key=lambda x: os.path.getctime(os.path.join(logs_dir, x)), reverse=True)

        enhanced_summary = f"""📁 ULTIMATE ENHANCED Manual Edit Analysis Logs Summary
Found {len(log_files)} comprehensive analysis log files with ALL FIXES APPLIED:

🔧 CRITICAL FIXES INCLUDED IN LOGS:
✅ 1. Fixed logging string formatting bug
✅ 2. Enhanced data type/formatting precision matching
✅ 3. Implemented attempt-to-attempt learning system
✅ 4. Added comprehensive data type validation

"""

        total_size = 0
        enhanced_features_count = 0
        learning_applied_count = 0

        for i, log_file in enumerate(log_files[:15], 1):  # Show last 15
            file_path = os.path.join(logs_dir, log_file)
            file_size = os.path.getsize(file_path)
            total_size += file_size
            created_time = datetime.fromtimestamp(os.path.getctime(file_path))

            # Try to extract enhanced information from log file
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    log_content = f.read()

                # Check for enhanced features
                has_enhanced_features = "ENHANCED FEATURES APPLIED" in log_content
                has_learning = "LEARNING SYSTEM" in log_content
                has_schema_validation = "SCHEMA VALIDATION" in log_content

                if has_enhanced_features:
                    enhanced_features_count += 1
                if has_learning:
                    learning_applied_count += 1

            except Exception:
                has_enhanced_features = False
                has_learning = False
                has_schema_validation = False

            # Extract session info from filename
            session_id = log_file.replace('_detailed_analysis.txt', '')

            enhanced_summary += f"""{i}. {session_id}
   📄 File: {log_file}
   📅 Created: {created_time.strftime('%Y-%m-%d %H:%M:%S')}
   💾 Size: {file_size:,} bytes
   🔧 Enhanced Features: {'✅ Yes' if has_enhanced_features else '❌ No'}
   🧠 Learning Applied: {'✅ Yes' if has_learning else '❌ No'}
   📋 Schema Validation: {'✅ Yes' if has_schema_validation else '❌ No'}
   📁 Path: {file_path}

"""

        if len(log_files) > 15:
            enhanced_summary += f"... and {len(log_files) - 15} more enhanced log files\n"

        enhanced_summary += f"""
📈 ENHANCED LOGS STATISTICS:
• Total Log Files: {len(log_files)}
• Total Storage Used: {total_size:,} bytes ({total_size/1024/1024:.1f} MB)
• Files with Enhanced Features: {enhanced_features_count}/{len(log_files)} ({enhanced_features_count/len(log_files)*100:.1f}%)
• Files with Learning System: {learning_applied_count}/{len(log_files)} ({learning_applied_count/len(log_files)*100:.1f}%)

📋 ULTIMATE ENHANCED LOG CONTENTS INCLUDE:
• Complete GPT-4.1 prompts and responses with schema awareness
• All Claude API calls with learning system feedback loops
• Step-by-step replication testing with precision validation
• Cell-by-cell dataframe comparison with data type analysis
• Schema compatibility analysis and validation results
• Business impact assessment with enhanced insights
• Learning system evolution and improvement tracking
• Data type mismatch detection and resolution strategies
• Comprehensive error handling and debugging information
• Attempt-to-attempt learning and prompt refinement details

🔧 CRITICAL FIXES VERIFICATION:
1. ✅ Logging String Formatting: All logs use proper string formatting
2. ✅ Data Type Precision: Enhanced schema validation throughout
3. ✅ Learning System: Progressive improvement across attempts
4. ✅ Type Validation: Comprehensive data type compatibility checking

📂 All enhanced logs saved in: {logs_dir}/
"""

        return enhanced_summary

    except Exception as e:
        return f"Error reading enhanced log files: {str(e)}"


# Enhanced Log Reader
def read_enhanced_manual_edit_log(session_id):
    """
    Read and return enhanced log with analysis insights
    """
    logs_dir = "manual_edit_analysis_logs"
    log_file_path = os.path.join(logs_dir, f"{session_id}_detailed_analysis.txt")

    if not os.path.exists(log_file_path):
        return f"Enhanced log file not found: {log_file_path}"

    try:
        with open(log_file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Analyze log content for enhanced insights
        enhanced_features_detected = "ENHANCED FEATURES APPLIED" in content
        learning_system_detected = "LEARNING SYSTEM" in content
        schema_validation_detected = "SCHEMA VALIDATION" in content
        replication_success = "REPLICATION SUCCESSFUL" in content

        file_size = len(content)
        line_count = content.count('\n')

        return f"""📄 ULTIMATE ENHANCED Manual Edit Analysis Log: {session_id}
{'='*80}

🔧 CRITICAL FIXES STATUS:
✅ Enhanced Features Applied: {enhanced_features_detected}
✅ Learning System Active: {learning_system_detected}
✅ Schema Validation Included: {schema_validation_detected}
✅ Replication Successful: {replication_success}

📊 LOG STATISTICS:
• File Size: {file_size:,} characters ({file_size/1024:.1f} KB)
• Total Lines: {line_count:,}
• Enhanced Analysis: {'✅ Complete' if enhanced_features_detected else '❌ Basic'}

{'='*80}

{content}

{'='*80}
End of ultimate enhanced log file: {log_file_path}

🔍 ANALYSIS SUMMARY:
This log contains {'comprehensive enhanced analysis' if enhanced_features_detected else 'basic analysis'} with:
• {'✅' if 'GPT-4.1' in content else '❌'} GPT-4.1 API interactions
• {'✅' if 'CLAUDE API' in content else '❌'} Claude API calls and responses
• {'✅' if learning_system_detected else '❌'} Learning system feedback loops
• {'✅' if schema_validation_detected else '❌'} Schema compatibility validation
• {'✅' if 'DATA TYPE' in content else '❌'} Data type precision analysis
• {'✅' if replication_success else '❌'} Successful replication testing
"""

    except Exception as e:
        return f"Error reading enhanced log file: {str(e)}"

In [ ]:
def save_edited_dataframe_enhanced(edited_df, description):
    """
    Enhanced version that analyzes changes with GPT-4.1 and records in session.
    """
    global app_state, session_recorder

    if edited_df is None or edited_df.empty:
        return "No data to save", gr.update()

    try:
        # Convert the edited dataframe to proper pandas DataFrame if needed
        if not isinstance(edited_df, pd.DataFrame):
            edited_df = pd.DataFrame(edited_df)

        # Get the original dataframe for comparison
        original_df = app_state["df"].copy()

        logger.info("Analyzing dataframe changes with GPT-4.1...")
        print("🤖 Analyzing changes with GPT-4.1...")

        # Use GPT-4.1 to analyze the differences
        change_analysis = analyze_dataframe_changes_with_gpt4(
            original_df=original_df,
            modified_df=edited_df,
            user_description=description
        )

        # Generate a meaningful description if not provided
        if not description:
            description = change_analysis.get("change_summary", "Manual edits via data editor")

        # Save as new version
        version_name = save_dataframe_version(edited_df, description)

        # Update the app state with the edited dataframe
        app_state["df"] = edited_df

        # Record this in the copiloting session if active
        if session_recorder.current_session_file:
            session_description = change_analysis.get("session_description", f"Manual data edits: {description}")

            # Create detailed session entry
            session_entry = f"""
MANUAL DATA EDIT SESSION
========================
Timestamp: {datetime.now().strftime('%H:%M:%S')}
Edit Description: {description}
Version Saved: {version_name}

GPT-4.1 CHANGE ANALYSIS:
{'-' * 40}
Change Summary: {change_analysis.get('change_summary', 'N/A')}
Change Type: {change_analysis.get('change_type', 'N/A')}

Structural Changes:
{json.dumps(change_analysis.get('structural_changes', {}), indent=2)}

Data Modifications:
{json.dumps(change_analysis.get('data_modifications', {}), indent=2)}

Business Impact:
{json.dumps(change_analysis.get('business_impact', {}), indent=2)}

Recommendations:
{chr(10).join([f"• {rec}" for rec in change_analysis.get('recommendations', [])])}

Original DataFrame Shape: {original_df.shape}
Modified DataFrame Shape: {edited_df.shape}
{'-' * 80}
"""

            # Append to session file
            with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
                f.write(session_entry + "\n")

            # Record in session data structure
            session_recorder.record_conversation_turn(
                user_message=f"MANUAL EDIT: {description}",
                ai_response=session_description,
                action_type="manual_data_edit",
                code_executed=None,
                version_saved=version_name
            )

            # Record the dataframe version change
            session_recorder.record_dataframe_version(
                version_name=version_name,
                description=description,
                shape=list(edited_df.shape),
                columns=list(edited_df.columns)
            )

            # Record any issues found by GPT-4
            if change_analysis.get('data_modifications', {}).get('data_quality_impact') == 'degraded':
                session_recorder.record_issue_found(
                    f"Data quality may have degraded due to manual edits: {description}",
                    severity="medium"
                )

            logger.info("Manual edit recorded in copiloting session")

        # Log the changes
        logger.info(f"Saved edited dataframe as version {version_name}")

        # Create detailed success message
        success_message = f"""✅ Successfully saved as version {version_name}

🤖 GPT-4.1 Analysis Summary:
{change_analysis.get('change_summary', 'Changes analyzed')}

📊 Change Details:
• Change Type: {change_analysis.get('change_type', 'Unknown')}
• Original Shape: {original_df.shape}
• New Shape: {edited_df.shape}

📝 Session Recording: {'✅ Recorded' if session_recorder.current_session_file else '❌ No active session'}
"""

        # Add recommendations if available
        if change_analysis.get('recommendations'):
            success_message += f"\n💡 Recommendations:\n"
            for rec in change_analysis['recommendations'][:3]:  # Show first 3
                success_message += f"• {rec}\n"

        return success_message, gr.update(value=edited_df)

    except Exception as e:
        error_msg = f"❌ Error saving: {str(e)}"
        logger.error(f"Error saving edited dataframe: {e}")
        logger.error(traceback.format_exc())

        # Still try to record the error in session
        if session_recorder.current_session_file:
            session_recorder.record_conversation_turn(
                user_message=f"MANUAL EDIT FAILED: {description}",
                ai_response=error_msg,
                action_type="manual_edit_error",
                code_executed=None,
                version_saved=None
            )

        return error_msg, gr.update()


def load_latest_version_for_editing_enhanced():
    """Enhanced version that records when user loads data for editing"""
    global app_state, session_recorder

    if app_state is None or app_state["df"] is None:
        return None, "No data loaded. Please upload a rent roll first."

    try:
        # Use the current dataframe (which is the latest)
        df = app_state["df"].copy()
        df = df.fillna('')

        # Get version info
        if app_state["df_versions"]:
            latest_version = app_state["df_versions"][-1]
            version_info = f"Loaded version: {latest_version['name']} - {latest_version['description']}"
        else:
            version_info = "Loaded current data (no versions saved yet)"

        # Record this action in session if active
        if session_recorder.current_session_file:
            session_entry = f"""
DATA EDITING SESSION STARTED
============================
Timestamp: {datetime.now().strftime('%H:%M:%S')}
Action: User loaded dataframe for manual editing
Version Loaded: {latest_version['name'] if app_state["df_versions"] else 'Current'}
DataFrame Shape: {df.shape}
DataFrame Columns: {list(df.columns)}
{'-' * 80}
"""

            # Append to session file
            with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
                f.write(session_entry + "\n")

            # Record in session data
            session_recorder.record_conversation_turn(
                user_message="LOAD FOR EDITING: User opened data editor",
                ai_response="Dataframe loaded for manual editing",
                action_type="load_for_editing",
                code_executed=None,
                version_saved=None
            )

        logger.info(f"Loaded dataframe for editing: {df.shape}")
        return df, version_info

    except Exception as e:
        error_msg = f"Error loading data: {str(e)}"
        logger.error(f"Error loading data for editing: {e}")

        # Record error in session
        if session_recorder.current_session_file:
            session_recorder.record_conversation_turn(
                user_message="LOAD FOR EDITING FAILED",
                ai_response=error_msg,
                action_type="load_editing_error",
                code_executed=None,
                version_saved=None
            )

        return None, error_msg


def load_specific_version_enhanced(version_name):
    """Enhanced version that records version loading with GPT-4 analysis"""
    global app_state, session_recorder

    if not version_name:
        return None, "Please select a version to load"

    try:
        # Extract clean version name (remove status indicators)
        clean_version_name = version_name.split(" (")[0]

        # Find the version file
        versions_dir = "rent_roll_versions"
        csv_filename = os.path.join(versions_dir, f"rent_roll_{clean_version_name}.csv")

        if os.path.exists(csv_filename):
            df = pd.read_csv(csv_filename)
            df = df.fillna('')

            # Record this action in session if active
            if session_recorder.current_session_file:
                session_entry = f"""
SPECIFIC VERSION LOADED FOR EDITING
===================================
Timestamp: {datetime.now().strftime('%H:%M:%S')}
Version Loaded: {clean_version_name}
DataFrame Shape: {df.shape}
DataFrame Columns: {list(df.columns)}
File Path: {csv_filename}
{'-' * 80}
"""

                # Append to session file
                with open(session_recorder.current_session_file, 'a', encoding='utf-8') as f:
                    f.write(session_entry + "\n")

                # Record in session data
                session_recorder.record_conversation_turn(
                    user_message=f"LOAD SPECIFIC VERSION: {clean_version_name}",
                    ai_response=f"Loaded version {clean_version_name} for editing",
                    action_type="load_specific_version",
                    code_executed=None,
                    version_saved=None
                )

            logger.info(f"Loaded version {clean_version_name} for editing")
            return df, f"Loaded version: {clean_version_name}"
        else:
            error_msg = f"Version file not found: {clean_version_name}"

            # Record error in session
            if session_recorder.current_session_file:
                session_recorder.record_conversation_turn(
                    user_message=f"LOAD VERSION FAILED: {clean_version_name}",
                    ai_response=error_msg,
                    action_type="load_version_error",
                    code_executed=None,
                    version_saved=None
                )

            return None, error_msg

    except Exception as e:
        error_msg = f"Error loading version: {str(e)}"
        logger.error(f"Error loading version {version_name}: {e}")

        # Record error in session
        if session_recorder.current_session_file:
            session_recorder.record_conversation_turn(
                user_message=f"LOAD VERSION ERROR: {version_name}",
                ai_response=error_msg,
                action_type="load_version_error",
                code_executed=None,
                version_saved=None
            )

        return None, error_msg


# Update the Gradio event handlers to use enhanced functions
def setup_enhanced_edit_data_handlers():
    """
    Setup function to update Gradio event handlers for enhanced edit data functionality.
    Add this to your Gradio interface setup.
    """

    # Enhanced event handlers for Edit Data tab
    refresh_versions_btn.click(
        refresh_version_dropdown,
        outputs=[version_dropdown]
    )

    load_version_btn.click(
        load_specific_version_enhanced,  # Use enhanced version
        inputs=[version_dropdown],
        outputs=[editable_df, edit_status]
    )

    save_changes_btn.click(
        save_edited_dataframe_enhanced,  # Use enhanced version
        inputs=[editable_df, save_description],
        outputs=[save_status, editable_df]
    ).then(
        refresh_version_dropdown,  # Refresh the dropdown after saving
        outputs=[version_dropdown]
    )

    # You can also add a "Load Latest" button that uses the enhanced function
    # load_latest_btn.click(
    #     load_latest_version_for_editing_enhanced,
    #     outputs=[editable_df, edit_status]
    # )

In [ ]:
class TemplateApplicationEngine:
    def __init__(self):
        self.templates_dir = "rent_roll_templates"
        self.application_sessions_dir = "template_applications"
        os.makedirs(self.application_sessions_dir, exist_ok=True)
        self.current_application = None

    def start_template_application(self, template_id, new_rent_roll_file, new_rent_roll_df):
        """Initialize a new template application session"""

        # Load the template data
        template_data, starting_template_df, final_template_df = enhanced_template_manager.load_template_dataframes(template_id)

        if template_data is None:
            return None, "❌ Failed to load template data"

        # Create application session
        app_session_id = f"app_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        self.current_application = {
            "session_id": app_session_id,
            "template_id": template_id,
            "template_data": template_data,
            "starting_template_df": starting_template_df,
            "final_template_df": final_template_df,
            "new_rent_roll_file": new_rent_roll_file,
            "new_rent_roll_df": new_rent_roll_df.copy(),
            "current_df": new_rent_roll_df.copy(),
            "step_results": [],
            "current_step": 0,
            "total_steps": len(template_data.get("raw_workflow_steps", [])),
            "completed_steps": [],
            "failed_steps": [],
            "log_file": None
        }

        # Create log file
        log_filename = f"{app_session_id}_application_log.txt"
        self.current_application["log_file"] = os.path.join(self.application_sessions_dir, log_filename)

        # Write initial log
        with open(self.current_application["log_file"], 'w', encoding='utf-8') as f:
            f.write(f"=== TEMPLATE APPLICATION SESSION ===\n")
            f.write(f"Session ID: {app_session_id}\n")
            f.write(f"Template ID: {template_id}\n")
            f.write(f"Template Name: {template_data.get('template_name', 'Unknown')}\n")
            f.write(f"New Rent Roll File: {new_rent_roll_file}\n")
            f.write(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Total Steps to Execute: {self.current_application['total_steps']}\n")
            f.write(f"=" * 60 + "\n\n")

        return app_session_id, "✅ Template application session started successfully"

    def get_next_step_to_execute(self):
        """Get the next workflow step to execute"""
        if not self.current_application:
            return None, "No active application session"

        current_step = self.current_application["current_step"]
        workflow_steps = self.current_application["template_data"].get("raw_workflow_steps", [])

        if current_step >= len(workflow_steps):
            return None, "All steps completed"

        return workflow_steps[current_step], f"Step {current_step + 1} of {len(workflow_steps)}"

    def execute_next_step_with_ai(self):
        """Execute the next template step using GPT-4.1 + Claude 3.7 with DETAILED LOGGING"""
        if not self.current_application:
            return "❌ No active application session"

        # Get next step
        next_step, step_info = self.get_next_step_to_execute()
        if next_step is None:
            return self._finalize_application()

        current_step_num = self.current_application["current_step"] + 1

        try:
            # Enhanced step start logging
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"\n{'='*80}\n")
                f.write(f"EXECUTING STEP {current_step_num} OF {self.current_application['total_steps']}\n")
                f.write(f"{'='*80}\n")
                f.write(f"Timestamp: {datetime.now().strftime('%H:%M:%S')}\n")
                f.write(f"Original User Query: {next_step.get('user_message', 'N/A')}\n")
                f.write(f"Original Action Type: {next_step.get('action_type', 'N/A')}\n")
                f.write(f"Original Code Executed: {next_step.get('code_executed', 'None')}\n")
                f.write(f"Original AI Response: {next_step.get('ai_response', 'N/A')[:200]}...\n")
                f.write(f"{'-'*80}\n\n")

            # Use GPT-4.1 to analyze step and create optimal prompt for Claude
            print(f"🤖 Step {current_step_num}: Analyzing with GPT-4.1...")
            step_analysis = self._analyze_step_with_gpt4(next_step, current_step_num)

            # LOG GPT-4.1 ANALYSIS IN DETAIL
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"🤖 GPT-4.1 STEP ANALYSIS\n")
                f.write(f"{'-'*40}\n")
                f.write(f"Analysis Result:\n")
                f.write(json.dumps(step_analysis, indent=2, default=str))
                f.write(f"\n\nCan Execute: {step_analysis.get('step_adaptation', {}).get('can_execute', 'Unknown')}\n")
                f.write(f"Reason: {step_analysis.get('step_adaptation', {}).get('reason', 'No reason provided')}\n")
                f.write(f"\nColumn Mapping: {step_analysis.get('step_adaptation', {}).get('column_mapping', {})}\n")
                f.write(f"Parameter Adjustments: {step_analysis.get('step_adaptation', {}).get('parameter_adjustments', [])}\n")
                f.write(f"\nGenerated Claude Prompt:\n")
                f.write(f"{'─'*40}\n")
                f.write(f"{step_analysis.get('claude_prompt', 'No prompt generated')}\n")
                f.write(f"{'─'*40}\n\n")

            # Use Claude 3.7 to execute the adapted step
            print(f"🧠 Step {current_step_num}: Executing with Claude 3.7...")
            execution_result = self._execute_step_with_claude(step_analysis, current_step_num)

            # LOG CLAUDE EXECUTION IN DETAIL
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"🧠 CLAUDE 3.7 EXECUTION\n")
                f.write(f"{'-'*40}\n")
                f.write(f"Execution Success: {execution_result.get('success', False)}\n")
                f.write(f"Summary: {execution_result.get('summary', 'No summary')}\n\n")

                # Log Claude's full response
                f.write(f"Claude's Full Response:\n")
                f.write(f"{'─'*40}\n")
                f.write(f"{execution_result.get('claude_response', 'No response captured')}\n")
                f.write(f"{'─'*40}\n\n")

                # Log the actual code that was executed
                if execution_result.get('executed_code'):
                    f.write(f"Code Generated and Executed:\n")
                    f.write(f"```python\n")
                    f.write(f"{execution_result.get('executed_code')}\n")
                    f.write(f"```\n\n")

                # Log execution output
                if execution_result.get('execution_output'):
                    f.write(f"Code Execution Output:\n")
                    f.write(f"{'─'*40}\n")
                    f.write(f"{execution_result.get('execution_output')}\n")
                    f.write(f"{'─'*40}\n\n")

                # Log any errors
                if not execution_result.get('success') and execution_result.get('error'):
                    f.write(f"❌ ERROR DETAILS:\n")
                    f.write(f"{execution_result.get('error')}\n\n")

            # Record results
            step_result = {
                "step_number": current_step_num,
                "original_step": next_step,
                "gpt4_analysis": step_analysis,
                "claude_execution": execution_result,
                "success": execution_result.get("success", False),
                "timestamp": datetime.now().isoformat()
            }

            self.current_application["step_results"].append(step_result)

            if execution_result.get("success", False):
                self.current_application["completed_steps"].append(current_step_num)
                # Update current dataframe if changes were made
                if execution_result.get("updated_df") is not None:
                    self.current_application["current_df"] = execution_result["updated_df"]

                    # Log dataframe changes
                    with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                        f.write(f"📊 DATAFRAME UPDATED\n")
                        f.write(f"New Shape: {execution_result['updated_df'].shape}\n")
                        f.write(f"New Columns: {list(execution_result['updated_df'].columns)}\n\n")
            else:
                self.current_application["failed_steps"].append(current_step_num)

            # Move to next step
            self.current_application["current_step"] += 1

            # Enhanced step completion logging
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"{'='*40}\n")
                f.write(f"STEP {current_step_num} {'✅ SUCCESS' if execution_result.get('success') else '❌ FAILED'}\n")
                f.write(f"Completed at: {datetime.now().strftime('%H:%M:%S')}\n")
                f.write(f"Duration: Approximately 30-60 seconds\n")
                f.write(f"Result Summary: {execution_result.get('summary', 'No summary')}\n")

                # Add business context if available
                if step_analysis.get('business_context'):
                    f.write(f"Business Context: {step_analysis.get('business_context')}\n")

                f.write(f"{'='*40}\n\n")

            # Prepare enhanced status message
            total_steps = self.current_application["total_steps"]
            completed = len(self.current_application["completed_steps"])
            failed = len(self.current_application["failed_steps"])

            status_msg = f"""🔄 Step {current_step_num}/{total_steps} {'✅ Completed' if execution_result.get('success') else '❌ Failed'}

            **Step Details:**
            • Original Query: {next_step.get('user_message', 'N/A')[:100]}...
            • Action Type: {next_step.get('action_type', 'N/A')}
            • GPT-4.1 Analysis: {'✅ Successful' if step_analysis.get('step_adaptation', {}).get('can_execute') else '❌ Cannot Execute'}
            • Claude Execution: {'✅ Successful' if execution_result.get('success') else '❌ Failed'}

            **AI Processing Details:**
            • Column Mapping: {len(step_analysis.get('step_adaptation', {}).get('column_mapping', {}))} columns mapped
            • Code Generated: {'Yes' if execution_result.get('executed_code') else 'No'}
            • Dataframe Updated: {'Yes' if execution_result.get('updated_df') is not None else 'No'}

            **Progress:**
            • Completed: {completed}/{total_steps}
            • Failed: {failed}/{total_steps}
            • Remaining: {total_steps - current_step_num}

            📝 Detailed logs saved to: {os.path.basename(self.current_application["log_file"])}

            {'🎉 All steps completed!' if current_step_num >= total_steps else '⏭️ Ready for next step'}"""

            return status_msg

        except Exception as e:
            error_msg = f"❌ Error executing step {current_step_num}: {str(e)}"

            # Enhanced error logging
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"{'='*40}\n")
                f.write(f"❌ CRITICAL ERROR in step {current_step_num}\n")
                f.write(f"{'='*40}\n")
                f.write(f"Error Time: {datetime.now().strftime('%H:%M:%S')}\n")
                f.write(f"Error Message: {str(e)}\n")
                f.write(f"Error Type: {type(e).__name__}\n")
                f.write(f"Stack Trace:\n")
                f.write(f"{traceback.format_exc()}\n")
                f.write(f"{'='*40}\n\n")

            self.current_application["failed_steps"].append(current_step_num)
            self.current_application["current_step"] += 1

            return error_msg

    def _analyze_step_with_gpt4(self, step_data, step_number):
        """Enhanced GPT-4.1 analysis with detailed logging"""

        client = OpenAI(api_key=DEFAULT_OPENAI_API_KEY)

        # Prepare enhanced context for GPT-4
        template_context = f"""
        CRE RENT ROLL TEMPLATE APPLICATION CONTEXT:
        ==========================================

        Current Step: {step_number}/{self.current_application['total_steps']}
        Template Name: {self.current_application['template_data'].get('template_name', 'Unknown')}
        Session ID: {self.current_application['session_id']}

        ORIGINAL TEMPLATE DATAFRAMES:
        - Starting Template DF Shape: {self.current_application['starting_template_df'].shape}
        - Starting Template Columns: {list(self.current_application['starting_template_df'].columns)}
        - Starting Template Sample Data:
        {self.current_application['starting_template_df'].head(2).to_string()}

        - Final Template DF Shape: {self.current_application['final_template_df'].shape}
        - Final Template Columns: {list(self.current_application['final_template_df'].columns)}

        NEW CRE RENT ROLL TO PROCESS:
        - Current DF Shape: {self.current_application['current_df'].shape}
        - Current DF Columns: {list(self.current_application['current_df'].columns)}
        - Current DF Sample Data:
        {self.current_application['current_df'].head(3).to_string()}

        ORIGINAL STEP FROM TEMPLATE:
        - User Query: {step_data.get('user_message', 'N/A')}
        - Action Type: {step_data.get('action_type', 'N/A')}
        - Original Code: {step_data.get('code_executed', 'N/A')}
        - AI Response Preview: {step_data.get('ai_response', 'N/A')[:300]}...

        PREVIOUS COMPLETED STEPS IN THIS APPLICATION:
        {[f"Step {i}: Success" for i in self.current_application['completed_steps']]}

        PREVIOUS FAILED STEPS:
        {[f"Step {i}: Failed" for i in self.current_application['failed_steps']]}

        TEMPLATE GPT-4 ANALYSIS (Original Workflow Analysis):
        {json.dumps(self.current_application['template_data'].get('gpt4_analysis', {}), indent=2)[:1500]}...
        """

        # Enhanced analysis prompt with more specific instructions
        analysis_prompt = f"""
        You are an expert at adapting commercial real estate (CRE) rent roll analysis workflows to new datasets.

        Your task is to analyze the original template step and adapt it for the new CRE rent roll data, considering:
        1. Column name differences between template and new data
        2. Data value variations (different property types, tenant structures)
        3. Business logic preservation for CRE analysis
        4. Code adaptation requirements

        {template_context}

        Please provide a comprehensive analysis in JSON format:
        {{
            "step_adaptation": {{
                "can_execute": true/false,
                "reason": "Detailed explanation of why this step can or cannot be executed",
                "column_mapping": {{"template_column": "new_data_column"}},
                "parameter_adjustments": ["Specific parameter changes needed"],
                "prerequisites": ["What must be true before this step"],
                "data_compatibility": "assessment of data compatibility",
                "business_logic_changes": ["Any changes to business rules needed"]
            }},
            "claude_prompt": "Detailed, specific prompt for Claude 3.7 to execute this adapted CRE analysis step",
            "expected_outcome": "Detailed description of what this step should accomplish",
            "validation_criteria": ["Specific ways to verify the step succeeded"],
            "business_context": "Why this step is important for CRE rent roll analysis",
            "risk_assessment": "Any potential risks or issues with this adaptation",
            "fallback_strategy": "Alternative approach if main execution fails"
        }}

        CRITICAL: Focus on CRE-specific considerations:
        1. Mapping tenant information, lease dates, rent amounts, square footage columns
        2. Adapting rent calculations, occupancy analysis, lease expiration tracking
        3. Handling vacant spaces, percentage rent, CAM charges appropriately
        4. Ensuring financial calculations remain accurate for CRE analysis
        5. Creating clear, executable instructions that Claude can follow precisely
        """

        # Log the GPT-4 prompt being sent
        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"📤 SENDING TO GPT-4.1\n")
            f.write(f"{'-'*40}\n")
            f.write(f"Prompt Length: {len(analysis_prompt)} characters\n")
            f.write(f"Context Length: {len(template_context)} characters\n")
            f.write(f"Model: gpt-4o\n")
            f.write(f"Temperature: 0.2\n")
            f.write(f"Max Tokens: 3000\n")
            f.write(f"\nFull Prompt Sent to GPT-4.1:\n")
            f.write(f"{'─'*60}\n")
            f.write(f"{analysis_prompt}\n")
            f.write(f"{'─'*60}\n\n")

        try:
            response = client.chat.completions.create(
                model="gpt-4.1",
                messages=[
                    {"role": "system", "content": "You are an expert commercial real estate data analyst who adapts CRE rent roll processing workflows to new datasets. Provide detailed, actionable analysis in JSON format that considers CRE-specific data patterns and business requirements."},
                    {"role": "user", "content": analysis_prompt}
                ],
                max_tokens=3000,
                temperature=0.2
            )

            gpt_response = response.choices[0].message.content

            # Log the complete GPT-4 response
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"📥 RECEIVED FROM GPT-4.1\n")
                f.write(f"{'-'*40}\n")
                f.write(f"Response Length: {len(gpt_response)} characters\n")
                f.write(f"Tokens Used: ~{len(gpt_response.split())}\n")
                f.write(f"\nComplete GPT-4.1 Response:\n")
                f.write(f"{'─'*60}\n")
                f.write(f"{gpt_response}\n")
                f.write(f"{'─'*60}\n\n")

            # Try to extract JSON
            json_match = re.search(r'{.*}', gpt_response, re.DOTALL)
            if json_match:
                try:
                    parsed_analysis = json.loads(json_match.group(0))

                    # Log successful JSON parsing
                    with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                        f.write(f"✅ JSON PARSING SUCCESSFUL\n")
                        f.write(f"Parsed JSON Keys: {list(parsed_analysis.keys())}\n\n")

                    return parsed_analysis
                except Exception as json_error:
                    # Log JSON parsing failure
                    with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                        f.write(f"❌ JSON PARSING FAILED\n")
                        f.write(f"JSON Error: {str(json_error)}\n")
                        f.write(f"Extracted JSON Text:\n{json_match.group(0)}\n\n")

                    return {"analysis": gpt_response, "error": f"JSON parsing failed: {str(json_error)}"}
            else:
                # Log no JSON found
                with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                    f.write(f"❌ NO JSON FOUND in GPT-4.1 response\n\n")

                return {"analysis": gpt_response, "error": "No JSON found in response"}

        except Exception as e:
            # Log API call failure
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"❌ GPT-4.1 API CALL FAILED\n")
                f.write(f"Error: {str(e)}\n")
                f.write(f"Error Type: {type(e).__name__}\n\n")

            return {"error": str(e), "fallback": "GPT-4 analysis failed"}
    def _load_latest_dataframe_file(self, dataframes_dir, step_number):
        """Load the latest dataframe from saved files"""

        # Look for step files
        pattern = os.path.join(dataframes_dir, "step_*_dataframe.csv")
        dataframe_files = glob.glob(pattern)

        if not dataframe_files:
            # No previous files, use original and save it
            original_df = self.current_application["current_df"]
            original_file = os.path.join(dataframes_dir, "step_0_original_dataframe.csv")
            original_df.to_csv(original_file, index=False)

            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"📂 No previous files - saved original as: {original_file}\n")

            return original_df

        # Find the latest file by step number
        latest_step = -1
        latest_file = None

        for file_path in dataframe_files:
            filename = os.path.basename(file_path)
            match = re.search(r'step_(\d+)_', filename)
            if match:
                step_num = int(match.group(1))
                if step_num > latest_step:
                    latest_step = step_num
                    latest_file = file_path

        if latest_file is None:
            # Fallback to original
            return self.current_application["current_df"]

        # Load the latest dataframe
        latest_df = pd.read_csv(latest_file)

        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"📂 Loaded latest dataframe from step {latest_step}: {latest_file}\n")
            f.write(f"Shape: {latest_df.shape}\n")

        return latest_df
    def _save_dataframe_file(self, dataframe, dataframes_dir, step_number, status):
        """Save dataframe to file after step execution"""

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"step_{step_number}_{status}_{timestamp}_dataframe.csv"
        file_path = os.path.join(dataframes_dir, filename)

        # Save CSV
        dataframe.to_csv(file_path, index=False)

        # Also save Excel for verification
        excel_path = file_path.replace('.csv', '.xlsx')
        dataframe.to_excel(excel_path, index=False, engine='openpyxl')

        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"💾 DATAFRAME SAVED:\n")
            f.write(f"CSV: {file_path}\n")
            f.write(f"Excel: {excel_path}\n")
            f.write(f"Shape: {dataframe.shape}\n")
            f.write(f"Size: {os.path.getsize(file_path)} bytes\n")

        return file_path
    def _finalize_application(self):
        """Simple finalization - load latest file and save final result"""
        if not self.current_application:
            return "No active session to finalize"

        session_id = self.current_application["session_id"]
        total_steps = self.current_application["total_steps"]
        completed = len(self.current_application["completed_steps"])
        failed = len(self.current_application["failed_steps"])

        # Load the very latest dataframe file
        dataframes_dir = os.path.join(self.application_sessions_dir, f"{session_id}_dataframes")
        final_df = self._load_latest_dataframe_file(dataframes_dir, total_steps + 1)

        # Save final result
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        final_csv = os.path.join(self.application_sessions_dir, f"{session_id}_FINAL_RESULT_{timestamp}.csv")
        final_excel = os.path.join(self.application_sessions_dir, f"{session_id}_FINAL_RESULT_{timestamp}.xlsx")

        final_df.to_csv(final_csv, index=False)
        final_df.to_excel(final_excel, index=False, engine='openpyxl')

        # Log final summary
        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"\n{'='*60}\n")
            f.write("🎉 TEMPLATE APPLICATION COMPLETED\n")
            f.write(f"{'='*60}\n")
            f.write(f"Total Steps: {total_steps}\n")
            f.write(f"Completed Successfully: {completed}\n")
            f.write(f"Failed: {failed}\n")
            f.write(f"Success Rate: {(completed/total_steps)*100:.1f}%\n")
            f.write(f"Final DataFrame Shape: {final_df.shape}\n")
            f.write(f"Final Result CSV: {final_csv}\n")
            f.write(f"Final Result Excel: {final_excel}\n")
            f.write(f"All Step Files: {dataframes_dir}\n")
            f.write(f"Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"{'='*60}\n")

        self.current_application = None

        return f"""🎉 Template Application Completed!
        📊 Results:
        • Total Steps: {total_steps}
        • Successfully Completed: {completed}
        • Failed: {failed}
        • Success Rate: {(completed/total_steps)*100:.1f}%

        📁 Final Output:
        • CSV: {final_csv}
        • Excel: {final_excel}
        • Shape: {final_df.shape}

        📂 All Step Files: {dataframes_dir}

        ✅ File-based persistence ensures no data loss!"""

    def _execute_step_with_claude(self, step_analysis, step_number):
        """Simple file-based dataframe persistence - save after every step"""

        if not step_analysis.get("step_adaptation", {}).get("can_execute", False):
            failure_reason = step_analysis.get("step_adaptation", {}).get("reason", "Unknown reason")

            # Log the skip reason
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"⏭️ STEP {step_number} SKIPPED\n")
                f.write(f"Reason: {failure_reason}\n\n")

            return {
                "success": False,
                "summary": f"Step {step_number} skipped: {failure_reason}",
                "error": "Step cannot be executed",
                "claude_response": "Step was not executed due to GPT-4.1 analysis"
            }

        claude_client = Anthropic(api_key=DEFAULT_ANTHROPIC_API_KEY)

        # Step 1: Load the latest dataframe from file
        session_id = self.current_application["session_id"]
        dataframes_dir = os.path.join(self.application_sessions_dir, f"{session_id}_dataframes")
        os.makedirs(dataframes_dir, exist_ok=True)

        latest_df = self._load_latest_dataframe_file(dataframes_dir, step_number)

        # Step 2: Prepare Claude prompt
        claude_prompt = step_analysis.get("claude_prompt", "Execute the data processing step")

        df_context = f"""
        CURRENT DATAFRAME STATUS:
        ========================
        Shape: {latest_df.shape}
        Columns: {list(latest_df.columns)}
        Data Types: {dict(latest_df.dtypes.astype(str))}

        Sample Data (First 3 rows):
        {latest_df.head(3).to_string()}

        The dataframe is already loaded as 'df' variable.
        """

        full_claude_prompt = f"""
        {claude_prompt}

        {df_context}

        EXECUTION INSTRUCTIONS:
        1. The dataframe 'df' is already loaded and available for use
        2. Execute the required data processing step as analyzed by GPT-4.1
        3. Show your work step by step with clear explanations
        4. Use proper error handling but don't suppress errors completely
        5. Display results clearly and comprehensively
        6. Include print statements to show what you're doing

        Expected Outcome: {step_analysis.get('expected_outcome', 'Process the data as required')}

        Validation Criteria:
        {chr(10).join([f"• {criteria}" for criteria in step_analysis.get('validation_criteria', [])])}

        Business Context: {step_analysis.get('business_context', 'CRE rent roll processing')}

        Please provide your code in ```python``` blocks and explain your approach clearly.
        """

        # Log prompt
        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"📤 SENDING TO CLAUDE 3.7 - STEP {step_number}\n")
            f.write(f"Input DataFrame Shape: {latest_df.shape}\n")
            f.write(f"Prompt Length: {len(full_claude_prompt)} characters\n\n")

        try:
            # Execute with Claude
            claude_response = claude_client.messages.create(
                model="claude-3-7-sonnet-20250219",
                messages=[{"role": "user", "content": full_claude_prompt}],
                max_tokens=4000,
                temperature=0.3
            )

            response_text = claude_response.content[0].text

            # Log Claude's response
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"📥 RECEIVED FROM CLAUDE 3.7 - STEP {step_number}\n")
                f.write(f"Response Length: {len(response_text)} characters\n\n")

            # Extract code blocks
            code_blocks = re.findall(r'```python\s*(.*?)\s*```', response_text, re.DOTALL)

            if not code_blocks:
                # Save current dataframe even if no code
                self._save_dataframe_file(latest_df, dataframes_dir, step_number, "no_code")

                with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                    f.write(f"❌ NO CODE BLOCKS FOUND - Saved unchanged dataframe\n\n")

                return {
                    "success": False,
                    "summary": f"Step {step_number}: No executable code generated by Claude",
                    "claude_response": response_text,
                    "error": "No code blocks found in Claude's response"
                }

            # Step 3: Execute code
            exec_globals = {
                "df": latest_df.copy(),
                "pd": pd,
                "np": np,
                "datetime": datetime,
                "os": os
            }

            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"⚡ CODE EXECUTION STARTING - STEP {step_number}\n")
                f.write(f"Input DataFrame Shape: {exec_globals['df'].shape}\n")
                f.write(f"Executing {len(code_blocks)} code block(s)...\n\n")

            execution_success = False
            execution_output = ""
            output_buffer = io.StringIO()

            try:
                with redirect_stdout(output_buffer):
                    for i, code_block in enumerate(code_blocks, 1):
                        print(f"--- Executing Code Block {i} ---")
                        exec(code_block, exec_globals)
                        print(f"--- Code Block {i} Completed ---\n")

                execution_output = output_buffer.getvalue()
                updated_df = exec_globals["df"]
                execution_success = True

            except Exception as e:
                execution_output = f"Execution error: {str(e)}\n{traceback.format_exc()}"
                execution_success = False
                updated_df = latest_df  # Use original if failed

                with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                    f.write(f"❌ CODE EXECUTION FAILED - STEP {step_number}\n")
                    f.write(f"Error: {str(e)}\n\n")

            # Step 4: ALWAYS save dataframe after execution (success or failure)
            status = "success" if execution_success else "failed"
            saved_file = self._save_dataframe_file(updated_df, dataframes_dir, step_number, status)

            # Update memory reference for consistency
            self.current_application["current_df"] = updated_df.copy()

            # Log results
            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                result_status = "✅ SUCCESS" if execution_success else "❌ FAILED"
                f.write(f"{result_status} - STEP {step_number}\n")
                f.write(f"Input Shape: {latest_df.shape}\n")
                f.write(f"Output Shape: {updated_df.shape}\n")
                f.write(f"💾 Saved to: {saved_file}\n")
                f.write(f"Execution Output:\n{execution_output}\n\n")

            return {
                "success": execution_success,
                "summary": f"Step {step_number}: {'✅ Successfully executed' if execution_success else '❌ Execution failed'}",
                "claude_response": response_text,
                "executed_code": "\n\n# --- Next Code Block ---\n\n".join(code_blocks),
                "execution_output": execution_output,
                "updated_df": updated_df,
                "saved_file": saved_file
            }

        except Exception as e:
            # Save current dataframe even on API failure
            self._save_dataframe_file(latest_df, dataframes_dir, step_number, "api_failure")

            with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
                f.write(f"❌ CLAUDE API FAILED - STEP {step_number}\n")
                f.write(f"Error: {str(e)}\n\n")

            return {
                "success": False,
                "summary": f"Step {step_number}: Claude execution failed - {str(e)}",
                "error": str(e),
                "claude_response": "Failed to get response from Claude"
            }


    def _finalize_application(self):
        """Finalize the template application session"""
        if not self.current_application:
            return "No active session to finalize"

        total_steps = self.current_application["total_steps"]
        completed = len(self.current_application["completed_steps"])
        failed = len(self.current_application["failed_steps"])

        # Save final results
        final_df = self.current_application["current_df"]
        session_id = self.current_application["session_id"]

        # Save final dataframe
        final_df_path = os.path.join(self.application_sessions_dir, f"{session_id}_final_result.csv")
        final_df.to_csv(final_df_path, index=False)

        # Write final summary to log
        with open(self.current_application["log_file"], 'a', encoding='utf-8') as f:
            f.write(f"\n{'=' * 60}\n")
            f.write("TEMPLATE APPLICATION COMPLETED\n")
            f.write(f"{'=' * 60}\n")
            f.write(f"Total Steps: {total_steps}\n")
            f.write(f"Completed Successfully: {completed}\n")
            f.write(f"Failed: {failed}\n")
            f.write(f"Success Rate: {(completed/total_steps)*100:.1f}%\n")
            f.write(f"Final Result Saved: {final_df_path}\n")
            f.write(f"Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

        success_msg = f"""🎉 Template Application Completed!

        📊 Final Results:
        • Total Steps: {total_steps}
        • Successfully Completed: {completed}
        • Failed: {failed}
        • Success Rate: {(completed/total_steps)*100:.1f}%

        📁 Output Files:
        • Final Processed Data: {final_df_path}
        • Application Log: {self.current_application["log_file"]}

        📈 Final Dataframe:
        • Shape: {final_df.shape}
        • Columns: {len(final_df.columns)}

        The template has been successfully applied to your new rent roll!"""

        # Reset application state
        self.current_application = None

        return success_msg

    def get_application_status(self):
        """Get current application status"""
        if not self.current_application:
            return "📴 No active template application session"

        total = self.current_application["total_steps"]
        current = self.current_application["current_step"]
        completed = len(self.current_application["completed_steps"])
        failed = len(self.current_application["failed_steps"])

        return f"""📋 Template Application Status

        🎯 Template: {self.current_application['template_data'].get('template_name', 'Unknown')}
        📁 Processing: {self.current_application['new_rent_roll_file']}

        📊 Progress:
        • Current Step: {current}/{total}
        • Completed: {completed}
        • Failed: {failed}
        • Remaining: {total - current}

        🔄 Status: {'🎉 Completed' if current >= total else '⏳ In Progress'}"""

# Global template application engine
template_app_engine = TemplateApplicationEngine()

# Functions for the Template Application tab

def load_template_for_application(template_id):
    """Load template details for application"""
    if not template_id:
        return "Please enter a template ID", "", ""

    try:
        template_summary = enhanced_template_manager.get_template_summary(template_id)

        # Get template steps for preview
        template_json_path = os.path.join("rent_roll_templates", f"{template_id}.json")
        if os.path.exists(template_json_path):
            with open(template_json_path, 'r') as f:
                template_data = json.load(f)

            steps_preview = ""
            workflow_steps = template_data.get("raw_workflow_steps", [])
            for i, step in enumerate(workflow_steps[:5], 1):  # Show first 5 steps
                steps_preview += f"{i}. {step.get('user_message', 'N/A')[:80]}...\n"

            if len(workflow_steps) > 5:
                steps_preview += f"... and {len(workflow_steps) - 5} more steps\n"

            return template_summary, steps_preview, f"✅ Template loaded: {len(workflow_steps)} steps found"
        else:
            return template_summary, "", "❌ Template file not found"

    except Exception as e:
        return f"❌ Error loading template: {str(e)}", "", "Failed to load"

def start_template_application_session(template_id, new_rent_roll_file):
    """Start applying template to new rent roll"""
    if not template_id:
        return "❌ Please select a template first"

    if not new_rent_roll_file:
        return "❌ Please upload a new rent roll file"

    try:
        # Load the new rent roll
        new_df = pd.read_excel(new_rent_roll_file.name)

        # Start application session
        session_id, status = template_app_engine.start_template_application(
            template_id=template_id,
            new_rent_roll_file=new_rent_roll_file.name,
            new_rent_roll_df=new_df
        )

        if session_id:
            return f"✅ Session started: {session_id}\n\n{status}\n\n📊 New Rent Roll Info:\n• Shape: {new_df.shape}\n• Columns: {list(new_df.columns)}\n\n🎯 Ready to execute {template_app_engine.current_application['total_steps']} template steps!"
        else:
            return status

    except Exception as e:
        return f"❌ Error starting application: {str(e)}"

def execute_next_template_step():
    """Execute the next step in template application"""
    try:
        result = template_app_engine.execute_next_step_with_ai()
        return result
    except Exception as e:
        return f"❌ Error executing step: {str(e)}"

def get_template_application_status():
    """Get current application status"""
    return template_app_engine.get_application_status()

def execute_all_remaining_steps():
    """Execute all remaining steps in sequence with guaranteed finalization"""
    if not template_app_engine.current_application:
        return "❌ No active application session"

    print("🚀 Starting batch execution of all remaining steps...")

    results = []
    step_count = 0
    max_steps = 20  # Prevent infinite loops

    initial_total_steps = template_app_engine.current_application["total_steps"]
    initial_current_step = template_app_engine.current_application["current_step"]

    print(f"📊 Will execute steps {initial_current_step + 1} through {initial_total_steps}")

    # Execute all remaining steps
    while (template_app_engine.current_application and
           template_app_engine.current_application["current_step"] < template_app_engine.current_application["total_steps"] and
           step_count < max_steps):

        current_step_before = template_app_engine.current_application["current_step"]
        step_result = template_app_engine.execute_next_step_with_ai()
        current_step_after = template_app_engine.current_application["current_step"]

        print(f"🔄 Executed step {current_step_after}/{initial_total_steps}")
        results.append(f"Step {current_step_after}: {step_result[:100]}...")
        step_count += 1

        # Safety check: if step didn't advance, break to avoid infinite loop
        if current_step_before == current_step_after:
            print("⚠️ Step didn't advance, breaking loop")
            break

    # ✅ GUARANTEED FINALIZATION: Always check if we need to finalize
    if template_app_engine.current_application:
        current_step = template_app_engine.current_application["current_step"]
        total_steps = template_app_engine.current_application["total_steps"]

        print(f"🔍 Final check - Current step: {current_step}, Total steps: {total_steps}")

        if current_step >= total_steps:
            print("✅ All steps completed, triggering finalization...")
            try:
                finalization_result = template_app_engine._finalize_application()
                results.append(f"✅ FINALIZATION SUCCESS: {finalization_result[:200]}...")
                print("🎉 Finalization completed successfully!")
            except Exception as e:
                error_msg = f"❌ Finalization failed: {str(e)}"
                results.append(error_msg)
                print(f"💥 Finalization error: {e}")
        else:
            incomplete_msg = f"⚠️ Not all steps completed: {current_step}/{total_steps}"
            results.append(incomplete_msg)
            print(incomplete_msg)
    else:
        results.append("⚠️ Application session ended unexpectedly")
        print("💥 Application session is None - may have been finalized already")

    # Create final summary
    final_summary = "\n".join(results[-4:])  # Show last 4 results

    if step_count >= max_steps:
        final_summary += f"\n\n⚠️ Stopped after {max_steps} steps to prevent timeout"

    # Add file location information
    final_summary += f"\n\n📁 Files should be saved in:"
    final_summary += f"\n  • template_applications/{template_app_engine.current_application['session_id'] if template_app_engine.current_application else 'unknown'}_final_result.csv"
    final_summary += f"\n  • rent_roll_versions/rent_roll_v_*_template_result.csv"

    print("🏁 Batch execution completed")
    return final_summary

# Add this new tab to your Gradio interface - place this BEFORE the run section

def add_template_application_tab():
    """Add the Template Application tab to the Gradio interface"""

    with gr.Tab("Apply Template"):
        gr.Markdown("""
        ### 🎯 Apply Saved Templates to New Rent Rolls

        Use your saved templates to automatically process similar rent roll files.
        The system will adapt the template steps to work with your new data.
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("#### 1. Select Template")
                template_id_select = gr.Textbox(
                    label="Template ID",
                    placeholder="e.g., template_20250526_143022",
                    lines=1
                )
                load_template_btn = gr.Button("📂 Load Template", variant="secondary")

                gr.Markdown("#### 2. Upload New Rent Roll")
                new_rent_roll_file = gr.File(
                    label="New Rent Roll File (.xlsx, .xls)",
                    file_types=[".xlsx", ".xls"]
                )

                start_application_btn = gr.Button("🚀 Start Application", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("#### Template Details")
                template_details_display = gr.HTML(label="Template Information")

                gr.Markdown("#### Workflow Steps Preview")
                template_steps_preview = gr.Textbox(
                    label="Steps to Execute",
                    lines=8,
                    interactive=False
                )

        gr.Markdown("---")

        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("#### 3. Execute Template Steps")

                with gr.Row():
                    execute_next_btn = gr.Button("▶️ Execute Next Step", variant="primary")
                    execute_all_btn = gr.Button("⏭️ Execute All Steps", variant="secondary")
                    status_btn = gr.Button("📊 Check Status")

                application_status = gr.Textbox(
                    label="Application Status & Results",
                    lines=15,
                    interactive=False
                )

            with gr.Column(scale=1):
                gr.Markdown("#### Progress Tracking")

                progress_info = gr.HTML(
                    label="Current Progress",
                    value="<p>No active session</p>"
                )

                gr.Markdown("""
                #### 💡 How It Works:
                1. **Select Template**: Choose a saved template
                2. **Upload File**: New rent roll to process
                3. **Auto-Adaptation**: GPT-4.1 adapts each step
                4. **Claude Execution**: Claude 3.7 runs the code
                5. **Step-by-Step**: Execute one or all steps
                6. **Results**: Get processed rent roll

                #### ⚙️ AI Workflow:
                - **GPT-4.1**: Analyzes & adapts template steps
                - **Claude 3.7**: Generates & executes code
                - **Auto-Mapping**: Matches columns intelligently
                - **Error Recovery**: Handles step failures gracefully
                """)

        # Event handlers for Template Application tab
        load_template_btn.click(
            load_template_for_application,
            inputs=[template_id_select],
            outputs=[template_details_display, template_steps_preview, application_status]
        )

        start_application_btn.click(
            start_template_application_session,
            inputs=[template_id_select, new_rent_roll_file],
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        execute_next_btn.click(
            execute_next_template_step,
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        execute_all_btn.click(
            execute_all_remaining_steps,
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        status_btn.click(
            get_template_application_status,
            outputs=[application_status]
        )

In [ ]:
# Initialize the global agent state
agent_state = None
custom_css = """
.chatbot-container .message-wrap .message.bot pre {
    white-space: pre !important;
    overflow-x: auto !important;
    max-width: 100% !important;
}
.chatbot-container .message-wrap .message.bot code {
    white-space: pre !important;
}
"""

with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue"), css=custom_css) as demo:
    gr.Markdown("# Agentic Commercial Real Estate Rent Roll Analyzer")
    gr.Markdown("## Hybrid AI System: GPT-4 for Decision Making & Claude for Code Generation")

    with gr.Tab("Setup"):
        with gr.Row():
            with gr.Column():
                file_input = gr.File(label="Upload Rent Roll Excel File (.xlsx, .xls)")

                # Add separate API key inputs for OpenAI and Anthropic
                anthropic_api_key = gr.Textbox(
                    label="Anthropic API Key (Optional - for code generation)",
                    placeholder="Leave blank to use the default API key",
                    type="password"
                )

                openai_api_key = gr.Textbox(
                    label="OpenAI API Key (Optional - for decision making and text responses)",
                    placeholder="Leave blank to use the default API key",
                    type="password"
                )

                # Updated auto-analyze checkbox
                auto_analyze = gr.Checkbox(
                    label="Automatically analyze for issues using GPT-4",
                    value=True,
                    info="When checked, GPT-4 will automatically identify issues in your rent roll"
                )

                upload_button = gr.Button("Load Rent Roll & Start Chat", variant="primary")

            with gr.Column():
                result = gr.Textbox(label="Status")
                preview = gr.HTML(label="Data Preview")

    with gr.Tab("Chat"):
        # Session management buttons
        with gr.Row():
            view_versions_btn = gr.Button("View Version History")
            create_template_btn = gr.Button("🎯 Create Template from Session", variant="primary")
            end_session_btn = gr.Button("🔚 End Current Session")
            session_status_btn = gr.Button("📊 Session Status")

        data_view = gr.HTML()
        chatbot = gr.Chatbot(label="Agentic Rent Roll Analysis Chat", height=500, type="tuples")

        with gr.Row():
            with gr.Column(scale=4):
                msg = gr.Textbox(label="Your question", placeholder="Ask about the rent roll...", lines=2)
            with gr.Column(scale=1):
                send_btn = gr.Button("Send", variant="primary")

        clear_btn = gr.Button("Clear Chat History")

        # Template creation input
        with gr.Accordion("Template Creation", open=False):
            template_name_input = gr.Textbox(
                label="Template Name (Optional)",
                placeholder="e.g., 'Monthly Rent Roll Cleanup Process'",
                lines=1
            )
            template_status = gr.Textbox(label="Template Creation Status", interactive=False, lines=5)

        # Set up event handlers with proper return values for Gradio chatbot
        msg.submit(
            chat,
            inputs=[msg, chatbot],
            outputs=[chatbot]
        ).then(
            lambda: "", None, msg  # Clear the message box after sending
        )

        send_btn.click(
            chat,
            inputs=[msg, chatbot],
            outputs=[chatbot]
        ).then(
            lambda: "", None, msg  # Clear the message box after sending
        )

        clear_btn.click(clear_chat, None, chatbot)

        # Enhanced event handlers for session management
        view_versions_btn.click(view_dataframe_versions, None, data_view)

        create_template_btn.click(
            lambda template_name: create_template_from_current_session(template_name),
            inputs=[template_name_input],
            outputs=[template_status]
        )

        end_session_btn.click(
            end_current_session,
            outputs=[template_status]
        )

        session_status_btn.click(
            get_current_session_status,
            outputs=[template_status]
        )

    with gr.Tab("Edit Data"):
        gr.Markdown("""
        ### 📝 Edit Rent Roll Data

        You can directly edit cells in the table below, just like in Excel.
        - Click on any cell to edit it
        - Use Tab or arrow keys to navigate
        - Changes are analyzed by GPT-4.1 and recorded in your session
        - All changes are automatically saved to session history
        """)

        with gr.Row():
            with gr.Column(scale=3):
                # Version selector
                version_dropdown = gr.Dropdown(
                    label="Select Version to Edit",
                    choices=get_version_choices(),
                    value=None,
                    interactive=True
                )

            with gr.Column(scale=1):
                refresh_versions_btn = gr.Button("🔄 Refresh Versions", size="sm")
                with gr.Row():
                    load_latest_btn = gr.Button("📂 Load Latest", variant="secondary", size="sm")
                    load_version_btn = gr.Button("📂 Load Selected", variant="primary", size="sm")

        # Status display
        edit_status = gr.Textbox(label="Status", interactive=False)

        # The editable dataframe
        editable_df = gr.Dataframe(
            label="Editable Data (Click any cell to edit) - Changes tracked by AI",
            interactive=True,
            wrap=True,
            max_height=500,
            column_widths=["100px"] * 20,
        )

        # Save controls
        with gr.Row():
            with gr.Column(scale=3):
                save_description = gr.Textbox(
                    label="Description of Changes (GPT-4.1 will analyze if left blank)",
                    placeholder="e.g., 'Updated rent for units 101-105' or leave blank for AI analysis",
                    lines=2
                )

            with gr.Column(scale=1):
                save_changes_btn = gr.Button("💾 Save & Analyze Changes", variant="primary", size="lg")

        save_status = gr.Textbox(label="Save Status & AI Analysis", interactive=False, lines=8)

        # Quick actions section
        with gr.Accordion("Quick Actions", open=False):
            gr.Markdown("""
            ### Bulk Operations
            Use these buttons for common bulk edits:
            """)

            with gr.Row():
                # Add quick action buttons here in future
                gr.Button("🧹 Clean Empty Rows", size="sm", interactive=False)
                gr.Button("💵 Round All Currency", size="sm", interactive=False)
                gr.Button("📅 Fix Date Formats", size="sm", interactive=False)
                gr.Button("🔢 Recalculate Totals", size="sm", interactive=False)

        # Enhanced session tracking notice
        gr.Markdown("""
        ### 🤖 AI-Powered Change Tracking
        - **GPT-4.1 Analysis**: Every edit is analyzed for business impact
        - **Session Recording**: All changes saved to copiloting session
        - **Template Ready**: Manual edits become part of reusable workflows
        - **Quality Assurance**: AI detects data quality improvements/issues
        """)

        # Event handlers for Edit Data tab with enhanced functions
        refresh_versions_btn.click(
            refresh_version_dropdown,
            outputs=[version_dropdown]
        )

        load_latest_btn.click(
            load_latest_version_for_editing_enhanced,  # ← Enhanced function
            outputs=[editable_df, edit_status]
        )

        load_version_btn.click(
            load_specific_version_enhanced,  # ← Enhanced function
            inputs=[version_dropdown],
            outputs=[editable_df, edit_status]
        )

        save_changes_btn.click(
            save_edited_dataframe_enhanced,  # ← Enhanced function
            inputs=[editable_df, save_description],
            outputs=[save_status, editable_df]
        ).then(
            refresh_version_dropdown,  # Refresh the dropdown after saving
            outputs=[version_dropdown]
        )

        # Instructions
        gr.Markdown("""
        ---
        ### 💡 How to Use Enhanced Edit Data:
        1. **Load Data**: Click "Load Latest" or select a specific version
        2. **Edit Cells**: Click on any cell and type to edit (just like Excel!)
        3. **Navigate**: Use Tab, Enter, or arrow keys to move between cells
        4. **Save Changes**: Enter a description (optional) and click "Save & Analyze Changes"
        5. **AI Analysis**: GPT-4.1 will analyze your changes and provide insights

        ### ⚠️ Enhanced Features:
        - **Automatic Analysis**: AI understands what you changed and why
        - **Business Impact**: Get insights on how changes affect rent calculations
        - **Session Integration**: All edits become part of your copiloting history
        - **Template Building**: Manual edits are included in reusable templates
        - **Quality Checks**: AI warns if changes might impact data quality
        """)

    # NEW TAB: Apply Template
    with gr.Tab("Apply Template"):
        gr.Markdown("""
        ### 🎯 Apply Saved Templates to New Rent Rolls

        Use your saved templates to automatically process similar rent roll files.
        The system will adapt the template steps to work with your new data.
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("#### 1. Select Template")
                template_id_select = gr.Textbox(
                    label="Template ID",
                    placeholder="e.g., template_20250526_143022",
                    lines=1
                )
                load_template_btn = gr.Button("📂 Load Template", variant="secondary")

                gr.Markdown("#### 2. Upload New Rent Roll")
                new_rent_roll_file = gr.File(
                    label="New Rent Roll File (.xlsx, .xls)",
                    file_types=[".xlsx", ".xls"]
                )

                start_application_btn = gr.Button("🚀 Start Application", variant="primary", size="lg")

            with gr.Column(scale=2):
                gr.Markdown("#### Template Details")
                template_details_display = gr.HTML(label="Template Information")

                gr.Markdown("#### Workflow Steps Preview")
                template_steps_preview = gr.Textbox(
                    label="Steps to Execute",
                    lines=8,
                    interactive=False
                )

        gr.Markdown("---")

        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("#### 3. Execute Template Steps")

                with gr.Row():
                    execute_next_btn = gr.Button("▶️ Execute Next Step", variant="primary")
                    execute_all_btn = gr.Button("⏭️ Execute All Steps", variant="secondary")
                    status_btn = gr.Button("📊 Check Status")

                application_status = gr.Textbox(
                    label="Application Status & Results",
                    lines=15,
                    interactive=False
                )

            with gr.Column(scale=1):
                gr.Markdown("#### Progress Tracking")

                progress_info = gr.HTML(
                    label="Current Progress",
                    value="<p>No active session</p>"
                )

                gr.Markdown("""
                #### 💡 How It Works:
                1. **Select Template**: Choose a saved template
                2. **Upload File**: New rent roll to process
                3. **Auto-Adaptation**: GPT-4.1 adapts each step
                4. **Claude Execution**: Claude 3.7 runs the code
                5. **Step-by-Step**: Execute one or all steps
                6. **Results**: Get processed rent roll

                #### ⚙️ AI Workflow:
                - **GPT-4.1**: Analyzes & adapts template steps
                - **Claude 3.7**: Generates & executes code
                - **Auto-Mapping**: Matches columns intelligently
                - **Error Recovery**: Handles step failures gracefully
                """)

        # Event handlers for Template Application tab
        load_template_btn.click(
            load_template_for_application,
            inputs=[template_id_select],
            outputs=[template_details_display, template_steps_preview, application_status]
        )

        start_application_btn.click(
            start_template_application_session,
            inputs=[template_id_select, new_rent_roll_file],
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        execute_next_btn.click(
            execute_next_template_step,
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        execute_all_btn.click(
            execute_all_remaining_steps,
            outputs=[application_status]
        ).then(
            get_template_application_status,
            outputs=[progress_info]
        )

        status_btn.click(
            get_template_application_status,
            outputs=[application_status]
        )

    with gr.Tab("Template Manager"):
        gr.Markdown("""
        ### 📋 Template Management System

        Create, view, and apply reusable rent roll processing templates.
        Templates capture your complete workflow including conversations, code, and manual edits.
        """)

        with gr.Row():
            with gr.Column(scale=2):
                gr.Markdown("#### Available Templates")
                template_list = gr.HTML(label="Template List")
                refresh_templates_btn = gr.Button("🔄 Refresh Template List")

            with gr.Column(scale=2):
                gr.Markdown("#### Template Details")
                template_details = gr.HTML(label="Template Summary")

        with gr.Row():
            template_id_input = gr.Textbox(
                label="Template ID",
                placeholder="e.g., template_20250526_143022"
            )
            with gr.Column():
                view_template_btn = gr.Button("👁️ View Template", variant="secondary")
                delete_template_btn = gr.Button("🗑️ Delete Template", variant="stop")

        template_action_status = gr.Textbox(label="Status", interactive=False, lines=3)

        # Template management event handlers
        refresh_templates_btn.click(
            lambda: list_available_templates(),
            outputs=[template_list]
        )

        view_template_btn.click(
            lambda template_id: enhanced_template_manager.get_template_summary(template_id) if template_id else "Please enter a template ID",
            inputs=[template_id_input],
            outputs=[template_details]
        )

        delete_template_btn.click(
            lambda template_id: enhanced_template_manager.delete_template(template_id) if template_id else "Please enter a template ID",
            inputs=[template_id_input],
            outputs=[template_action_status]
        ).then(
            lambda: list_available_templates(),  # Refresh list after deletion
            outputs=[template_list]
        )

    # Initially hide the chat interface
    chatbot.visible = False

    # Updated upload button event with both API keys and version dropdown
    upload_button.click(
        upload_rent_roll,
        inputs=[file_input, anthropic_api_key, openai_api_key, auto_analyze],
        outputs=[result, preview, chatbot, version_dropdown]
    )

    # Updated style and help info
    gr.Markdown("""
    ## How to use this Enhanced Agentic Rent Roll Analyzer:

    ### 🚀 **NEW: Template Application System**

    #### **Workflow Overview:**
    1. **Create Templates** (Chat Tab): Work through your rent roll analysis normally
    2. **Save Templates**: Click "Create Template from Session" to save your workflow
    3. **Apply Templates** (Apply Template Tab): Use saved templates on new rent roll files
    4. **Automated Processing**: GPT-4.1 + Claude 3.7 adapt and execute each step

    ### 📋 **Step-by-Step Guide:**

    #### **Phase 1: Create Your First Template**
    1. **Setup Tab**: Upload your rent roll Excel file
    2. **Chat Tab**: Interact normally - ask questions, get analysis, make changes
    3. **Edit Data Tab**: Make any manual edits (tracked by AI)
    4. **Create Template**: Click "🎯 Create Template from Session"

    #### **Phase 2: Apply Template to New Files**
    1. **Apply Template Tab**: Select your saved template ID
    2. **Upload New File**: Choose a similar rent roll file
    3. **Start Application**: Click "🚀 Start Application"
    4. **Execute Steps**: Run "▶️ Execute Next Step" or "⏭️ Execute All Steps"

    ### 🤖 **AI Workflow in Template Application:**

    **For Each Template Step:**
    1. **GPT-4.1 Analysis**:
       - Analyzes original template step
       - Maps columns from template to new file
       - Adapts parameters and business rules
       - Creates optimized prompt for Claude

    2. **Claude 3.7 Execution**:
       - Receives adapted instructions
       - Generates appropriate Python code
       - Executes data processing
       - Returns results and updates dataframe

    3. **Validation & Progress**:
       - Validates step completion
       - Records success/failure
       - Logs detailed results
       - Moves to next step

    ### 🔧 **Key Features:**

    - **Intelligent Adaptation**: Automatically maps different column names
    - **Business Logic Preservation**: Maintains the intent of original analysis
    - **Error Recovery**: Handles failures gracefully and continues
    - **Progress Tracking**: Real-time status of template application
    - **Complete Logging**: Detailed logs of every step and decision

    ### 💡 **Use Cases:**

    - **Monthly Processing**: Apply same cleanup to each month's rent roll
    - **Property Portfolios**: Use one template across multiple properties
    - **Team Workflows**: Share proven analysis methods
    - **Quality Assurance**: Ensure consistent processing standards
    - **Time Savings**: Automate repetitive analysis tasks

    ### ⚡ **Quick Start:**

    1. Upload rent roll → Chat about analysis → Create template
    2. Get template ID from Template Manager
    3. Go to Apply Template → Enter template ID → Upload new file → Execute!

    The system transforms your one-time analysis into reusable, intelligent automation!
    """)

# Additional helper functions for Template Manager tab (keeping existing)
def list_available_templates():
    """Generate HTML list of available templates"""
    try:
        templates = enhanced_template_manager.list_templates()

        if not templates:
            return "<p>No templates available yet. Create your first template by using the 'Create Template from Session' button in the Chat tab.</p>"

        html = "<div style='max-height: 400px; overflow-y: auto;'>"

        for template in templates:
            gpt_status = "🤖 GPT-4 Analysis" if template.get('gpt4_analysis_available') else "📝 Basic Info"

            html += f"""
            <div style='border: 1px solid #ddd; margin: 10px 0; padding: 15px; border-radius: 8px; background-color: #f9f9f9;'>
                <h4 style='margin: 0 0 10px 0; color: #333;'>{template['template_name']}</h4>
                <p style='margin: 5px 0; color: #666;'><strong>ID:</strong> <code>{template['template_id']}</code></p>
                <p style='margin: 5px 0; color: #666;'><strong>Created:</strong> {template['created_date'][:10]}</p>
                <p style='margin: 5px 0; color: #666;'><strong>Source:</strong> {template['source_file']}</p>
                <p style='margin: 5px 0; color: #666;'><strong>Steps:</strong> {template['steps_count']} workflow steps</p>
                <p style='margin: 5px 0;'><span style='background-color: #e3f2fd; padding: 2px 6px; border-radius: 4px; font-size: 12px;'>{gpt_status}</span></p>
            </div>
            """

        html += "</div>"
        return html

    except Exception as e:
        return f"<p>Error loading templates: {str(e)}</p>"

<ipython-input-28-da48583de568>:58: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Agentic Rent Roll Analysis Chat", height=500, type="tuples")


In [ ]:
# Run the application
if __name__ == "__main__":
    logger.info("Starting Agentic Rent Roll Analyzer application")
    demo.launch(debug=True)
    logger.info("Application shutdown")

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://14d57cac9a50ee73f5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✓ Saved dataframe version v_20250606_002003: Initial upload - original dataset
  - CSV: rent_roll_versions/rent_roll_v_20250606_002003.csv
  - Excel: rent_roll_versions/rent_roll_v_20250606_002003.xlsx


<ipython-input-14-3df7e6be88cc>:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  {rent_roll_df.head(5).fillna('').to_html(index=False)}
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1965: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  state[block._id] = block.__class__(**kwargs)


📝 Started session recording: session_20250606_002115

==== STARTING CODE GENERATION ====
User query: for the given table add a new column Residents that classifies the tenants as "Occupied" if the corresponding lease start date is not empty or "Vacant" if the lease start date is empty.
Dataframe has 30 rows and 9 columns
Sending FIRST 50 ROWS to GPT-4.1 (sample instead of full dataset)

==== STEP 1: GENERATING PROMPT WITH GPT-4 (WITH SAMPLE OF 50 ROWS) ====

==== GPT-4 GENERATED PROMPT FOR CLAUDE ====
You are an expert Python analyst working with a rent roll dataframe that is already loaded as the variable df. The dataframe contains ALL 30 rows of real data, as described in the provided sample (see above for structure and sample content). You do NOT need to load or preview the data; df is ready for analysis.

Your task:  
Add a new column named Residents to df, classifying each row as follows:
- If the 'Lease Start Date' is NOT empty (i.e., not null/NaT), set 'Residents' to "Occupied".

Summarize the key user-guided instructions and solutions from this rent roll copiloting session.

For each step, briefly describe:

What the user wanted to accomplish (without quoting them directly).

How the request was addressed or solved by the copilot.

Avoid excessive detail, don’t repeat the user’s exact instructions, and keep each summary concise and clear.

In [ ]:
!rm -rf /content/rent_roll_versions

Copying any files generated during the session to the google drive

In [ ]:
import os
import shutil
from google.colab import drive

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

# Define the source folders (excluding sample_data)
folders_to_save = [
    'copiloting_sessions',
    'rent_roll_templates',
    'rent_roll_versions',
    'template_applications'
]

# Define the destination path in Google Drive
# You can change this path to wherever you want to save in your Drive
gdrive_destination = '/content/drive/MyDrive/CRE_AI_agent/'

# Create the destination directory if it doesn't exist
os.makedirs(gdrive_destination, exist_ok=True)
print(f"Created destination directory: {gdrive_destination}")

# Copy each folder to Google Drive
for folder in folders_to_save:
    source_path = f'/content/{folder}'
    destination_path = os.path.join(gdrive_destination, folder)

    if os.path.exists(source_path):
        print(f"Copying {folder}...")

        # Remove destination folder if it already exists
        if os.path.exists(destination_path):
            shutil.rmtree(destination_path)
            print(f"  Removed existing {folder} folder")

        # Copy the folder
        shutil.copytree(source_path, destination_path)
        print(f"  ✓ Successfully copied {folder} to Google Drive")
    else:
        print(f"  ⚠️  Warning: {folder} not found in /content/")

print("\n" + "="*50)
print("COPY OPERATION COMPLETED!")
print("="*50)
print(f"All folders have been saved to: {gdrive_destination}")
print("\nFolders copied:")
for folder in folders_to_save:
    destination_path = os.path.join(gdrive_destination, folder)
    if os.path.exists(destination_path):
        print(f"  ✓ {folder}")
    else:
        print(f"  ✗ {folder} (failed)")

# Optional: List the contents of the destination folder
print(f"\nContents of {gdrive_destination}:")
try:
    contents = os.listdir(gdrive_destination)
    for item in contents:
        print(f"  📁 {item}")
except:
    print("  Unable to list contents")